# Part III 고급 AI 트레이딩 및 리스크 관리 응용

이 책의 이 부분에서는 이전 장에서 배운 AI 및 머신러닝 기법의 실제 적용에 대해 더 깊이 있게 다룹니다. 우리는 현대 알고리즘 트레이딩 전략의 기본 구성 요소를 실용적이고 단순화된 예제를 통해 식별하고 설명했습니다. 이러한 예제들은 여러분이 자신만의 트레이딩 알고리즘을 생성하고 관리할 수 있을 만큼의 경험과 자신감을 제공하도록 설계되었습니다.

## 소스 코드 시작하기

이 부분의 예제와 연습에 사용된 모든 소스 코드는 GitHub 저장소에서 확인할 수 있습니다. 다음 단계를 따라 코드를 사용하세요:

1. 설명 읽기: 소스 코드를 최대한 활용하는 방법에 대한 설명은 다음 링크에서 확인할 수 있습니다.  
   https://github.com/QuantConnect/HandsOnAITradingBook

2. 저장소 클론하기: 터미널이나 명령 프롬프트를 열고 아래 명령어를 실행하여 저장소를 로컬 머신에 클론합니다.  
   `git clone https://github.com/QuantConnect/HandsOnAITradingBook.git`

3. 코드 탐색하기: 저장소 안에는 장과 주제별로 구성된 폴더가 있습니다. 각 폴더에는 자세한 코드 예제와 설명이 포함된 Jupyter 노트북이나 Python 스크립트가 들어 있습니다.

4. QuantConnect에 모델 배포하기: QuantConnect 계정에 로그인한 후 Learning Center로 이동하여 책의 소스 코드를 자신의 프로젝트로 클론하세요. 그러면 코드 변경이 트레이딩 전략 결과에 어떤 영향을 주는지 즉시 확인할 수 있습니다.



# Chapter 6 Applied Machine Learning

## Example 1—ML Trend Scanning with MLFinlab

| 예측 대상 | 가격 방향 |
| :-- | :-- |
| 기술 | 회귀/분류 |
| 자산군 | 암호화폐 |
| 난이도 | 쉬움 |
| 유형 | 리서치 노트북 |
| 소스 코드 | gnt.co/book-example1 |

### Summary

MLFinLab 라이브러리의 트렌드 스캐닝 패키지를 사용하여 가격 트렌드(하락/무추세/상승)를 감지하고 주요 추세에 투자합니다.

### Motivation

정확한 트렌드 예측은 다음과 같은 알고리즘 트레이딩 전략에 매우 중요합니다:

1. **전략 선택** – 정확한 트렌드 식별은 시장 상황에 가장 적합한 트레이딩 전략을 선택하는 데 도움을 줍니다. 각기 다른 시장 국면에는 서로 다른 전략이 유리합니다.  
2. **최적의 진입/청산 시점 파악** – 시장 추세를 식별하면 가장 수익성 있는 진입 및 청산 시점을 결정할 수 있습니다.  
3. **추세 추종 전략** – 트레이더는 트렌드를 정확하게 식별하고 이를 따라감으로써 시장 흐름을 탈 수 있습니다.  
4. **역추세 전략** – 과도하게 진행된 트렌드를 식별하고 반전을 예상하여 시장 조정에서 수익을 낼 수 있습니다.

### Model

BTCUSD 종가 시계열을 모델링합니다.

- **예측 대상:** $\quad-1, 0, 1$ (하락 추세, 무추세, 상승 추세)
- **레이블:** MLFinLab의 `trend_scanning_labels` 메서드 사용

`trend_scanning_labels`는 MLFinLab의 트렌드 스캐닝 패키지의 구성요소로, 본 도서를 위해 오픈소스화되었으며, Marcos Lopez de Prado에 의해 소개되었습니다 ([참고 링크](https://qnt.co/book-trend-scanning)).

```python
def trend_scanning_labels(
    price_series: pd.Series,
    t_events: list = None,
    observation_window: int = 20,
    metric: str = 't_value',
    look_forward: bool = True,
    min_sample_length: int = 5,
    step: int = 1
) -> pd.DataFrame
```

이 메서드는 시장 국면(하락/무추세/상승)의 분류에 유용합니다:

* 입력된 관측치 벡터(예: 종가)를 -1, 0, 1로 매핑합니다.
* 다양한 관측 윈도우에 대해 선형 회귀 모델을 생성합니다:

  * 독립 변수: $l, l=0, \ldots, L-1$, 여기서 $L=20$
  * 종속 변수: $l$ 스텝 후의 종가
* 각 모델에 대해 회귀 계수의 **t-값**을 계산하고, 가장 큰 절댓값의 t-값을 선택합니다.
* t-값의 부호가 최종 **추세 레이블**이 됩니다.

리턴되는 DataFrame 주요 컬럼:

* **Index**: 레이블의 타임 인덱스
* **t1**: 레이블 종료 시점 (추세가 가장 강했던 종료 시점)
* **t-value**: 추세의 방향 (양수 = 상승)
* **ret**: 시작 시점부터 종료 시점까지의 가격 변화율
* **bin**: 가격 변화의 부호 기반 이진 레이블

예시 차트 (Figure 6.1)에서는 다음을 시각화합니다:

* 파란색: 종가 시계열
* 빨간색: 가장 큰 t-값 시계열
* 초록색 마커: 상승 추세
* 노란색 마커: 무추세
* 빨간색 마커: 하락 추세

<img src="./images/fig_06_01.png" width=800>

**Figure 6.1**: 선형 회귀 모델을 이용한 트렌드 식별 예시

```python
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from trend_scanning_labels import trend_scanning_labels

# 예제용 데이터 생성 (종가 시계열)
np.random.seed(0)
dates = pd.date_range(start='2020-01-01', periods=100)
closes = pd.Series(np.random.randn(100).cumsum() + 100, index=dates)

# 관측 윈도우 설정
L = 20

# 트렌드 레이블 생성
trend_labels_df = trend_scanning_labels(
    closes,
    closes.index,
    look_forward=False,
    observation_window=L,
    min_sample_length=5
)

# t-값 및 추세 레이블 추출
t_values = trend_labels_df['t_value'].dropna()
trend_labels = trend_labels_df['bin'].dropna()

# 시각화
fig, ax1 = plt.subplots(figsize=(12, 6))

# 종가 플롯
color = 'tab:blue'
ax1.set_xlabel('Date')
ax1.set_ylabel('Closing Prices', color=color)
ax1.plot(closes.index, closes, color=color, label='Closing Prices')
ax1.tick_params(axis='y', labelcolor=color)

# t-value용 보조 y축 생성
ax2 = ax1.twinx()
color = 'tab:red'
ax2.set_ylabel('t-values', color=color)
ax2.plot(t_values.index, t_values, color=color, label='t-values')
ax2.tick_params(axis='y', labelcolor=color)

# 트렌드 레이블 추가
colors = {1: 'green', 0: 'yellow', -1: 'red'}
for i in range(len(trend_labels)):
    label = trend_labels.iloc[i]
    date = trend_labels.index[i]
    ax1.scatter(date, closes.loc[date], color=colors[label], s=30, label=f'Trend Label {label}')

# 중복 레이블 제거 후 범례 표시
handles, labels = ax1.get_legend_handles_labels()
unique_labels = dict(zip(labels, handles))
ax1.legend(unique_labels.values(), unique_labels.keys())

# 제목 추가
fig.suptitle('Trend Identification using Linear Regression Models', fontsize=16)
fig.tight_layout()
```

### Trading Universe 

비트코인(BTCUSD)에만 투자한다고 가정합니다.

### Portfolio Construction 

| Model Training Time | 하루에 한 번 |
| :-- | :-- |
| Portfolio Rebalancing | 하루에 한 번 |
| Time |  |
| Portfolio Weights | 트렌드가 상승일 경우 비트코인에 $100\%$ 투자, 그 외에는 $0\%$ 투자 |

비트코인의 포트폴리오 비중은 다음과 같습니다:

- 트렌드가 상승일 경우 $100\%$
- 그 외의 경우 $0\%$

### Trading Logic

비록 단순하지만, 트렌드가 상승할 때 포트폴리오의 $100\%$를 BTCUSD에 투자하고 그렇지 않을 때는 $100\%$를 USD로 보유하는 이 전략은 단순한 비트코인 매수 후 보유 전략보다 더 나은 결과를 제공합니다 (Figure 6.2).

```python
from trend_scanning_labels import trend_scanning_labels
from backtestlib import rough_daily_backtest

trend = trend_scanning_labels(
    closes,
    closes.index,
    observation_window=20,
    look_forward=False,
    min_sample_length=5
)['bin'].dropna()

portfolio_weights = pd.DataFrame(trend.where(trend != -1, 0))
portfolio_weights.columns = [symbol]

rough_daily_backtest(qb, portfolio_weights)
```

여기서 `closes`는 BTCUSD의 종가를 저장한 DataFrame이며, 다음과 같이 로드합니다:

```python
qb = QuantBook()
symbol = qb.add_crypto("BTCUSD", Resolution.DAILY).symbol
history = qb.history(symbol, datetime(2016, 1, 1), datetime(2024, 1, 1))
closes = history.loc[symbol]['close']
```

<img src="./images/fig_06_02.png" width=800>

Figure 6.2 Equity curves of the strategy and selected benchmark.

### Implementation Insights

이 전략은 QuantConnect의 Research Notebook에서 구현됩니다.
예측 정확도는 다음과 같이 정의됩니다: 정확히 예측한 수 / 전체 예측 수

```python
correct_predictions = np.sign(
    closes.pct_change().shift(-1).loc[trend.index].iloc[:-1]
) == np.sign(trend.iloc[:-1])
```

이 설정에서의 정확도는 $51.2%$로, 무작위 추정보다 약간 높은 수준입니다. 그럼에도 불구하고 이 트레이딩 전략은 단순히 BTCUSD를 보유하는 전략보다 훨씬 더 수익성이 높습니다 (Figure 6.3).

<img src="./images/fig_06_03.png" width=800>

Figure 6.3 Predictions and observed log price.

---


## Example 2—Factor Preprocessing Techniques for Regime Detection  

| Predicting | 미래 수익률 |
| :-- | :-- |
| Technology | 분류 및 PCA |
| Asset Class | 미국 지수 |
| Difficulty | 쉬움 |
| Type | 리서치 노트북 |
| Source Code | qnt.co/book-example2 |

### Summary

이 예제는 머신러닝 모델을 훈련하기 전에 팩터 값에 다양한 변환을 적용했을 때 예측 성능에 어떤 영향을 미치는지를 보여줍니다.

### Motivation

금융 시장의 원시 데이터는 일반적으로 노이즈가 많고, 왜곡되어 있으며 다양한 분포와 스케일을 포함하고 있어 변수 간의 적절한 관계를 찾기 어렵습니다. 적절한 전처리 기법을 적용하면 머신러닝 모델이 더 적합한 형태의 데이터 표현을 학습하게 되어 예측 성능이 크게 향상됩니다.

### Model 

| Model | 랜덤 팩터 |
| :-- | :-- |
| Features |  |
| Predicted | SPY 시장의 다음 주 오픈 대비 오픈 수익률이 양수인 경우 레이블 1 |
| Label | 그 외의 경우는 레이블 0 |
| Model | 멀티클래스 랜덤 포레스트 분류기 |

우리는 다양한 팩터 전처리 기법이 모델 성능에 미치는 영향을 살펴보며, 테스트 세트에서의 정확도(OOS accuracy)를 기준으로 비교합니다.

모델은 멀티클래스 랜덤 포레스트 분류기로, 각 입력 팩터가 두 클래스 중 어느 쪽에 속할 확률을 예측합니다:

- 클래스 레이블 1: 주간 수익률이 양수
- 클래스 레이블 0: 그 외

다음의 팩터 전처리 기법을 비교합니다:

- 원시 팩터
- 정상성 처리된 팩터
- 표준화된 팩터
- PCA 변환된 팩터

### Trading Universe

2000년 1월 1일부터 2024년 1월 1일까지 SPY ETF의 일간 데이터를 사용합니다.

### Implementation Insights 

### Step 1: Build the label

레이블은 아래와 같이 `history` 데이터프레임에서 계산됩니다 (Figure 6.4 참조):

```python
label = history.loc[symbol]['open'].pct_change(5).shift(-5).dropna().apply(
    lambda x: int(x > 0)
)
```

<img src="./images/fig_06_04.png" width=800>

Figure 6.4 미래 수익률 방향에 따라 각 종가를 색으로 나타낸 산점도

레이블 분포는 Figure 6.5와 같이, 레이블 1이 더 자주 발생함을 보여줍니다.

<img src="./images/fig_06_05.png" width=800>

Figure 6.5 각 레이블의 빈도를 나타낸 막대 차트

### Step 2: Define the factors used to predict the label

모델에 입력할 팩터를 정의합니다. 단순화를 위해 랜덤 팩터를 도입하고, 이 중 하나는 비정상(nonstationary)하게 만듭니다:

```python
import numpy as np

np.random.seed(2)
num_factors = 4
num_samples = len(label)
factors = np.random.rand(num_samples, num_factors)

# 하나의 팩터를 비정상성으로 만듦
factors[:, -1] = factors[:, -1].cumsum()
```

### Step 3: Define the performance benchmark for evaluating different factors

다양한 팩터의 성능을 평가하는 `oos_accuracy` 메서드는 다음과 같은 절차로 작동합니다:

1. 학습: 전체 데이터의 75%를 학습, 나머지 25%를 테스트에 사용
2. 모델: LightGBM 멀티클래스 분류 모델 학습
3. 성능 평가: 테스트 데이터에 대한 예측 정확도 출력

```python
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import plotly.graph_objects as go

def oos_accuracy(factors, label):
    X_train, X_test, y_train, y_test = train_test_split(
        factors, label, test_size=0.25, shuffle=False
    )
    model = lgb.train(
        {
            'seed': 1234,
            'verbose': -1,
            'boosting_type': 'rf',
            'feature_fraction': 0.8,
            'objective': 'multiclass',
            'num_class': 2,
            'bagging_freq': 5,
            'bagging_fraction': 0.8,
            'is_unbalanced': True
        },
        train_set=lgb.Dataset(
            data=X_train,
            label=y_train,
            free_raw_data=True
        ).construct()
    )
    predictions = model.predict(X_test)
    x = list(range(len(predictions)))
    go.Figure(
        [
            go.Scatter(x=x, y=predictions[:, 0], name="0"),
            go.Scatter(x=x, y=predictions[:, 1], name="1")
        ],
        layout=dict(
            title="Probability of Each Label<br><sup>Class 1 gets a greater probability because the SPY has an upward bias</sup>",
            xaxis_title="Date",
            yaxis_title="Probability"
        )
    ).show()
    
    y_hat = predictions.argmax(axis=1)
    print(f"Accuracy: {round(accuracy_score(y_hat, y_test), 4)}")
```


### Step 4: Evaluate the model's performance on the raw factors. 

첫 번째 테스트는 원시 팩터 값을 그대로 사용하는 것입니다. OOS 정확도는 0.6101입니다 (Figure 6.6 참조).

```python
oss_accuracy(factors, label)
```

<img src="./images/fig_06_06.png" width=800>

Figure 6.6 모델 예측으로부터 각 레이블에 대한 확률을 나타내는 시계열 그래프

### Step 5: Test model accuracy using stationary factors.

Lopez De Prado (2018)는 "지도학습 알고리즘은 일반적으로 정상성(stationary)을 가진 특성을 요구한다"고 설명합니다.
팩터들이 정상적인지 여부를 확인하기 위해, Augmented Dickey-Fuller 테스트를 수행합니다.

```python
from statsmodels.tsa.stattools import adfuller

for factor_idx in range(num_factors):
    factor = factors[:, factor_idx]
    test_results = adfuller(factor, maxlag=1, regression='c', autolag=None)
    output = "Stationary" if test_results[1] <= 0.05 else "Not stationary"
    print(f"Factor {factor_idx}: {output}")
```

출력 결과에 따르면, 우리가 설계한 대로 하나를 제외한 모든 팩터가 이미 정상성을 띕니다:

```
Factor 0: Stationary
Factor 1: Stationary
Factor 2: Stationary
Factor 3: Not stationary
```

### Step 6: Make factors stationary, if necessary.

원시 팩터가 정상성이 없다면, 이를 변환하여 정상성 있게 만들 수 있습니다.
우리의 경우, fractional differentiation을 적용한 팩터를 사용한 결과 OOS 정확도는 0.5849입니다 (Figure 6.7).

```python
oss_accuracy(stationary_factors.values, label)
```

<img src="./images/fig_06_07.png" width=800>

Figure 6.7 정상성 팩터 기반 예측 결과에서 각 레이블의 확률을 나타내는 시계열 그래프

### Step 7: Introduce standardization of factors.

다음으로, 팩터를 평균이 0이고 표준편차가 1이 되도록 표준화합니다.
표준화는 서로 다른 단위나 스케일을 갖는 팩터를 모델에서 비교 가능하게 만들며, PCA 적용 전에도 필수적인 전처리 과정입니다.

```python
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
standardized_factors = scaler.fit_transform(stationary_factors)
```

표준화된 팩터를 사용한 결과 OOS 정확도는 0.5882입니다 (Figure 6.8).

```python
oss_accuracy(standardized_factors, label)
```

<img src="./images/fig_06_08.png" width=800>

Figure 6.8 표준화된 팩터 기반 예측 결과에서 각 레이블의 확률을 나타내는 시계열 그래프

### Step 8: Introduce PCA factorization.

다음으로, PCA를 적용하여 차원을 축소한 후 예측합니다.

```python
from sklearn.decomposition import PCA

pca = PCA(random_state=0)
principal_components = pca.fit_transform(standardized_factors[1:, :])
```

차원 축소된 데이터로 훈련 및 테스트를 수행한 결과 OOS 정확도는 0.5962입니다 (Figure 6.9).

```python
oss_accuracy(principal_components, label)
```

<img src="./images/fig_06_09.png" width=800>

Figure 6.9 주성분 기반 예측 결과에서 각 레이블의 확률을 나타내는 시계열 그래프

### Step 9: Compare the OSS accuracies.

이 특정 사례에서는 팩터가 무작위 숫자이기 때문에 어떤 전처리 기법도 원시 데이터보다 유의미한 성능 향상을 제공하지 못합니다.
가장 적절한 전처리 방식은 데이터 및 상황에 따라 달라집니다.

| Factor Preprocessing Technique | OSS Accuracy |
| :----------------------------- | :----------: |
| Raw Data                       |    0.6101    |
| Stationary Factors             |    0.5849    |
| Standardized Factors           |    0.5882    |
| PCA Factors                    |    0.5962    |


---



## Example 3—Reversion vs. Trending: Strategy Selection by Classification  

| Predicting | 변동성 |
| :-- | :-- |
| Technology | 신경망 |
| Asset Class | 미국 주식 |
| Difficulty | 쉬움 |
| Type | 리서치 노트북 |
| Source Code | qnt.co/book-example3 |

### Summary

신경망을 활용하여 다음 거래일이 모멘텀 노출에 유리할지, 되돌림 리스크 노출에 유리할지를 예측합니다. 이는 모멘텀 vs. 되돌림 구간을 분류하고, 현재 변동성 국면(Volatility, VIX, ATR, RSI 등)에 따라 SPY 지수와 TLT 채권 간 전략을 전환하는 방식입니다.

### Motivation 

모멘텀 전략은 주가가 일정 기간 동안 같은 방향으로 움직인다는 경향을 활용합니다. S&P 500을 추종하는 SPY 인덱스를 보유하면 시장의 전반적인 추세에 노출됩니다. 시장이 추세를 보일 경우 SPY가 그 방향으로 계속 움직일 가능성이 높습니다.

반면, 되돌림 리스크 전략은 자산 가격이 일정 시간 동안 평균에서 벗어난 뒤 다시 평균으로 회귀한다고 보는 전략입니다. TLT는 장기 미국 국채를 추종하며, 되돌림 국면에서 방어적인 성격을 지닙니다. 시장이 하락 전환의 신호를 보일 때 TLT를 보유하면 방어적 포지션이 됩니다.

변동성이 높은 시장 환경에서 다음 날이 모멘텀에 유리할지 되돌림에 유리할지를 예측하는 능력은, 투자자가 동적으로 포트폴리오를 조정할 수 있게 하여, 상승장에서는 수익을 극대화하고, 되돌림 장에서는 손실을 최소화할 수 있도록 도와줍니다.

### Model

모델 입력 특성:

- SPY의 21일 RSI (상대 강도 지수)
- SPY의 21일 ATR (평균 진폭), 21일 단순이동평균으로 정규화됨
- SPY의 21일 일간 수익률 표준편차
- VIX 지수 값 (시카고 옵션 거래소 변동성 지수)

예측 레이블:  
레이블 1 = 모멘텀 국면, 레이블 0 = 되돌림 국면  
모델: 순차 신경망 (Sequential Neural Network)

Kenneth R. French의 웹사이트에서 제공하는 모멘텀 및 단기 반전 팩터 시계열을 사용하여 다음 날이 모멘텀 노출에 유리한지를 판단합니다.

```python
# Label 1 = Momentum regime; Label 0 = Reversion regime.
label = (momentum > reversion.reindex(momentum.index)).dropna().astype(int)
label = label.shift(-1).dropna()
```

변동성 국면을 나타내는 지표들로 구성된 행렬을 만들어 신경망에 입력합니다.

신경망은 Keras를 사용하여 두 개의 밀집(Dense) 레이어로 구성됩니다:

* 입력: 4개 지표
* 첫 번째 은닉층: 8개의 노드, ReLU 활성화
* 출력층: 1개 노드, 시그모이드 함수 → 출력값은 \[0, 1] 사이로, 모멘텀일 확률을 나타냄

```python
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras.utils import set_random_seed
from sklearn.metrics import accuracy_score

set_random_seed(0)

model = keras.Sequential([
    layers.Input(shape=(X.shape[1],)),
    layers.Dense(8, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])
```

옵티마이저로는 `adam`을 사용하고, 손실 함수는 이진 분류용 `binary_crossentropy`입니다.

```python
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)
```

`adam`은 확률적 경사 하강법(SGD)의 확장으로 학습률을 자동 조정하며, 메모리 사용이 적고 효율적인 알고리즘입니다.
`binary_crossentropy`는 실제값과 예측값의 오차를 측정하는 이진 분류용 손실 함수이며, 모델이 이 값을 최소화하도록 훈련됩니다.

모델은 100 에폭(epoch) 동안 훈련 데이터에 대해 학습됩니다.

```python
model.fit(X_train, y_train, epochs=100, verbose=0)
```

모델 성능은 훈련셋(in-sample)과 테스트셋(out-of-sample)으로 평가합니다.

```python
# 훈련셋 정확도
y_hat = (model.predict(X_train, verbose=0) > 0.5).astype(int)
print(f"In-sample accuracy: {accuracy_score(y_hat, y_train)}")

# 테스트셋 정확도
y_hat = (model.predict(X_test, verbose=0) > 0.5).astype(int)
print(f"Out-of-sample accuracy: {accuracy_score(y_hat, y_test)}")
print(f"OOS Label counts: {np.unique(y_hat, return_counts=True)[1]}")
```

결과:

* In-sample accuracy: 0.554843141744905
* Out-of-sample accuracy: 0.5276615527661552
* OOS label counts: \[592 1559]

이 전략의 벤치마크는 SPY ETF의 상승일 비율로, 2014\~2024년 기간 동안 약 54.4%였습니다.
모델의 훈련셋 정확도는 55.48%로 벤치마크 및 무작위 추정보다 높습니다. 테스트셋 정확도는 52.77%로, 다소 낮지만 일반적인 현상이며 모델이 무작위보다 약간 우수함을 보여줍니다. 더 나은 입력 데이터 또는 더 깊은 신경망 구조, 다양한 팩터 추가가 필요할 수 있습니다.


### Trading Universe 

SPY와 TLT 두 자산만을 고려합니다.

### Portfolio Construction

| Model Training <br> Time | 백테스트 시작 전에 한 번만 학습 |
| :-- | :-- |
| Portfolio <br> Rebalancing Time | 매일 장 시작 시점에 리밸런싱 |
| Portfolio Weights | 모델이 현재 시장이 모멘텀에 유리하다고 예측하면 SPY에 $100\%$ 투자, 그렇지 않으면 TLT에 $100\%$ 투자 |

전략 규칙은 다음과 같습니다:

- 모델이 현재 시장 국면이 모멘텀에 유리하다고 판단하면 SPY 매수
- 그렇지 않으면 TLT 매수
- 장 시작 시점에 리밸런싱 수행

### Trading Logic 

포트폴리오는 매일 장 시작 시점에 리밸런싱됩니다. 오늘이 모멘텀 시장이라고 모델이 예측한 경우(`predictions == 1`) SPY에 투자하고, 그렇지 않으면 TLT에 투자합니다.

```python
portfolio_weights = pd.DataFrame({
    spy_symbol: predictions,
    qb.add_equity("TLT", Resolution.DAILY).symbol: abs(predictions - 1)
})
```

### Implementation Insights

RSI와 ATR 특성은 QuantConnect의 표준 Indicator 클래스를 사용해 구현하며, 일간 수익률의 표준편차(STD)는 사용자 정의 인디케이터로 설정합니다.

```python
spy_symbol = qb.add_equity("SPY", Resolution.DAILY).symbol

# 파라미터 설정
period = 21
start_date = datetime(1990, 1, 1)
end_date = datetime(2024, 1, 1)

# RSI 인디케이터
rsi = qb.Indicator(
    RelativeStrengthIndex(period), spy_symbol,
    start_date, end_date
)['relativestrengthindex']

# ATR 인디케이터 (가격의 21일 SMA로 정규화)
atr = qb.Indicator(
    AverageTrueRange(period, MovingAverageType.SIMPLE),
    spy_symbol, start_date, end_date
)['averagetruerange']

atr /= qb.Indicator(
    SimpleMovingAverage(period),
    spy_symbol, start_date, end_date
)['simplemovingaverage']

# 일간 수익률 표준편차 (STD) 인디케이터
roc = RateOfChange(1)
std = IndicatorExtensions.of(StandardDeviation(period), roc)

history = qb.history[TradeBar](spy_symbol, start_date, end_date)
window = {column: [] for column in ['time', 'std_of_roc']}

def update_window(sender, updated):
    if not sender.is_ready:
        return
    window['time'].append(updated.end_time)
    window['std_of_roc'].append(updated.value)

std.updated += update_window

for bar in history:
    roc.update(bar.end_time, bar.close)

std = pd.DataFrame(window).set_index('time')['std_of_roc']

# VIX 인덱스 값 불러오기
vix_symbol = qb.add_data(CBOE, "VIX", Resolution.DAILY).symbol
vix = qb.history(vix_symbol, start_date, end_date).loc[vix_symbol]['close']
vix.name = "vix"

# 모든 팩터를 하나의 매트릭스로 결합
factors = pd.concat([rsi, atr, std, vix], axis=1).dropna()
```

---


## Example 4—Alpha by Hidden Markov Models  

| Predicting | 변동성 국면 |
| :-- | :-- |
| Technology | 히든 마르코프 모델(HMM) 클러스터링 |
| Asset Class | 미국 주식, 주식 옵션, 지수 옵션 |
| Difficulty | 쉬움–중간 |
| Type | 전체 전략 |
| Source Code | qnt.co/book-example4 |

### Summary

마르코프 회귀 모델을 사용하여 시장의 변동성 국면(저변동성 또는 고변동성)을 예측하고, 이에 따라 자금을 배분합니다. 세 가지 포트폴리오 구성 알고리즘을 제공합니다: 처음엔 SPY/TLT ETF에 투자하고, 이후 SPY 스트래들 옵션 전략을 추가하며, 마지막으로 SPX 지수 옵션을 활용한 전략으로 확장합니다.

### Motivation

변동성은 자산의 가격이 얼마나 자주, 얼마나 크게 변동하는지를 의미하며, 자산의 위험 및 수익 구조에 직접적인 영향을 줍니다.

변동성 국면을 예측하는 것은 효과적인 트레이딩 전략의 핵심 요소입니다.

변동성 국면은 보통 다음과 같이 분류됩니다:

- **저변동성 국면**: 가격이 안정적이며 예측 가능한 움직임을 보이고 상승 추세가 우세함. 주식 같은 성장 자산에 유리.
- **고변동성 국면**: 가격의 불규칙한 급등락이 특징이며, 시장 불확실성이나 하락장의 신호일 수 있음. 채권이나 옵션 같은 방어적 자산이 선호됨.

이러한 국면을 예측하면 투자자는 자산 배분을 동적으로 조정할 수 있습니다 (Figure 6.10).

<img src="./images/fig_06_10.png" width=800>

Figure 6.10 SPY 가격과 변동성, 그리고 고/저 변동성 구간 시각화

SPY 종가와 변동성(21일 기준 수익률의 연율화 표준편차)을 시각화하여, 시장 변동성이 시간에 따라 어떻게 변하는지 보여줍니다. 변동성이 75% 분위수를 초과하면 고변동성(red shading), 25% 분위수 이하이면 저변동성(green shading)으로 표시됩니다.

마르코프 회귀 모델을 통해 고/저 변동성 환경을 정확히 예측하고, 이를 바탕으로 수익 최적화 및 리스크 관리를 위한 자산 배분이 가능합니다.

### Model 

| Model | SPY의 일간 수익률 |
| :-- | :-- |
| Features |  |
| Predicted | 현재 시장 국면: 0 = 저변동성, 1 = 고변동성 |
| Label |  |
| Model | 마르코프 전환 회귀 (Markov-switching regression) |

이 모델은 두 개의 뚜렷한 변동성 국면(저변동성, 고변동성)을 탐지하기 위해 마르코프 전환 회귀를 사용합니다.  
모델 훈련을 위해 S&P 500의 최근 3년 일간 수익률을 사용합니다.

### Trading Universe

이 예제에서는 3가지 자산 배분 알고리즘을 구현합니다.

1. **ETF 기반 전략**: SPY (S&P 500) – 상승장 수익 추구, TLT (미국 국채) – 하락장 방어
2. **SPY 스트래들 옵션 전략**: 방향성 상관없이 변동성이 높을 때 수익 가능
3. **SPX 지수 옵션 전략**: SPY 대신 S&P 500 지수(SPX)를 활용하여 레버리지 수익 추구

<img src="./images/fig_06_11.png" width=800>

Figure 6.11 콜옵션, 풋옵션, 스트래들 전략의 수익 구조

스트래들 옵션 전략은 동일한 행사가격과 만기일을 갖는 콜옵션과 풋옵션을 동시에 매수합니다.  
기초 자산 가격이 행사가격에서 크게 벗어날 경우 수익이 발생하며, 이는 변동성이 높은 시장에 유리합니다.

### Portfolio Construction 

| 항목 | 내용 |
| :-- | :-- |
| 모델 학습 시간 | 매일 장 시작 후 1분 |
| 리밸런싱 시간 | 모델 학습 직후 |
| 포트폴리오 구성 | 예측된 국면에 따라 3가지 전략 중 하나 실행 |

|  | 저변동성 환경 | 고변동성 환경 |
| :-- | :-- | :-- |
| 전략 1 | 포트폴리오의 100%를 SPY에 투자 | 포트폴리오의 100%를 TLT에 투자 |
| 전략 2 | SPY 옵션 스트래들 매도 | SPY 옵션 스트래들 매수 |
| 전략 3 | SPX 옵션 스트래들 매도 | SPX 옵션 스트래들 매수 |

### Trading Logic

모델은 매일 아침 훈련되며, 국면 전환이 감지될 때만 리밸런싱을 수행합니다.  
즉, 저→고 또는 고→저 변동성으로 바뀔 때만 포트폴리오가 변경됩니다.

```python
# 마르코프 모델 생성
# k_regimes는 국면의 수를 지정하며, 여기서는 고/저 변동성 2가지
# switching_variance=True는 각 국면이 고유의 분산을 가질 수 있도록 허용
model = MarkovRegression(
    self._daily_returns, k_regimes=2,
    switching_variance=True
)
```

```python
# 현재 시장 국면 추출 (0 = 저변동성, 1 = 고변동성)
regime = model.fit().smoothed_marginal_probabilities.values.argmax(axis=1)[-1]
```


### Tearsheet  

### Algorithm 1

결과는 다음과 같습니다:

- 룩백 기간 4년이 가장 높은 샤프 비율을 달성함
- 샤프 비율은 룩백 기간 변화에 민감함
- 룩백 기간이 1년일 경우 샤프 비율은 음수가 됨
- Figure 6.12 및 6.13 참고

백테스트 파라미터:

- lookback_years: 3

<img src="./images/fig_06_12.png" width=800>

Figure 6.12 Example 4.1의 수익 곡선, 성과 플롯 및 사용자 정의 플롯

<img src="./images/fig_06_13.png" width=800>

Figure 6.13 Example 4.1의 월별 수익률, 위기 이벤트 및 민감도 테스트

### Algorithm 2 

Figure 6.14 및 6.15 참고

백테스트 파라미터:

- lookback_years: 3  
- min_expiry: 90  
- max_expiry: 365  
- min_hold_period: 7  

<img src="./images/fig_06_14.png" width=800>

Figure 6.14 Example 4.2의 수익 곡선, 성과 플롯 및 사용자 정의 플롯

<img src="./images/fig_06_15.png" width=800>

Figure 6.15 Example 4.2의 월별 수익률 및 위기 이벤트

### Algorithm 3 

Figure 6.16 및 6.17 참고

백테스트 파라미터:

- lookback_years: 3  
- min_expiry: 0  
- max_expiry: 180  
- min_hold_period: 0  

<img src="./images/fig_06_16.png" width=800>

Figure 6.16 Example 4.3의 수익 곡선, 성과 플롯 및 사용자 정의 플롯

<img src="./images/fig_06_17.png" width=800>

Figure 6.17 Example 4.3의 월별 수익률 및 위기 이벤트

### Implementation Insights 

### Algorithm 1

모델이 고변동성 국면을 감지하면 TLT에 포트폴리오의 100%를 할당하고,  
저변동성 국면을 감지하면 SPY에 100%를 할당합니다.  
모델은 매일 장 시작 1분 후에 재학습되며, 국면이 변경되면 리밸런싱을 수행합니다.

```python
def _trade(self):
    # 마르코프 모델 생성
    model = MarkovRegression(
        self._daily_returns,
        k_regimes=2,
        switching_variance=True
    )

    # 현재 시장 국면 추출 (0: 저변동성, 1: 고변동성)
    regime = model.fit().smoothed_marginal_probabilities.values.argmax(axis=1)[-1]

    self.plot('Regime', 'Volatility Class', regime)

    # 국면이 변경된 경우에만 리밸런싱 수행
    if regime != self._previous_regime:
        self.set_holdings([
            PortfolioTarget(self._tlt, regime),
            PortfolioTarget(self._spy, int(not regime))
        ])
        self._previous_regime = regime
```



### Algorithm 2 

이 알고리즘은 이전 알고리즘을 확장한 것으로, SPY와 TLT 간 로테이션 대신 SPY 옵션 계약을 이용한 스트래들 전략을 수행합니다.

리밸런싱 로직은 다음과 같이 변경됩니다:

- **저변동성 국면**: 숏 스트래들(short straddle) 개시
- **고변동성 국면**: 롱 스트래들(long straddle) 개시

스트래들을 구성하기 위해 만기가 가장 가까우며 ATM(행사가와 현재가가 유사한) 옵션을 선택합니다.  
미국식 옵션을 사용하므로, 스트래들 매도 시 매수자가 조기 행사할 수 있습니다. 이 경우 남은 레그를 청산하고 다음 장 개시에 새로운 스트래들을 개시합니다.

주식 옵션 거래 시 액면분할(splits)이 발생할 경우, 옵션 거래 정보를 저장하는 데이터 구조를 조정해야 합니다.

이 알고리즘은 단순히 예측된 변동성 국면에 따라 액션만 교체하며, 포지션 사이징은 단순화를 위해 생략됩니다.

```python
def _trade(self):
    model = MarkovRegression(
        self._daily_returns, k_regimes=2, switching_variance=True
    )
    regime = model.fit().smoothed_marginal_probabilities.values.argmax(axis=1)[-1]
    self.plot('Regime', 'Volatility Class', regime)

    if regime != self._previous_regime or not self.portfolio.invested:
        # 기존 스트래들 청산
        for symbol in self._equity.hedge_contracts:
            self.liquidate(symbol)
        self._equity.hedge_contracts = []

        option_chains = self.current_slice.option_chains
        if self._option_symbol not in option_chains:
            return
        chain = option_chains[self._option_symbol]
        min_expiry_date = self.time + self._min_expiry + self._min_hold_period

        expiries = [contract.expiry for contract in chain if contract.expiry >= min_expiry_date]

        if regime == 0:
            option_type = OptionStrategies.short_straddle
            expiry = min(expiries)
        else:
            option_type = OptionStrategies.straddle
            expiry = max(expiries)

        strike = sorted(
            [contract for contract in chain if contract.expiry == expiry],
            key=lambda contract: abs(chain.underlying.price - contract.strike)
        )[0].strike

        option_strategy = option_type(self._option_symbol, strike, expiry)
        tickets = self.buy(option_strategy, 1)
        self._equity.hedge_contracts = [t.symbol for t in tickets]
        self._previous_regime = regime

def on_order_event(self, order_event):
    # 스트래들 레그 중 하나가 행사되었을 경우 남은 레그도 청산
    if (order_event.status == OrderStatus.FILLED and
        self._equity.invested and
        self._equity.hedge_contracts):
        self.liquidate(self._equity.symbol)
        for symbol in self._equity.hedge_contracts:
            self.liquidate(symbol)
        self._equity.hedge_contracts = []
```

### Algorithm 3

이 알고리즘은 이전 알고리즘을 확장한 것으로, 다음과 같은 차이점이 있습니다:

* **주식 옵션 대신 유럽식 지수 옵션 사용**: 조기 행사를 방지
* **기초 자산으로 SPY 대신 SPX 지수 사용**: S\&P 500을 더 정확히 반영

코드 구조는 동일하지만, 유럽식 옵션은 만기일에만 행사되므로 스트래들 청산 관련 로직은 제거됩니다.

```python
def _trade(self):
    model = MarkovRegression(
        self._daily_returns, k_regimes=2, switching_variance=True
    )
    regime = model.fit().smoothed_marginal_probabilities.values.argmax(axis=1)[-1]
    self.plot('Regime', 'Volatility Class', regime)

    if (regime != self._previous_regime or
        not self.portfolio.invested or
        self._expiry - self.time < self._min_expiry):
        
        self.liquidate()

        option_chains = self.current_slice.option_chains
        if self._option_symbol not in option_chains:
            return
        chain = option_chains[self._option_symbol]
        min_expiry_date = self.time + self._min_expiry + self._min_hold_period

        expiries = [contract.expiry for contract in chain if contract.expiry >= min_expiry_date]
        if not expiries:
            return

        if regime == 0:
            option_type = OptionStrategies.short_straddle
            expiry = min(expiries)
        else:
            option_type = OptionStrategies.straddle
            expiry = max(expiries)

        strike = sorted(
            [contract for contract in chain if contract.expiry == expiry],
            key=lambda contract: abs(chain.underlying.price - contract.strike)
        )[0].strike

        option_strategy = option_type(self._option_symbol, strike, expiry)
        tickets = self.buy(option_strategy, 1)
        self._index.hedge_contracts = [t.symbol for t in tickets]
        self._expiry = expiry
        self._previous_regime = regime

def on_order_event(self, order_event):
    # 스트래들 한 레그가 행사되었을 때 나머지 레그도 자동 청산되므로 참조 제거
    if (order_event.status == OrderStatus.FILLED and order_event.is_assignment):
        self._index.hedge_contracts = []
        self._expiry = datetime.min
```

---



## Example 5—FX SVM Wavelet Forecasting  

| Predicting | 가격 |
| :-- | :-- |
| Technology | 서포트 벡터 머신 회귀 (SVM Regression) |
| Asset Class | 외환 (Forex) |
| Difficulty | 중간 |
| Type | 전체 전략 |
| Source Code | qnt.co/book-example5 |

### Summary 

이 알고리즘은 SVM과 웨이블릿(wavelet)을 결합하여 외환(FX) 통화쌍의 미래 가격을 예측합니다. 먼저 과거 종가를 웨이블릿 분해하여 여러 성분으로 나눈 뒤, 각 성분에 대해 SVM을 사용하여 1일 후 예측을 수행합니다. 마지막으로 예측된 성분들을 다시 결합하여 최종 가격을 추정합니다.

### Motivation

외환 시장은 경제 지표, 시장 심리, 지정학적 사건 등 다양한 주기와 패턴의 영향을 받으며, 비정상적이고 노이즈가 많은 데이터로 구성되어 있어 패턴을 식별하기 어렵습니다.

웨이블릿 분해는 다음과 같은 이유로 특히 유용합니다:

- 복잡한 시장 동역학을 개별 구성 요소로 분해
- 노이즈를 제거하고 중요한 신호만 추출
- 단기 변동성과 장기 추세를 동시에 포착

SVM은 비선형 관계를 잘 다루기 때문에 각 웨이블릿 구성 요소를 예측하는 데 적합합니다. SVM은 각 분해 성분에 대해 개별 예측을 수행하고, 이를 바탕으로 전체 시계열을 재구성합니다.

### Model

- **입력 특성**: 외환 종가
- **예측 라벨**: 하루 뒤의 외환 가격
- **모델 구조**: 웨이블릿으로 분해한 후 SVM으로 예측

`SVMWavelet` 클래스는 `forecast`라는 공개 메서드 하나만을 갖습니다.

1. `pywt` 패키지를 사용하여 웨이블릿 처리
   ```python
   import pywt
```

2. 웨이블릿 종류는 smoothing 특성이 있는 Symlet 10 사용

   ```python
   w = pywt.Wavelet('sym10')
   ```

3. 입력 길이 계산: 3단계 분해를 위해 필요한 입력 길이는 152
   $\log_2\left(\frac{\text{len(data)}}{\text{wave length} - 1}\right) = 3$
   → `len(data) = 152`

4. 웨이블릿 계수 노이즈 제거 임계값 계수 설정

   ```python
   threshold = 0.5
   ```

5. 시계열 데이터 분해

   ```python
   coeffs = pywt.wavedec(data, w)
   ```

6. 첫 번째 근사 계수를 제외하고 denoising + SVM 예측 수행

   ```python
   for i in range(len(coeffs)):
       if i > 0:
           # 근사 계수는 건드리지 않음
           coeffs[i] = pywt.threshold(coeffs[i], threshold * max(coeffs[i]))
       forecasted = self._svm_forecast(coeffs[i])
       coeffs[i] = np.roll(coeffs[i], -1)
       coeffs[i][-1] = forecasted
   ```

7. 역 웨이블릿 변환을 통해 미래 값을 복원하고 마지막 값을 반환

   ```python
   datarec = pywt.waverec(coeffs, w)
   return datarec[-1]
   ```

### Trading Universe

기준 통화쌍은 EURUSD이며, 다음의 4개 통화쌍을 거래합니다:
**EUR/JPY**, **GBP/USD**, **AUD/CAD**, **NZD/CHF**

### Portfolio Construction

| 항목       | 내용                    |
| :------- | :-------------------- |
| 모델 학습 시점 | 새로운 FX 일간 데이터 수신 시    |
| 리밸런싱 시점  | 모델 학습 이후 즉시           |
| 포트폴리오 비중 | 예측 수익률이 임계값 이상일 경우 투자 |

비중 설정 방식:

* 비중(weight)은 `(예측 가격 / 현재 종가) - 1`로 계산
* 절댓값이 `self._weight_threshold` (기본값: 0.005)보다 크면 실행
* 비중에 `self._leverage` (기본값: 20)를 곱해 실제 투자 비율 설정

```python
prices = np.array(list(security.window))[::-1]
forecasted_value = self._wavelet.forecast(prices)
weight = (forecasted_value / bar.close) - 1
if abs(weight) > self._weight_threshold:
    self.set_holdings(security.symbol, weight * self._leverage)
```

### Trading Logic

거래는 매일 데이터를 수신하는 consolidation handler에서 수행되며,
다음날 종가를 예측하고, 예상 수익률이 임계값을 초과하면 해당 통화쌍에 레버리지를 적용해 투자합니다.

### Tearsheet

| 파라미터   | 설명                                      |
| :----- | :-------------------------------------- |
| period | 최소 63일(3개월), 최대 189일(9개월), 간격은 21일(1개월) |

결과 요약:

* \*\*8개월 주기(period)\*\*일 때 샤프 비율이 가장 높았으나, 민감도가 큼
* **weight\_threshold > 0.004**일 때 샤프 비율이 일반적으로 더 높음
* Figure 6.18, 6.19 참고

백테스트 파라미터:

* period: 152
* leverage: 20
* weight\_threshold: 0.005

<img src="./images/fig_06_18.png" width=800>

Figure 6.18 Example 5의 수익 곡선, 성과 플롯 및 사용자 정의 플롯

<img src="./images/fig_06_19.png" width=800>

Figure 6.19 Example 5의 월별 수익률, 위기 이벤트, 민감도 테스트



### Implementation Insights 

훈련 코드는 두 가지 측면에서 중요합니다:  
1) 훈련 데이터셋을 어떻게 구축하는지  
2) SVR 하이퍼파라미터의 최적값을 어떻게 찾는지

SVR 메서드는 다음과 같은 순서로 진행됩니다:

1. 먼저 데이터를 크기 10의 청크로 분할합니다.  
   ```python
    X, y = self._partition_array(data, size=sample_size)
    data = np.array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
    sample_size = 3
    y = example._partition_array(data, size=sample_size)
    ```

결과:
```
Partitions (X): [[1 2 3] [2 3 4] [3 4 5] [4 5 6] [5 6 7] [6 7 8] [7 8 9]]
One-step ahead values (y): [4 5 6 7 8 9 10]
```

위는 SVR 훈련을 위한 특징과 레이블을 구성하는 방식입니다.

2. SVR의 하이퍼파라미터 `C`와 `epsilon`을 찾기 위해 그리드 서치를 실행합니다.

   ```python
   from sklearn.model_selection import GridSearchCV
   from sklearn.svm import SVR

   gsc = GridSearchCV(
       SVR(),
       {
           'C': [.05, .1, .5, 1, 5, 10],
           'epsilon': [0.001, 0.005, 0.01, 0.05, 0.1]
       },
       scoring='neg_mean_squared_error'
   )
   model = gsc.fit(X, y).best_estimator_
   ```

3. 마지막으로 다음 시점의 값을 예측합니다.

   ```python
   model.predict(data[np.newaxis, -sample_size:])[0]
   ```

따라서 `_svm_forecast` 메서드는 다음과 같이 정의됩니다:

```python
def _svm_forecast(self, data, sample_size=10):
    """
    데이터를 분할하여 SVM 모델을 학습하고,
    한 스텝 미래 값을 예측합니다.
    """
    X, y = self._partition_array(data, size=sample_size)
    gsc = GridSearchCV(
        SVR(),
        {
            'C': [.05, .1, .5, 1, 5, 10],
            'epsilon': [0.001, 0.005, 0.01, 0.05, 0.1]
        },
        scoring='neg_mean_squared_error'
    )
    model = gsc.fit(X, y).best_estimator_
    return model.predict(data[np.newaxis, -sample_size:])[0]
```

---


## Example 6—Dividend Harvesting Selection of High-Yield Assets

| Predicting  | 배당 수익률 순위            |
| :---------- | :------------------- |
| Technology  | 회귀 모델                |
| Asset Class | 미국 주식                |
| Difficulty  | 중간–상                 |
| Type        | 전체 전략                |
| Source Code | gnt.co/book-example6 |

### Summary

우리는 향후 높은 배당 수익률이 예상되는 자산으로 포트폴리오를 구성합니다.
이를 위해 결정 트리 회귀(decision tree regression) 모델을 사용하여 다음과 같은 재무 지표를 바탕으로 미래 배당 수익률을 예측합니다:

* PER (주가수익비율)
* 매출 성장률
* 영업현금흐름 대비 잉여현금흐름 비율
* 배당성향
* 유동비율 (Current Ratio)



### Motivation 

수익을 극대화하면서 리스크를 최소화하는 가장 효과적인 전략 중 하나는 **배당 하베스팅(dividend harvesting)** 입니다. 이는 투자자가 가장 높은 배당 수익률(주당 배당금 ÷ 주가)을 가진 자산에 투자하는 방식으로,  
꾸준한 수익 흐름을 창출하며 포트폴리오에 안정성을 더합니다.  
정기적으로 배당을 지급하는 기업은 일반적으로 재무적으로 안정적인 경우가 많습니다.

### Model

**모델 입력 특성 (Features)**

- **PER (PE ratio)**: 주가 ÷ 주당순이익(EPS)
- **매출 성장률**: 최근 두 재무보고서 간의 매출 증가율
- **잉여현금흐름/영업현금흐름 비율**: 최근 재무보고서 기준
- **배당성향**: 직전 분기의 배당금 ÷ 순이익
- **유동비율(Current ratio)**: 유동자산 ÷ 유동부채

**예측 대상 (Label)**  
→ 다음 배당 지급에서의 **예상 배당 수익률**

**모델**: 결정 트리 회귀(Decision Tree Regressor)  
복잡한 변수 간 관계를 다룰 수 있어 결정 트리 모델을 선택하였습니다 (참고: [qnt.co/book-decision-tree](https://qnt.co/book-decision-tree)).

### Trading Universe 

QQQ ETF의 구성 종목 중 상위 100개 종목을 대상으로 하며, 매월 초 기준으로 가장 높은 비중을 가진 종목들로 선택합니다.

### Portfolio Construction

| Model Training <br> Time | 매월 첫 거래일 장 시작 30분 전 |
| :-- | :-- |
| Portfolio <br> Rebalancing Time | 모델 훈련 직후 즉시 |
| Portfolio Weights | 각 자산의 예측된 배당 수익률 ÷ 전체 예측 합계로 비중 산정 |

### Trading Logic

예측된 배당 수익률이 높을수록 포트폴리오 내 비중을 더 크게 설정합니다.  
즉, 포트폴리오는 기대 배당 수익률에 비례하여 구성됩니다.

```python
def _trade(self):
    r_squared_values = []
    prediction_by_symbol = {}
    for symbol in self._universe.selected:
        symbol_data = self._symbol_data_by_symbol[symbol]
        r_squared = symbol_data.train()
        if r_squared is None:
            continue
        r_squared_values.append(r_squared)
        prediction_by_symbol[symbol] = symbol_data.predict()

    prediction_sum = sum(prediction_by_symbol.values())
    portfolio_targets = [
        PortfolioTarget(symbol, prediction / prediction_sum)
        for symbol, prediction in prediction_by_symbol.items()
    ]
    self.set_holdings(portfolio_targets, True)
    self.plot("Portfolio Size", "Count", len(portfolio_targets))
```

### Tearsheet

결과 요약:

* Sharpe 비율은 0.476 \~ 0.617 사이로 형성됨
* **universe\_size**가 클수록 Sharpe 비율이 더 높은 경향
* lookback\_years보다는 universe\_size 변화에 더 민감함
* 모든 파라미터 조합에서 Sharpe 비율은 양수
* Figure 6.20, 6.21 참고

**파라미터 universe\_size:**

* 최소: 20 (최소 한 종목은 배당 지급 필요)
* 최대: 100 (QQQ ETF 전체)
* 간격: 20

**파라미터 lookback\_years:**

* 최소: 4 (배당이 자주 지급되지 않기 때문)
* 최대: 8 (이후 데이터는 의미가 줄어듦)
* 간격: 1

**백테스트 파라미터**

* universe\_size: 100
* lookback\_years: 5

<img src="./images/fig_06_20.png" width=800>

Figure 6.20 Example 6의 수익 곡선, 성과 플롯, 사용자 정의 플롯

<img src="./images/fig_06_21.png" width=800>

Figure 6.21 Example 6의 월별 수익률, 위기 이벤트, 민감도 분석



### Implementation Insights 

거래 유니버스를 선택하는 동안, 우리는 ETF의 각 구성 종목에 대해 `SymbolData` 클래스의 인스턴스를 생성합니다. 이 클래스는 각 ETF 구성 종목을 다음과 같이 캡슐화합니다:

- 모든 펀더멘털 요소의 5년치 히스토리, 레이블(즉, 배당 수익률)의 5년치 히스토리 및 히스토리를 구축하는 클래스 메서드 포함
- `sklearn`의 `DecisionTreeRegressor` 인스턴스
- 훈련 메서드는 기술적인 작업 위주이며, 현재 기간의 펀더멘털 요소들과 다음 배당 수익률을 예측하기 위한 회귀용 훈련 데이터를 생성
- 예측 메서드는 최신 펀더멘털 요소를 입력으로 사용해 훈련된 모델로부터 다음 기간의 배당 수익률을 반환

거래 알고리즘은 수신된 배당금도 로그로 기록합니다:

- `on_data` 메서드에서 새로운 시장 데이터를 받을 때마다 새로 수신된 배당금을 기준으로 `Dividends Received` 차트를 업데이트
- `on_end_of_algorithm` 시점에 백테스트 전반에서 수신된 배당금 로그 기록

```python
def on_data(self, data):
    dividends_received = 0
    for symbol, dividend in data.dividends.items():
        security_holding = self.portfolio[symbol]
        if security_holding.invested:
            dividends_received += (
                dividend.distribution * security_holding.quantity
            )
    self.plot("Dividends Received", "Value", dividends_received)

def on_end_of_algorithm(self):
    self.log("Dividends received:")
    dividend_by_symbol = {
        security.symbol: security.holdings.total_dividends
        for security in self.securities.total
        if security.holdings.total_dividends
    }
    sorted_by_dividends_earned = sorted(
        dividend_by_symbol.items(), key=lambda x: x[1], reverse=True
    )
    for symbol, total_dividends in sorted_by_dividends_earned:
        self.log(f"- ${total_dividends} ({symbol.value})")
    self.log("-----------------")
    self.log(f"Total: ${sum(dividend_by_symbol.values())}")
```

---


## Example 7—Effect of Positive-Negative Splits

| Predicting  | Returns (equities)   |
| :---------- | :------------------- |
| Technology  | Regression           |
| Asset Class | US equities          |
| Difficulty  | Easy-medium          |
| Type        | Full strategy        |
| Source Code | qnt.co/book-example7 |

### Summary

이 알고리즘은 주식 분할(split)로 인해 예상되는 변동성에서 수익을 창출하려 합니다.
다중 선형 회귀 모델을 사용하여 분할이 임박한 시점에서 미래 수익률을 추정하고,
해당 방향으로 매매한 뒤 1주일 후 포지션을 청산합니다.

### Motivation

기술기업은 혁신적인 제품, 많은 투자 유입, 역동적인 사업 모델 등으로 인해
전통 산업보다 빠른 성장을 경험하는 경우가 많습니다. 이로 인해 주가가 급격히 상승하고,
이때 너무 높은 주가 수준으로 인해 유동성을 높이기 위한 주식 분할이 자주 발생합니다.
분할은 주식 수를 늘리고 주가를 비례적으로 낮추어 접근성과 유동성을 높입니다.

역사적으로 주식 분할 발표 이후 주가는 일반적으로 상승하는 경향이 있습니다.
이는 낮아진 가격과 접근성 증가로 인해 투자 수요가 증가하기 때문이며,
시장에서는 분할을 기업 경영진이 자사 성장에 대한 확신을 보인 신호로 해석하기도 합니다.

이 전략은 이러한 심리와 과거 데이터를 활용하여 분할로 인한 변동성을 공략합니다.
회귀 모델은 아래의 요소를 사용합니다:

* 분할 경고 시점에 발표된 분할 비율 (split factor)
* 기술 섹터 ETF (XLK)의 최근 1개월 수익률 (섹터 모멘텀)
* 해당 주식의 보유 기간 이후 미래 수익률

### Model

**모델 입력 특성 (Features)**

* 각 종목에 대한 분할 비율 (split factor)
* XLK ETF 기준 섹터 월간 수익률

**예측 대상 (Label)**
→ 분할 발표일 기준 **3일간의 예상 미래 수익률**

**모델**: 선형 회귀 (Linear Regression)

### Trading Universe

Morningstar의 기술 섹터로 분류된 전체 미국 주식 자산을 유니버스로 설정합니다.

### Portfolio Construction

| Model Training <br> Time             | 매월 첫 거래일 자정                                     |
| :----------------------------------- | :---------------------------------------------- |
| Portfolio <br> Rebalancing <br> Time | 분할이 보고될 때마다                                     |
| Portfolio <br> Weights               | 동시에 최대 4개 포지션만 보유하며, <br> 각 포지션에 전체 자산의 25%씩 할당 |



### Trading Logic

거래 알고리즘은 분할(split) 이벤트를 모니터링하며,  
각 분할은 `on_splits` 이벤트 핸들러를 트리거합니다.  
이 핸들러는 구독된 주식에 대한 모든 분할 이벤트를 수신합니다.

해당 메서드는 다음과 같이 진행됩니다:  
**1단계:** 기술 섹터 ETF의 분할 이벤트는 거래 대상이 아니므로 필터링합니다.

**2단계:** 분할의 유형에 따라 다음과 같이 행동합니다:

- **분할 경고(split warning)**의 경우:  
  열린 거래 수가 제한(최대 4개) 이내이면,  
  분할 비율과 현재 섹터 모멘텀(rate of change) 지표를 기반으로 예상 수익률을 예측합니다.  
  예측 수익률이 양수이고 최적 거래 수가 양수일 경우,  
  해당 종목에 대한 `Trade` 클래스의 인스턴스를 생성하고  
  `_trades_by_symbol` 딕셔너리에 등록하여 거래를 수행합니다.

- **실제 분할 발생 시:**  
  기존에 실행된 거래 객체의 정보를 업데이트합니다 (실제 분할 비율 적용).

결과적으로 이 알고리즘은 XLK 벤치마크 대비 월등한 성과를 보입니다.

### Tearsheet  

다음과 같은 결과가 관찰됩니다:

- `hold_duration`이 3일일 때 가장 높은 Sharpe ratio가 생성됨
- 모든 Sharpe 비율은 $>=0.7$
- Figure 6.22, 6.23 참고

**백테스트 파라미터:**

- `max_open_trades`: 4  
- `hold_duration`: 3  
- `training_lookback_years`: 4

**hold_duration 파라미터:**

- 최소값: 1 (최소 양의 정수)
- 최대값: 5 (1주일)
- step size: 1

**training_lookback_years 파라미터:**

- 최소값: 3 (분할이 자주 발생하지 않음)
- 최대값: 6 (지나치게 오래된 데이터는 무의미할 수 있음)
- step size: 1

<img src="./images/fig_06_22.png" width=800>

Figure 6.22 Example 7의 수익률 곡선, 성과 그래프, 커스텀 플롯

<img src="./images/fig_06_23.png" width=800>

Figure 6.23 Example 7의 월별 수익률, 위기 시기, 민감도 분석

### Implementation Insights 

모든 거래 기록은 글로벌 딕셔너리 `_trades_by_symbol`에 저장됩니다.  
이 딕셔너리는 각 종목에 대한 거래 리스트를 `Trade` 클래스 인스턴스로 보관합니다.

이 클래스는 선택된 분할에 대해 거래를 실행하며,  
분할 발생 시 실제 `split_factor`에 따라 수량을 조정하고,  
보유 기간이 지나면 거래를 종료합니다.

```python
class Trade:
    def __init__(self, algorithm, symbol, hold_duration, quantity):
        self.closed = False
        self._symbol = symbol
        self._close_time = algorithm.time + hold_duration
        self._quantity = quantity
        algorithm.market_on_open_order(symbol, quantity)

    def on_split_occurred(self, split):
        self._quantity = int(self._quantity / split.split_factor)

    def scan(self, algorithm):
        if not self.closed and self._close_time <= algorithm.time:
            algorithm.market_on_open_order(self._symbol, -self._quantity)
            self.closed = True
```

우리는 매일 자정에 모델을 재훈련하고, 미래 수익률을 업데이트하고,
거래를 종료하기 위해 스케줄 기반 이벤트를 사용합니다.

`_train` 메서드는 다음과 같은 순서로 수행됩니다:

1. 모니터링 중인 종목들의 분할 히스토리를 조회
2. 분할이 발생한 종목들의 과거 가격 데이터를 조회
3. 분할 비율, 섹터 수익률, 향후 수익률의 세 컬럼으로 구성된 훈련용 numpy 배열을 생성
4. 분할 비율과 섹터 수익률을 입력값으로 하여 선형 회귀 모델을 훈련하여 미래 수익률을 예측

---


## Example 8——Stop Loss Based on Historical Volatility and Drawdown Recovery

| Predicting  | Downside volatility for stop loss |
| :---------- | :-------------------------------- |
| Technology  | Regression/forecasting            |
| Asset Class | US equities and US options        |
| Difficulty  | Medium-hard                       |
| Type        | Full strategy                     |
| Source Code | qnt.co/book-example8              |

### Summary

우리는 회귀 모델을 활용하여 포트폴리오를 하방 위험으로부터 보호합니다.
기본적으로 고정 비율의 손절매(stop-loss) 주문을 사용한 단순 모델로 시작한 후,
LASSO 회귀를 활용하여 손절매 폭을 동적으로 조절하는 복잡한 모델로 발전시킵니다.
최종적으로는 손절매 시장 주문을 풋옵션으로 대체하며,
해당 옵션의 파라미터 또한 LASSO 회귀를 통해 최적화합니다.


### Motivation

시장 변동성과 예기치 못한 경제 또는 기업 관련 뉴스는 적절한 리스크 관리 전략이 없을 경우 심각한 손실로 이어질 수 있습니다.

전통적인 리스크 관리 기법은 일반적으로 지정된 가격에 도달하면 자동으로 매도되는 손절매(stop-loss) 주문을 포함합니다.

고정 비율 기반의 손절매는 단순하고 널리 사용되지만, 시장 상황의 동적인 특성을 반영하지 못한다는 한계가 있습니다.  
이는 일시적인 하락에도 거래가 종료되어 회복 기회를 놓치거나 거래 비용이 증가할 수 있습니다.

이러한 한계를 인식하고, 우리는 점점 더 정교한 회귀 모델들을 적용하여 손절매 전략의 효율성을 향상시키고  
포트폴리오를 하방 위험으로부터 더 잘 보호합니다:

1. **고정 비율 손절매 주문:**  
   고정된 비율의 손절매 주문을 사용하는 단순 모델을 출발점으로 사용합니다.  
   이 모델은 전략의 벤치마크로, 하방 위험에 대한 기본적인 보호 수단을 제공합니다.  
   하지만 시장 변화에 대응하지 못해 성능이 제한적입니다.

2. **LASSO 회귀 기반 동적 손절매 거리 조정:**  
   고정 손절매의 경직성을 해결하기 위해, 가격 움직임에 영향을 주는 핵심 요인을 식별하고  
   이들에 기반하여 최적 손절매 수준을 결정하는 LASSO 회귀 기반 모델로 확장합니다.  
   이 접근 방식은 큰 손실을 방지하는 동시에 단기 변동성에 대한 불필요한 매도를 줄입니다.

3. **풋옵션 대체 및 LASSO 회귀 최적화:**  
   마지막 모델에서는 손절매 주문을 풋옵션으로 대체합니다.  
   풋옵션은 기초 자산에 대해 보장된 최저가를 제공하여 더 강력한 보호를 제공합니다.  
   행사가격과 만기일 등 옵션의 파라미터는 LASSO 회귀로 최적화되어,  
   시장 상황에 맞게 조정되며 리스크 보호는 극대화되고 비용은 최소화됩니다.

이처럼 고급 회귀 모델을 통해, 데이터 기반의 적응형 전략을 구현하여 전통적인 방법보다 우수한 리스크 관리를 달성합니다.

### Model

첫 번째 거래 알고리즘은 고정 비율 가격 기반 손절매 주문을 실행합니다.  
두 번째 및 세 번째 거래 알고리즘은 동일한 모델을 공유하며, 실험 간 모델 자체는 변경되지 않아 성능 비교가 용이합니다.

**Model Features**

- VIX 지수  
- 최근 n개월의 평균 진폭 (Average True Range)  
- 최근 n개월의 표준편차

**예측 대상 (Label)**

- 주간 저점 수익률: 해당 주의 시작가 대비, 이후 5거래일 동안의 최저가까지의 수익률

**사용 모델: LASSO 회귀 모델**  
LASSO 회귀는 불필요한 변수를 제외해 노이즈를 줄이고,  
다중공선성을 처리하며 과적합을 방지하는 정규화 효과 덕분에  
이 전략에 매우 적합합니다.  
결과적으로 안정적이고 일반화 가능한 모델을 제공하며,  
시장 변화에 따라 동적으로 대응할 수 있습니다.

### Trading Universe

KO 주식을 거래 대상으로 하며, 알고리즘 3에서는 KO 풋옵션도 포함됩니다.

### Portfolio Construction

- **모델 훈련 시간:** 매주 첫 거래일의 장 시작 2분 후  
- **포트폴리오 리밸런싱 시간:** 모델 훈련 직후 즉시  
- **포트폴리오 구성 비율:** KO 주식에 100% 투자

### Trading Logic

세 알고리즘 모두 매주 초 KO 주식을 $100,000 어치 매수하고,  
그 주 안에 포지션을 청산합니다:

- **알고리즘 1:**  
  벤치마크 알고리즘으로, 매주 초 오전 9:32에 매수 후  
  원가의 95%에 손절매 주문을 설정합니다.  
  다음 주 시장 개장 전 오전 9시에 잔여 주식을 전량 청산합니다.

- **알고리즘 2:**  
  동일한 구조이나, 손절매 가격을 LASSO 회귀 모델이 예측한 저가보다 $0.01 낮은 위치에 설정합니다.

- **알고리즘 3:**  
  손절매 주문을 LASSO 회귀 예측 저가 이하의 행사가격을 갖는 풋옵션 매수로 대체합니다.


### Tearsheet

### Algorithm 1: Benchmark—Fixed Percentage Stop Loss

결과는 다음과 같습니다:

- 손절매가 너무 타이트한 경우(≥ 0.985) 샤프 비율이 급격히 하락합니다.
- 테스트된 손절매 거리 중 17개는 단순 매수 후 보유 전략(0.263 샤프)을 능가합니다.
- Figure 6.24 및 6.25 참조.

**백테스트 파라미터:**

- `stop_loss_percent`: 0.95

**파라미터 설명:**

- 최소값은 0.9입니다. KO 주식은 큰 변동이 적으며, 백테스트 기간 중 20% 이상 하락한 경우는 한 번뿐입니다.
- 최대값은 0.995입니다. 0.5%보다 타이트한 손절매는 거의 모든 거래에서 손절매가 작동될 가능성이 있습니다.
- 스텝 사이즈는 0.005이며, 이는 소수점 한 자리로 20개의 서로 다른 값을 테스트할 수 있게 해줍니다.

<img src="./images/fig_06_24.png" width=800>

Figure 6.24 Equity curve and performance plots for Example 8.1.

<img src="./images/fig_06_25.png" width=800>

Figure 6.25 Monthly returns, crisis events, and sensitivity tests for Example 8.1.

### Algorithm 2: ML Placed Stop Loss

결과는 다음과 같습니다:

- 테스트된 구간 내에서 `stop_loss_buffer`가 클수록 샤프 비율이 일반적으로 상승합니다.
- 테스트된 30개 조합 중 28개가 단순 매수 후 보유 전략(0.263 샤프)을 능가합니다.
- 모든 파라미터 조합에서 샤프 비율이 양(positive)입니다.
- Figure 6.26 및 6.27 참조.

**백테스트 파라미터:**

- `indicator_lookback_months`: 3
- `stop_loss_buffer`: 0.01
- `alpha_exponent`: 4

**파라미터 설명:**

- `indicator_lookback_months`:
  - 최소값은 1이며, 선택 가능한 가장 작은 정수입니다.
  - 최대값은 6으로, 약 반 년에 해당합니다.
  - 스텝 사이즈는 1입니다.

- `stop_loss_buffer`:
  - 최소값은 0.01이며, 이는 자산의 최소 가격 변동 단위입니다.
  - 최대값은 1.01이며, 거의 정확히 1달러로 모델 예측치보다 멀리 손절매를 설정합니다.
  - 스텝 사이즈는 0.25로, 최소와 최대 사이에서 여러 샘플을 제공합니다.

<img src="./images/fig_06_26.png" width=800>

Figure 6.26 Equity curve, performance plots, and custom plots for Example 8.2.

<img src="./images/fig_06_27.png" width=800>

Figure 6.27 Monthly returns, crisis events, and sensitivity tests for Example 8.2.

### Algorithm 3: ML Put Option Hedge

Figure 6.28 및 6.29 참조.

**백테스트 파라미터:**

- `alpha_exponent`: 4

<img src="./images/fig_06_28.png" width=800>

Figure 6.28 Equity curve, performance plots, and custom plots for Example 8.3.

<img src="./images/fig_06_29.png" width=800>

Figure 6.29 Monthly returns and crisis events for Example 8.3.

### Implementation Insights

### Algorithm 1: Benchmark—Fixed Percentage Stop Loss

우리는 매주 초, 오전 9시 30분 시장 개장 후 2분 뒤에 100,000달러 상당의 KO 주식을 시장가로 매수합니다. 동시에, 구매 가격의 주어진 비율(예: `self._stop_loss_percent`가 95%)로 하락할 경우를 대비하여 동일 수량에 대해 매도 스톱 마켓 주문을 설정합니다.

해당 주 동안 스톱로스 주문이 실행되지 않으면, 다음 주 초에 Market On Open 명령으로 포지션을 청산합니다 (`liquidate` 메서드 참고).

### Algorithm 2: ML Placed Stop Loss

모델 입력 요소는 다음과 같습니다:

- VIX
- KO 주식의 최근 22일 종가 기준 Average True Range
- KO 주식의 최근 22일 종가 기준 표준편차

```python
def initialize(self):
    self._security = self.add_equity(
        "KO",
        data_normalization_mode=DataNormalizationMode.RAW
    )
    self._symbol = self._security.symbol
    period = 22 * self.get_parameter('indicator_lookback_months', 1)
    self._vix = self.add_data(CBOE, "VIX", Resolution.DAILY).symbol
    self._atr = AverageTrueRange(period, MovingAverageType.SIMPLE)
    self._std = StandardDeviation(period)
```

이 버전의 알고리즘은 LASSO 회귀 모델을 사용하여 스톱로스 거리(stop-loss distance)를 동적으로 조절하여 성능을 향상시킵니다. 회귀 모델은 `_samples`라는 데이터프레임에서 3년치 샘플을 기반으로 훈련됩니다.

```python
def initialize(self):
    # ...
    # 요인과 라벨을 저장할 DataFrame 생성
    self._samples = pd.DataFrame(
        columns=['vix', 'atr', 'std', 'weekly_low_return'],
        dtype=float
    )
    self._samples_lookback = timedelta(3 * 365)
```

모델 데이터프레임의 컬럼은 다음과 같습니다:

* `vix`: CBOE VIX의 일일 수치
* `atr`: 최근 22일간 AAPL 주식의 평균 진폭
* `std`: 최근 22일간 AAPL 주식의 표준편차
* `weekly_low_return`: 모델이 예측하도록 훈련되는 라벨, 오늘 시가 대비 향후 5 거래일(1주) 내 저가의 변동률

`samples` 데이터프레임을 구성하는 코드는 기술적으로 복잡하며, 분 단위 데이터를 일일 바 형태로 집계하기 위한 consolidator를 설정합니다. 주식이 분할되는 경우, 조정된 가격을 반영하기 위해 전체 데이터프레임을 다시 채웁니다. 또한, VIX 값이 갱신되면 학습 데이터도 함께 갱신됩니다.

알고리즘은 `_samples`에 훈련 데이터셋이 준비되면 거래가 가능합니다. 매주 초 다음과 같은 방식으로 동작합니다:

1. KO 주식을 매수합니다.
2. 해당 주의 저가를 예측합니다.
3. 예측된 주간 저가보다 `self._stop_loss_buffer`(예: $0.01) 낮은 가격에 스톱 마켓 주문을 설정합니다. 시장이 예측보다 더 변동성이 클 경우에만 손절이 실행되며, 그렇지 않으면 주간 종료 후 해당 주문을 취소하고 Market On Open으로 포지션을 청산합니다.

```python
def initialize(self):
    self._stop_loss_buffer = self.get_parameter('stop_loss_buffer', 0.01)
    date_rule = self.date_rules.week_start(self._symbol)
    self.schedule.on(
        date_rule,
        self.time_rules.after_market_open(self._symbol, 2),
        self._enter
    )
    self.schedule.on(
        date_rule,
        self.time_rules.after_market_open(self._symbol, -30),
        self.liquidate
    )
    self._model = Lasso(alpha=10 ** (-self.get_parameter('alpha_exponent', 4)))
```

```python
def _enter(self):
    # 오늘 시가에서 주간 저가까지의 수익률 예측을 위한 모델 훈련
    training_samples = self._samples.dropna()
    self._model.fit(
        training_samples.iloc[:, :-1],
        training_samples.iloc[:, -1]
    )
    prediction = self._model.predict(
        [self._samples.iloc[:, :-1].dropna().iloc[-1]]
    )[0]
    predicted_low_price = self._security.open * (1 + prediction)
    self.plot("Stop Loss", "Distance", 1 + prediction)
    
    # 매수 주문 실행
    quantity = self.calculate_order_quantity(self._symbol, 1)
    self.market_order(self._symbol, quantity)
    
    # 예측된 저가보다 self._stop_loss_buffer 낮은 위치에 스톱로스 설정
    self.stop_market_order(
        self._symbol, -quantity,
        round(predicted_low_price - self._stop_loss_buffer, 2)
    )
```

### Algorithm 3: ML Put Option Hedge

이 전략은 이전 버전을 기반으로 스톱 마켓 주문을 풋옵션으로 대체합니다. 두 방식 모두 하방 가격 움직임에 대한 방어를 제공하지만, 각각 장단점이 있습니다:

- 스톱 마켓 주문의 경우, 정밀하게 손절 가격을 지정할 수 있습니다. 하지만, 실제 체결 가격은 예측할 수 없습니다. 시장이 변동성이 크거나 장 마감 후 큰 갭이 발생하면, 스톱 가격보다 낮은 가격에 체결될 수 있습니다. 또한, 스톱 마켓 주문이 발동되면 시장가 주문으로 실행되므로, 스프레드 비용 및 시장 충격의 영향을 받습니다.
- 반면 풋옵션을 사용한 헷지 전략의 단점은 옵션 계약을 구매해야 하므로 추가 비용이 발생한다는 점입니다. 또한, 원하는 정확한 행사가를 선택할 수 없고, 가장 가까운 수준만 선택할 수 있습니다. 하지만, 풋옵션을 사용하면 우리가 지정한 행사가가 손절 가격으로 정확히 작동하며, 갭, 스프레드 비용, 시장 충격으로부터 보호받을 수 있습니다. 기초 자산 가격이 행사가보다 하락하면 언제든 옵션을 행사하여 주식을 팔 수 있기 때문입니다.

LEAN 거래 알고리즘에서는 기본적으로 수수료가 포함되어 있습니다. 본 예제는 기본적으로 `InteractiveBrokersFeeModel`을 사용하지만, 수수료 모델을 변경하는 방법을 시연합니다.

```python
class CaseOfTheMondaysAlgorithm(QCAlgorithm):
    def initialize(self):
        self.set_security_initializer(
            IBFeesSecurityInitializer(
                self.brokerage_model,
                FuncSecuritySeeder(self.get_last_known_prices)
            )
        )

class IBFeesSecurityInitializer(BrokerageModelSecurityInitializer):
    def __init__(self, brokerage_model, security_seeder):
        super().__init__(brokerage_model, security_seeder)

    def initialize(self, security):
        super().initialize(security)
        security.set_fee_model(InteractiveBrokersFeeModel())
```

우리는 Algorithm 2와 동일한 요인과 훈련 데이터프레임을 생성합니다. 알고리즘의 동작 방식은 다음과 같습니다:

* 매주 월요일 오전 9:32에 KO 주식을 매수하고, 이전과 같이 해당 주간의 저가를 예측합니다.
* 그러나 이번에는 스톱 마켓 주문 대신, 예측된 저가 + 옵션 매도호가 이하의 행사가를 가진 첫 번째 풋옵션 계약을 선택합니다. 옵션 비용이 존재하므로, 이 비용만큼 손절 가격을 더 보수적으로 설정합니다.
* 계약은 금요일 만료됩니다. 만약 만기 시점에 옵션이 ITM(가격 내)이면 자동 행사되어 주식을 매도하게 되고, OTM(가격 외)일 경우 무효화되며, 잔여 주식은 시장가 주문으로 청산됩니다.

```python
class CaseOfTheMondaysAlgorithm(QCAlgorithm):
    def initialize(self):
        # ...
        date_rule = self.date_rules.week_start(self._symbol)
        self.schedule.on(
            date_rule,
            self.time_rules.after_market_open(self._symbol, 2),
            self._enter
        )
        self.schedule.on(
            date_rule,
            self.time_rules.after_market_open(self._symbol, -30),
            self._liquidate_if_possible
        )
        self._model = Lasso(alpha=10 ** (-self.get_parameter('alpha_exponent', 4)))

    def _liquidate_if_possible(self):
        self.liquidate(self._symbol)
        for symbol, security_holding in self.portfolio.items():
            # 옵션이고, 미체결 주문이 없다면 청산
            if (security_holding.type == SecurityType.OPTION and
                not list(self.transactions.get_open_order_tickets(symbol))):
                self.liquidate(symbol)

    def _enter(self):
        # 오늘 시가에서 주간 저가까지의 수익률 예측을 위한 모델 훈련
        training_samples = self._samples.dropna()
        self._model.fit(
            training_samples.iloc[:, :-1],
            training_samples.iloc[:, -1]
        )
        prediction = self._model.predict(
            [self._samples.iloc[:, :3].dropna().iloc[-1]]
        )[0]
        predicted_low_price = self._security.open * (1 + prediction)
        self.plot("Stop Loss", "Distance", 1 + prediction)

        for chain in self.current_slice.option_chains.values():
            # KO 주식 매수
            quantity = self.calculate_order_quantity(self._symbol, 1)
            self.market_order(self._symbol, quantity)

            # 풋옵션 계약 선택
            puts = [
                contract for contract in chain
                if contract.strike < predicted_low_price + contract.ask_price
            ]
            contract = sorted(puts, key=lambda contract: contract.strike)[-1]

            # 풋옵션 계약 매수
            tag = f"Predicted week low price: {round(predicted_low_price, 2)}"
            self.market_order(contract.symbol, quantity // 100, tag=tag)
```

### Discussion of the Performance of Algorithm 1, Algorithm 2, and Algorithm 3

세 알고리즘은 독립적으로 거래를 수행하므로, 성능 비교가 용이합니다.

결과적으로, 풋옵션 헷지 전략은 스톱 마켓 주문 방식보다 낮은 성과를 보였습니다. Algorithm 1(고정 퍼센트 스톱로스)이 가장 우수한 성과를 보였습니다. 이 구현은 전략을 개선하기 위한 출발점이며, 추가적인 연구와 개선이 가능함을 시사합니다.


---


## Example 9—ML Trading Pairs Selection  

예상 전처리된 페어 클러스터 예측  
기술: PCA, 페어 클러스터링  
자산 클래스: 미국 주식  
난이도: 중-상  
유형: 리서치 노트북  
소스 코드: qnt.co/book-example9  

### Summary  

우리는 PCA를 활용하여 자산의 표준화된 수익률을 주성분으로 변환합니다. 이후 OPTICS를 사용하여 자산을 클러스터링하고, 공적분 검정을 수행한 뒤, 평균회귀 성향을 파악하기 위해 허스트 지수를 계산합니다. 마지막으로 해당 페어의 반감기와 12개월 간 스프레드 교차 횟수를 계산합니다.  

### Motivation  

금융 분야에서 PCA는 자산 가격 변동을 유도하는 근본적인 요인을 식별하는 데 필수적입니다. 주식 수익률의 세 가지 주요 주성분(PC)은 다음과 같이 해석할 수 있습니다:

1. 첫 번째 주성분 (PC1) — 시장 요인: 주식 수익률 데이터에서 가장 큰 분산을 설명하며, 이는 대다수의 주식이 시장 전반의 흐름을 따르기 때문에 전체 시장 트렌드를 반영하는 시장 요인으로 해석됩니다.
2. 두 번째 주성분 (PC2) — 섹터/산업 요인: 다음으로 큰 분산을 설명하며, 섹터 또는 산업별 요인으로 해석될 수 있습니다.
3. 세 번째 주성분 (PC3) — 스타일/규모 요인: 앞선 두 주성분으로 설명되지 않는 분산을 설명하며, 성장/가치 스타일이나 대형주/소형주의 차이에 따른 요인으로 해석됩니다.

주성분 변환 이후, 우리는 OPTICS 클러스터링 알고리즘을 사용하여 유사한 특성을 가진 자산을 그룹화합니다. 이렇게 유사한 자산을 그룹화하면 페어 트레이딩에 적합한 후보군을 식별할 수 있습니다.

공적분 검정은 단기적인 가격 변동에도 불구하고 장기적으로 균형 관계를 유지하는 자산 페어를 식별하는 데 중요합니다. 허스트 지수는 시계열이 평균으로 되돌아가는 성향(평균회귀) 또는 추세를 따르는 성향을 측정합니다. 공적분과 허스트 지수를 조합함으로써 평균회귀 특성을 엄격하게 검증할 수 있습니다.

마지막으로, 페어의 반감기와 12개월 동안의 스프레드 교차 횟수를 계산합니다. 반감기는 두 공적분된 자산 간의 스프레드가 평균으로 얼마나 빠르게 되돌아가는지를 측정하며, 짧을수록 트레이딩 기회가 더 자주 생겨 유리합니다. 스프레드 교차 횟수는 평균을 얼마나 자주 교차했는지를 의미하며, 많을수록 평균회귀 전략에 적합한 페어로 간주됩니다.

### Implementation Insights  

이 예제는 MLFinLab 블로그 글 (gnt.co/book-hudsonpairs)을 기반으로 하며, 머신러닝을 사용해 페어 트레이딩 전략에 적합한 주식 페어를 탐색합니다.  

### Step 1: 고려 자산의 일일 수익률 계산

2024년 1월 3일 기준으로 거래 중인 모든 자산을 선택합니다. 해당 자산의 과거 3년간 종가 데이터를 수집하고, 결측치가 있는 자산은 제거하여 최종 927개의 자산을 남깁니다. 이후 종가 데이터를 기반으로 일일 수익률을 계산합니다.

```python
qb = QuantBook()
end_date = datetime(2024, 1, 3)
universe_history = qb.universe_history(
    qb.universe.etf('IWB'), end_date - timedelta(1), end_date
)
symbols = [fundamental.symbol for fundamental in universe_history.iloc[0]]
prices = qb.history(
    symbols, end_date - timedelta(3 * 365), end_date, Resolution.DAILY
)['close'].unstack(0).dropna(axis=1)
daily_returns = prices.pct_change().dropna()
```

### Step 2: 일일 수익률 표준화

PCA 분석을 위한 표준화를 수행합니다.

```python
from sklearn.preprocessing import StandardScaler
standardized_returns = StandardScaler().fit_transform(daily_returns)
```

### Step 3: PCA 실행

데이터 차원을 세 가지 주성분으로 줄이기 위해 PCA를 적용합니다.

```python
from sklearn.decomposition import PCA
pca = PCA(n_components=3, random_state=0)
pca.fit(standardized_returns)
```

### Step 4: 주성분을 데이터프레임에 로드

각 자산이 각 주성분에 얼마나 노출되어 있는지 확인할 수 있도록 주성분을 데이터프레임에 저장합니다.

```python
factor_exposures = pd.DataFrame(
    index=[f"component_{i}" for i in range(pca.components_.shape[0])],
    columns=daily_returns.columns,
    data=pca.components_
).T
```

결과를 시각화합니다 (그림 6.30과 6.31).

```python
go.Figure(
    [
        go.Scatter3d(
            x=[row['component_0']],
            y=[row['component_1']],
            z=[row['component_2']],
            mode='markers',
            marker=dict(size=3, color='blue'),
            text=symbol,
            showlegend=False
        )
        for symbol, row in factor_exposures.iterrows()
    ],
    dict(
        scene=dict(
            xaxis_title='X',
            yaxis_title='Y',
            zaxis_title='Z',
            camera=dict(eye=dict(x=2, y=1.5, z=1.5))
        ),
        title='3D Representation of Assets<br><sup>The coordinates represent '
              + 'the contribution the asset has to each component</sup>',
        height=600,
        width=900
    )
).show()
```

> 각 `pca.components_`의 행은 하나의 주성분을 나타내며, 해당 행 내 값은 원본 피처들이 해당 주성분에 기여한 정도를 나타냅니다.

```python
factor_exposures
```

---

3D Representation of Assets
The coordinates represent the contribution the asset has to each component

<img src="./images/fig_06_30.png" width=800>

Figure 6.30 자산의 3D 공간 표현

<img src="./images/fig_06_31.png" width=800>

Figure 6.31 자산 좌표 (component\_0, component\_1, component\_2) 예시:

| Symbol           | component\_0 | component\_1 | component\_2 |
| ---------------- | ------------ | ------------ | ------------ |
| RPTMYV3VC57P A   | 0.036965     | 0.031454     | 0.030914     |
| R735QTJ8XC9X AA  | 0.040697     | -0.023039    | -0.035844    |
| WF7IHVI76I5H AA  | 0.031304     | -0.017483    | -0.042513    |
| VM9RIYHM8ACL AAL | 0.034789     | 0.006323     | -0.047596    |
| SA48O8J43YAT AAP | 0.027324     | -0.013965    | -0.000761    |
| ...              | ...          | ...          | ...          |

총 927개의 자산 × 3개의 주성분


### Step 5: Organize the assets into clusters using the OPTICS algorithm.

OPTICS 클러스터링 알고리즘을 사용하여, 자산들을 3차원 요인 노출(3D-factor exposures)을 기반으로 클러스터로 구성합니다 (그림 6.32).

```python
from sklearn.cluster import OPTICS
clustering = OPTICS().fit(factor_exposures)

# 결과를 표시합니다.
group_by_cluster_id = {
    cluster_id: factor_exposures.iloc[
        [i for i, x in enumerate(clustering.labels_) if x == cluster_id]
    ]
    for cluster_id in sorted(set(clustering.labels_))
}

go.Figure(
    [
        go.Scatter3d(
            x=group['component_0'],
            y=group['component_1'],
            z=group['component_2'],
            mode='markers',
            marker=dict(size=3),
            text=group.index,
            name=f"Group {cluster_id}" if cluster_id >= 0 else "Noisy group",
            visible=True if cluster_id >= 0 else 'legendonly'
        )
        for cluster_id, group in group_by_cluster_id.items()
    ],
    dict(
        scene=dict(
            xaxis_title='X',
            yaxis_title='Y',
            zaxis_title='Z',
            camera=dict(eye=dict(x=1.752, y=1.25, z=1.25))
        ),
        title='3D Representation of Assets<br><sup>The coordinates represent '
              + 'the contribution the asset has to each component</sup>',
        height=600,
        width=900
    )
).show()

# 노이즈 샘플 제거
labels = clustering.labels_[clustering.labels_ != -1]
print(
    f"{len(clustering.labels_)}개의 자산 중, OPTICS는 {len(labels)}개의 "
    f"노이즈가 아닌 샘플을 찾아 {len(set(labels))}개의 그룹으로 구성했습니다."
)
```

927개의 자산 중, OPTICS는 347개의 노이즈가 아닌 샘플을 찾아 44개의 그룹으로 구성했습니다.

<img src="./images/fig_06_32.png" width=800>

Figure 6.32 자산의 3차원 공간 내 표현. 그룹 클러스터별로 색상으로 구분됨.
### Step 6: Identify potential trading pairs.

자산들을 서로 다른 그룹으로 분리한 후, 이제 각 그룹 내에서 거래 후보 쌍을 탐색합니다. 각 그룹 내에서 가능한 모든 자산 조합을 분석합니다. (각 쌍은 동일한 그룹 내의 두 자산으로 구성됩니다.)

다음 네 가지 조건을 모두 만족하면 쌍을 선택합니다:

1. 두 자산이 공적분 관계를 가져야 합니다.
2. 두 자산 간의 스프레드의 허스트 지수가 평균회귀 특성을 나타내야 합니다.
3. 스프레드가 일정 시간 내에 확산 후 수렴해야 합니다.
4. 스프레드가 평균을 자주 교차해야 합니다.

각 테스트를 자세히 살펴보겠습니다.

**Test 6.1: 두 자산이 공적분 관계를 가져야 함**

이를 위해 Engle-Granger 테스트를 수행하며, 독립변수를 서로 바꾸어 두 번 테스트합니다. 그 중 통계값이 가장 낮은(가장 유의미한) 결과를 채택하며, p-value가 1% 이하이어야 합니다.

```python
from arch.unitroot.cointegration import engle_granger

cointegration_test_results = [
    engle_granger(prices[symbol_a], prices[symbol_b]),
    engle_granger(prices[symbol_b], prices[symbol_a])
]
cointegration_test_result = sorted(cointegration_test_results, key=lambda x: x.stat)[0]

if cointegration_test_result.pvalue > 0.01:
    test_results.loc[test_results_index[0], 'count'] += 1
    continue  # 테스트 실패: 공적분 아님
```

Engle-Granger 테스트는 두 비정상 시계열 간 공적분 관계를 검정하는 계량경제학적 방법입니다.
Wikipedia에 따르면:
"두 시계열이 각각 비정상(non-stationary)이지만, 어떤 선형 결합이 보다 낮은 차수의 통합성을 가지면 이들은 공적분 상태라 할 수 있다."

테스트는 두 단계로 구성됩니다.

1. 두 시계열이 1차 차분으로 정상성을 가진다고 가정한 후, 장기적인 선형 관계를 추정합니다.
2. 잔차가 정상성을 가지는지를 확인함으로써 안정적이고 장기적인 관계인지 검증합니다.

**Test 6.2: 스프레드의 허스트 지수가 평균회귀 특성을 가져야 함**

이를 위해, 앞선 회귀 분석으로 도출된 헤지 비율을 사용하여 스프레드를 계산하고, 허스트 지수를 구한 후 그 값이 0.5보다 작은지 확인합니다.

```python
spread = (
    prices[cointegration_test_result.cointegrating_vector.index[0]] +
    cointegration_test_result.cointegrating_vector.values[1] *
    prices[cointegration_test_result.cointegrating_vector.index[1]]
)

H, _, _ = compute_Hc(spread, kind='price', simplified=False)

if H >= 0.5:
    test_results.loc[test_results_index[1], 'count'] += 1
    continue  # 테스트 실패: 평균회귀 특성이 없음
```

허스트 지수(H)는 시계열의 장기적 기억(= 평균회귀 특성)을 나타냅니다.

* **H < 0.5** : 평균으로 되돌아가는 경향 (mean-reverting)
* **H == 0.5** : 랜덤 워크, 과거와 독립
* **H > 0.5** : 추세 지속 경향 (지속적인 상승 또는 하락)

**Test 6.3: 스프레드가 일정 시간 내에 확산 후 수렴해야 함**

이를 위해 스프레드의 반감기(평균으로 되돌아가는 데 걸리는 예상 시간)를 계산하고, 그 값이 1일 이상 1년(252일) 이하인지 확인합니다.

```python
lagged_spread = np.roll(spread, 1)
lagged_spread[0] = 0
spread_delta = spread - lagged_spread
spread_delta.iloc[0] = 0

# OLS 회귀 수행
model = OLS(spread_delta, add_constant(lagged_spread))
beta = model.fit().params.iloc[1]
half_life = -np.log(2) / beta

if not (1 < half_life < 252):
    test_results.loc[test_results_index[2], 'count'] += 1
    continue  # 테스트 실패: 반감기가 1일 ~ 1년 사이 아님
```

**Test 6.4: 스프레드가 평균을 자주 교차해야 함**

이를 위해, 스프레드가 평균을 교차한 횟수가 연평균 12회 이상(3년간 36회 이상)인지 확인합니다.

```python
mean_value = spread.mean()
crossings = ((spread > mean_value) & (spread.shift(1) <= mean_value)) | ((spread < mean_value) & (spread.shift(1) >= mean_value))
num_crossings = crossings.sum()

if num_crossings < 3 * 12:  # 3년간 36회 미만이면 실패
    test_results.loc[test_results_index[3], 'count'] += 1
    continue  # 테스트 실패: 평균 교차 횟수 부족
```

위 네 가지 조건을 모두 통과하면, 최종적으로 탐지된 쌍의 수, 테스트 통과/실패 통계 등을 시각화합니다 (그림 6.33).

테스트된 쌍 수: 1357

**그림 6.33**
쌍 탐지 프로세스의 출력

| 구간         | 실패 수 | 누적 실패 비율 |
| ---------- | ---- | -------- |
| 공적분 테스트 실패 | 1315 | 96.9%    |
| 허스트 테스트 실패 | 120  | 0.88%    |
| 반감기 테스트 실패 | 0    | 0.00%    |
| 평균 교차 실패   | 0    | 0.00%    |

탐지된 최종 거래 쌍 수: **30**

이후 각 쌍의 가격과 스프레드를 플로팅하여 시각적으로도 거래 쌍으로 적합한지를 확인합니다 (그림 6.34).

<img src="./images/fig_06_34_01.png" width=800>
<img src="./images/fig_06_34_02.png" width=800>
<img src="./images/fig_06_34_03.png" width=800>
<img src="./images/fig_06_34_04.png" width=800>
<img src="./images/fig_06_34_05.png" width=800>

**Figure 6.34** 후보 쌍의 가격과 정규화된 스프레드.

---


## Example 10—Stock Selection through Clustering Fundamental Data

### Summary

우리는 수백 개의 펀더멘털 지표를 주성분 분석(PCA)을 통해 주요 성분으로 축소합니다. 이후 생성된 벡터를 사용하여 주식의 상대적인 성과 순위를 예측합니다. 우리는 가장 높은 미래 수익률이 기대되는 주식 하위 집합으로 포트폴리오를 구성하기 위해 "랭킹 학습 알고리즘(Learning-to-Rank algorithm)"을 사용합니다.

### Motivation

주성분 분석(PCA)을 적용함으로써, 수백 개의 펀더멘털 지표를 주성분으로 축소하고, 주식 수익률의 주요 결정 요인을 포착하며, 잡음을 최소화합니다. 이렇게 얻은 주성분 벡터를 바탕으로 랭킹 학습 알고리즘을 활용하여 주식의 상대적 성과를 예측합니다. 랭킹 학습은 정보 검색과 머신러닝에서 자주 사용되는 기법으로, 관련성이 높은 항목이 위에 오도록 항목을 정렬합니다. 이 예제에서는 개별 주식의 절대적인 성과뿐 아니라 상대적인 성과도 예측할 수 있습니다. 가장 높은 순위를 받은 주식들로 포트폴리오를 구성함으로써, PCA의 데이터 축소 능력과 랭킹 학습 알고리즘의 정밀도를 활용하여 향후 수익을 극대화하는 것이 목표입니다.

### Model

* 모델: 시장에서 유동성이 가장 높은 100개 주식의 표준화된 100개 펀더멘털 지표로부터 추출된 상위 5개의 PCA 주성분.
* 예측 대상 레이블: 주식을 22 거래일 동안 보유한 후의 수익률을 기반으로 한 순위.
* 사용 모델: LGBMRanker.

LGBMRanker는 성능, 유연성, 효율성의 균형으로 선택되었습니다. 선형 회귀처럼 해석은 쉬우나 복잡한 관계를 잘 반영하지 못하는 단순 모델보다, LGBMRanker는 특성 간 비선형 상호작용을 효과적으로 모델링합니다. 속도 및 메모리 효율성 면에서 SVM이나 랜덤 포레스트보다 우수하며, 신경망보다 자원 소모도 적습니다. 하이퍼파라미터 튜닝이 필요하긴 하지만, 대규모 데이터셋에 대해 강력한 성능을 발휘합니다.

### Trading Universe

알고리즘은 매월 초에 펀더멘털 기반의 유니버스를 선정합니다. 먼저, 펀더멘털 데이터가 있는 자산 중 가장 유동성이 높은 상위 100개 주식을 선택합니다. 이후, 향후 수익률 기준으로 가장 높은 순위를 받은 10개 주식으로 축소합니다.

### Portfolio Construction

* **모델 학습 시점**: 매월 초, 10개 거래 주식을 선택한 후.
* **포트폴리오 리밸런싱 시점**: 매월 초, 시장 개장 1분 후.
* **포트폴리오 구성 방식**: 10개 주식으로 동일 비중(equal-weight) 포트폴리오 구성. 각 주식은 1/10 비중을 가짐.

### Tearsheet

결과는 다음을 보여줍니다:

* `final_universe_size` 및 `components` 파라미터가 가장 작은 값일 때 샤프 비율이 가장 높게 나타남.
* 모든 샤프 비율은 0 이상.
* `components` 파라미터의 작은 변화에도 성과가 민감함. 예: 유니버스 크기가 5일 때 `components`를 3에서 4로 바꾸면 샤프 비율이 0.45 → 0.09로 감소.
* 자세한 내용은 Figure 6.35 및 6.36 참조.

**Parameter final\_universe\_size**:

* 최소값: 5 (너무 작으면 소수 자산에 집중됨)
* 최대값: 25 (상위 25% 자산 선택하는 일반적 방법)
* 증가 단위: 5 (트레이더들이 선호할 만한 라운드 숫자)

**Parameter components**:

* 최소값: 3 (상위 3개 주성분이 전체 분산의 80% 이상 설명)
* 최대값: 7 (추가 성분은 설명력 감소)
* 증가 단위: 1 (가장 작은 단위)

**백테스트 설정**:

* liquid\_universe\_size: 100
* final\_universe\_size: 10
* lookback\_period: 365
* components: 5

<img src="./images/fig_06_35.png" width=800>

Figure 6.35 Example 10의 주식 곡선, 성과 플롯, 사용자 정의 플롯

<img src="./images/fig_06_36.png" width=800>

Figure 6.36 Example 10의 월간 수익률, 위기 시기, 민감도 분석

### Implementation Insights

우리는 SPY라는 보조 심볼 객체를 사용하여 트레이딩 캘린더를 추론합니다.

```python
schedule_symbol = Symbol.create("SPY", SecurityType.EQUITY, Market.USA)
date_rule = self.date_rules.month_start(schedule_symbol)
```

각 자산의 펀더멘털 팩터 히스토리는 딕셔너리에 저장되며, 키는 심볼이고 값은 각 팩터가 열로 구성된 pandas DataFrame입니다. 각 행은 특정 시점의 팩터 값을 나타냅니다.

```python
factors_by_symbol = {
    symbol: pd.DataFrame(columns=self._factors)
    for symbol in liquid_symbols
}
history = self.history[Fundamental](
    liquid_symbols, self._lookback_period + timedelta(2)
)
for fundamental_dict in history:
    for symbol, asset_fundamentals in fundamental_dict.items():
        factor_values = []
        for factor in self._factors:
            factor_values.append(eval(f"asset_fundamentals.{factor}"))
        t = asset_fundamentals.end_time
        factors_by_symbol[symbol].loc[t] = factor_values
```

PCA를 적용하려면 NaN이 없어야 하며, 20개 미만의 팩터가 남은 자산은 제외합니다. 이 과정의 결과는 `tradable_symbols`와 `factors_to_use`입니다.

```python
all_non_nan_factors = []
tradable_symbols = []
min_accepted_non_nan_factors = len(self._factors)

for symbol, factor_df in factors_by_symbol.items():
    non_nan_factors = set(factor_df.dropna(axis=1).columns)
    if len(non_nan_factors) < 20:
        continue
    min_accepted_non_nan_factors = min(
        min_accepted_non_nan_factors, len(non_nan_factors)
    )
    tradable_symbols.append(symbol)
    all_non_nan_factors.append(non_nan_factors)

if not all_non_nan_factors:
    return []

factors_to_use = all_non_nan_factors[0]
for x in all_non_nan_factors[1:]:
    factors_to_use = factors_to_use.intersection(x)
factors_to_use = sorted(list(factors_to_use))
```

모델은 자산을 한 달(22 거래일) 동안 보유한 후 수익률의 순위를 예측하도록 학습됩니다. 미래 수익률을 계산합니다.

```python
history = self.history(
    tradable_symbols,
    self._lookback_period + timedelta(1),
    Resolution.DAILY
)

label_by_symbol = {}
for symbol in tradable_symbols[:]:
    if symbol not in history.index:
        tradable_symbols.remove(symbol)
        continue
    open_prices = history.loc[symbol]['open'].shift(-1)
    label_by_symbol[symbol] = open_prices.pct_change(
        self._prediction_period
    ).shift(-self._prediction_period).dropna()
```

`self._prediction_period = 22`

이 코드는 `prediction_period` 간격으로 수익률을 계산하고 미래 값을 과거 시점에 맞게 시프트하며, NaN을 제거합니다.

팩터 행렬과 레이블 벡터를 결합하여 학습용 데이터 프레임을 생성합니다.

```python
X_train = pd.DataFrame()
y_train = pd.DataFrame()

for symbol in tradable_symbols:
    labels = label_by_symbol[symbol]
    factors = factors_by_symbol[symbol][factors_to_use].reindex(labels.index).ffill()
    X_train = pd.concat([X_train, factors])
    y_train[symbol] = labels

X_train = X_train.sort_index()
```

PCA를 통해 차원을 5개로 축소하고 데이터를 변환합니다.

```python
def initialize(self):
    self._components = self.get_parameter('components', 5)
    self._scaler = StandardScaler()
    self._pca = PCA(n_components=self._components, random_state=0)
```

```python
def _select_assets(self, fundamental):
    X_train_pca = self._pca.fit_transform(self._scaler.fit_transform(X_train))
```

LGBRanker를 사용하기 전에, 레이블을 랭킹 형태로 변환합니다.

```python
y_train = y_train.rank(axis=1, method='first').values.flatten() - 1
y_train = y_train[~np.isnan(y_train)]
```

* `.rank(axis=1, method='first')`: 행마다 랭크 부여
* `.values`: DataFrame → NumPy 배열
* `.flatten()`: 1차원 배열로 평탄화
* `-1`: 랭크를 0부터 시작하도록 조정

LGBMRanker 모델을 LambdaRank 목표로 학습시킵니다.

```python
model = LGBMRanker(
    objective="lambdarank",
    label_gain=list(range(len(tradable_symbols)))
)
group = X_train.reset_index().groupby("time")["time"].count()
model.fit(X_train_pca, y_train, group=group)
```

이제 다음 달을 위한 자산 순위를 예측할 수 있습니다.

```python
X = pd.DataFrame()
for symbol in tradable_symbols:
    X = pd.concat(
        [X, factors_by_symbol[symbol][factors_to_use].iloc[-1:]]
    )

prediction_by_symbol = {
    tradable_symbols[i]: prediction
    for i, prediction in enumerate(
        model.predict(self._pca.transform(self._scaler.transform(X)))
    )
}
```

랭킹이 높은 자산을 선택합니다.

```python
sorted_predictions = sorted(
    prediction_by_symbol.items(), key=lambda x: x[1]
)
```

상위 10개의 자산을 선택합니다.

```python
return [x[0] for x in sorted_predictions[-self._final_universe_size:]]
```

`final_universe_size`는 `initialize` 메서드에서 다음과 같이 설정됩니다.

```python
self._final_universe_size = self.get_parameter('final_universe_size', 10)
```

---


## Example 11—Inverse Volatility Rank and Allocate to Future Contracts

### Summary

우리는 향후 1주일간의 예상 시가 변동성의 역수(inverse)에 비례하여 가중치를 부여한 선물 계약 포트폴리오를 구성합니다.
향후 1주일간의 시가 변동성은 과거 종가 기준 변동성, 평균 진폭 범위(ATR), 표준편차, 미결제약정(Open Interest)을 기반으로 한 릿지 회귀(Ridge Regression) 모델을 통해 예측합니다.

### Motivation

이 전략은 변동성이 낮은 자산이 보다 안정적이고 예측 가능한 수익을 제공한다는 가정 하에, 역변동성 순위를 활용합니다.
예상 변동성에 역비례하여 자산에 자본을 배분함으로써 전체 포트폴리오의 리스크를 자연스럽게 줄이고, 동시에 샤프 비율을 향상시키는 경향이 있습니다.
트레이딩 유니버스는 전체 포트폴리오에 대한 단일 자산의 영향력을 제한하기 위해 광범위하게 분산되어 구성되었습니다.

### Model

* **입력 특징(Features)**: 최근 3개월간의 일간 수익률 표준편차 (변동성)
* **예측 대상**: 향후 시가(open price)의 변동성
* **레이블(Label)**: 예측된 시가 변동성
* **모델**: Ridge Regression

추가적으로 다음 요소들이 특징으로 사용됩니다:

* **ATR (Average True Range)**:

  * True range는 다음 중 최대값:

    * 당일 고가 - 저가
    * 당일 저가 - 전일 종가의 절댓값
    * 당일 고가 - 전일 종가의 절댓값
  * ATR은 위 true range의 지수이동평균(EMA)

* **미결제약정(Open Interest)**: 현재까지 청산되지 않은 파생계약 수. 포지션 시작부터 종료까지 "열려 있음"으로 간주

**Ridge 회귀 선택 이유**:

1. 특징 간 다중공선성(multi-collinearity)을 효과적으로 처리할 수 있음
2. 정규화 항(불필요한 특징의 계수를 축소시킴)을 포함하여 과적합 방지 및 안정성 확보
3. 단순하고 계산 효율이 높음

### Trading Universe

다음 선물의 근월물(front-month contracts)로 제한된 유니버스를 사용합니다:

* 지수:

  * VIX
  * S\&P 500 E-Mini
  * Nasdaq 100 E-Mini
  * DOW 30 E-Mini
* 에너지:

  * Brent crude
  * Gasoline
  * Heating oil
  * Natural gas
* 곡물:

  * Corn
  * Oats
  * Soybeans
  * Wheat

### Portfolio Construction

| 모델 학습 시점                                                    | 매주 첫 거래일, 시장 개장 2분 후 |
| ----------------------------------------------------------- | -------------------- |
| 포트폴리오 리밸런싱 시점                                               | 모델 학습 직후             |
| 포트폴리오 가중치                                                   |                      |
| 각 자산의 가중치는 해당 자산의 변동성에 기반하여 다음과 같이 설정됩니다:                   |                      |
|   **weight = 3 / σ / sum(σ) / Futures Contract Multiplier** |                      |

이 수식은 변동성이 낮은 자산에 더 큰 비중을 부여하도록 설계됨.

* 분자 3은 포지션 크기 조절용 스케일링 계수
* 값이 1이면 최소 주문 마진 비율 때문에 일부 자산만 거래됨
* 값이 너무 크면 마진 콜 위험 발생
* **3**은 대부분의 선물을 거래하면서도 최소 주문 마진 설정을 0으로 만들지 않는 적정값으로 선택됨

### Tearsheet

결과는 다음을 보여줍니다:

* **샤프 비율은 보통 하나 이상의 지표가 6개월 회고 기간을 사용할 때 가장 높음**
* **샤프 비율은 지표 중 하나라도 3개월 회고 기간이면 가장 낮은 경향**
* **모든 파라미터 조합은 수익성이 있음**

**Figure 6.37**, **Figure 6.38** 참조

**파라미터 std\_months** (표준편차 계산용 기간):

* 최소값: 1 (가장 작은 정수)
* 최대값: 6 (6개월)
* 스텝: 1

**파라미터 atr\_months** (ATR 계산용 기간):

* 최소값: 1
* 최대값: 6
* 스텝: 1

**백테스트에 사용된 파라미터**:

* `std_months`: 3
* `atr_months`: 3
* `training_set_duration`: 365

<img src="./images/fig_06_37.png" width=800>

Figure 6.37 Example 11의 주식 곡선, 성과 플롯, 사용자 정의 플롯

<img src="./images/fig_06_38.png" width=800>

Figure 6.38 Example 11의 월간 수익률, 위기 이벤트, 민감도 테스트

### Implementation Insights

우리는 지표 체이닝(chaining)을 통해 변동성 모델의 피처를 구현합니다.

### **Step 1**: 일일 종가와 시가 수익률에 대한 지표를 생성합니다.

```python
security.close_roc = RateOfChange(1)
security.open_roc = RateOfChange(1)
```

### **Stem 2**: 일일 종가 및 시가 수익률의 표준편차(Standard Deviation)에 대한 지표를 생성합니다.

```python
security.std_of_close_returns = IndicatorExtensions.of(
    StandardDeviation(self._std_period),
    security.close_roc
)
security.std_of_open_returns = IndicatorExtensions.of(
    StandardDeviation(self._future_std_period),
    security.open_roc
)
```

**ATR 지표는 다음과 같이 구현합니다**:

```python
security.atr = AverageTrueRange(self._atr_period)
```

여기서 `self._atr_period`는 기본적으로 3개월(3 × 26일)로 설정되며 `initialize` 메서드에서 다음과 같이 정의됩니다:

```python
def initialize(self):
    # ...
    self._atr_period = self.get_parameter('atr_months', 3) * 26
```

**지표 데이터는 다음과 같이 일 단위로 통합하여 저장됩니다**:

```python
security.indicator_history = pd.DataFrame()
security.label_history = pd.Series()
security.consolidator = self.consolidate(
    security.symbol, Resolution.DAILY,
    self._consolidation_handler
)
```

각 종목의 `indicator_history`에는 일자별 atr, std\_of\_close\_returns 등의 피처가 저장되며, `label_history`는 향후 1주간의 시가 변동성을 포함합니다.

```python
def _consolidation_handler(self, consolidated_bar):
    security = self.securities[consolidated_bar.symbol]
    t = consolidated_bar.end_time

    if security.atr.update(consolidated_bar):
        security.indicator_history.loc[t, 'atr'] = security.atr.current.value

    security.close_roc.update(t, consolidated_bar.close)
    if security.std_of_close_returns.is_ready:
        security.indicator_history.loc[t, 'std_of_close_returns'] = security.std_of_close_returns.current.value

    security.open_roc.update(t, consolidated_bar.open)
    if (security.std_of_open_returns.is_ready and
        len(security.indicator_history.index) > self._future_std_period):
        security.label_history.loc[
            security.indicator_history.index[-self._future_std_period - 1]
        ] = security.std_of_open_returns.current.value

    security.indicator_history = security.indicator_history[
        (security.indicator_history.index >= self.time - self._training_set_duration)
    ]
    security.label_history = security.label_history[
        (security.label_history.index >= self.time - self._training_set_duration)
    ]
```

**주요 구현 사항 요약**:

* `update()` 메서드는 지표가 준비되었을 때 `True`를 반환하며, 이 경우만 기록합니다.
* `open_roc`는 결과를 바로 확인하지 않고, 종속 지표인 `std_of_open_returns`가 준비되었는지로 판단합니다.
* 미래 시점의 변동성 예측을 위해 인덱스를 과거로 시프트합니다.
* 오래된 학습 데이터는 `self._training_set_duration` 기준으로 필터링합니다.

**모델 학습 전, 모든 활성 선물 계약의 Open Interest 데이터를 가져옵니다**:

```python
open_interest = self.history(
    OpenInterest, [c.symbol for c in self._contracts],
    self._training_set_duration, fill_forward=False
)
open_interest.index = open_interest.index.droplevel(0)
```

데이터프레임 인덱스는 (만기, 심볼, 시간)의 3중 멀티인덱스이며, 여기서 만기(expiry) 레벨은 제거합니다.

**지표 데이터와 open\_interest를 열 기준(axis=1)으로 병합하여 학습용 피처를 구성합니다**:

```python
factors = pd.concat(
    [security.indicator_history, open_interest.loc[symbol]],
    axis=1
).ffill().loc[security.indicator_history.index].dropna()
```

* 결측치는 `ffill()`로 보완
* 인덱스는 지표 데이터에 맞춰 정렬
* NaN 행 제거

**레이블 구조를 생성합니다**:

```python
label = security.label_history
idx = sorted(
    list(set(factors.index).intersection(set(label.index)))
)
if len(idx) < 20:
    continue
```

레이블과 피처가 모두 있는 날짜만 사용하며, 20개 이상일 때만 학습에 사용합니다.

**모델을 학습하고 예측값을 저장합니다**:

```python
model = Ridge()
model.fit(factors.loc[idx], label.loc[idx])
prediction = model.predict([factors.iloc[-1]])[0]

if prediction > 0:
    expected_volatility_by_security[security] = prediction
```

예상 변동성을 계산한 후, 해당 결과를 기반으로 포트폴리오 리밸런싱 주문을 실행할 수 있습니다.

---


## Example 12—Trading Costs Optimization

### Summary

이 알고리즘은 머신러닝을 활용하여 거래 비용을 줄이는 방법을 보여줍니다. 우리는 다음과 같은 요소들을 기반으로 거래 비용을 예측하는 `DecisionTreeRegressor` 모델을 구축합니다:

* 절대 주문 수량
* 평균 진폭 범위(ATR)
* 일 평균 거래량
* 스프레드 비율 ((ask - bid)/bid)
* 주문 상위 호가 규모 (달러 기준)

그 후, 투자 금액당 예측된 비용이 최근 10 거래일 평균 거래 비용보다 낮은 경우에만 주문을 실행합니다. 자세한 분석을 위해 연구용 Jupyter Notebook에 광범위한 데이터를 저장합니다.

### Motivation

금융 거래에서 비용 최소화는 수익 극대화를 위한 핵심 요소입니다. 이 알고리즘은 머신러닝 기반 `DecisionTreeRegressor` 모델을 활용해 거래 비용을 줄이는 전략을 수행합니다.

거래 비용 모델링은 다양한 시장 요소에 의해 영향을 받는 실행 비용을 예측하는 것을 의미하며, 본 알고리즘은 다음 요소들을 고려합니다:

* **절대 주문 수량**: 주문 규모가 크면 시장 충격과 유동성 제약으로 인해 비용이 증가할 수 있습니다.
* **ATR**: 시장 변동성을 측정하여, 가격 움직임에 따른 비용 변동성을 반영합니다.
* **일 평균 거래량**: 거래량이 많을수록 유동성이 높고 비용은 낮아지는 반면, 거래량이 낮으면 슬리피지와 시장 충격으로 비용 증가 가능성이 있습니다.
* **스프레드 비율 ((ask - bid)/bid)**: 매수호가와 매도호가 간 차이를 나타내며, 스프레드가 넓을수록 거래 비용이 큽니다.
* **상위 호가 규모**: 최우선 호가에서의 유동성 수준을 나타내며, 주문 크기에 대한 즉시 실행 비용을 가늠할 수 있게 해줍니다.

### Model

**모델 입력 특징**:

* 절대 주문 수량
* 평균 진폭 범위 (ATR)
* 일 평균 거래량
* 스프레드 비율 ((ask - bid)/bid)
* 상위 호가 규모 (달러 기준)

**예측 대상 (레이블)**: 실제 거래 비용
**사용 모델**: `DecisionTreeRegressor`

### Trading Universe

BTCUSD 한 종목만 거래합니다.

### Portfolio Construction

| 항목         | 내용                     |
| ---------- | ---------------------- |
| 모델 학습 시점   | 매월 1일 0시               |
| 포트폴리오 리밸런싱 | 매일 자정과 새로운 시장 데이터 수신 시 |
| 포지션 규모     | 매수 주문당 10 BTC 매수       |

### Trading Logic

이 알고리즘은 두 가지 모드로 실행됩니다:

**Mode 1 (벤치마크 실행)**:

* `self._benchmark = True`로 설정
* 매일 자정에 비트코인 10개를 매수하고, 오전 1시에 청산
* 이 과정에서 주문 체결 가격, 수량, 비용을 기록
* 알고리즘 종료 시, 모든 데이터는 Jupyter Notebook 분석을 위해 QuantConnect의 Object Store에 저장됨

**Mode 2 (후보 알고리즘 — 거래 비용 최적화)**:

* 벤치마크 실행 후 `self._benchmark = False`로 설정
* 자정에 비트코인 10개를 매수
* 이후, 오전 1시\~오후 11:59 사이 중 예측된 청산 비용이 평균보다 낮은 시점에 포지션을 청산
* 오후 11:59까지 청산하지 못한 경우, 예측과 관계없이 무조건 청산하며 이 주문에는 `"Hit time limit"` 태그를 부여

즉, **Mode 1**은 매일 무조건 청산하는 단순 기준으로 ML 알고리즘의 성능 비교를 위한 기준선이며,
**Mode 2**는 청산 시점을 최적화하여 거래 비용을 낮추는 "지능형" 알고리즘입니다.
### Implementation Insights

우리는 시장 스프레드가 거래 체결 가격에 미치는 영향을 시뮬레이션하기 위해 `SpreadSlippageModel`을 정의합니다. 이는 시장가 주문 환경에서 높은 거래 비용을 가정하기 위한 프록시로 사용됩니다.

```python
class SpreadSlippageModel:
    def get_slippage_approximation(self, asset, order):
        return asset.ask_price - asset.bid_price
```

**Mode 2 (후보 알고리즘 — 거래 비용 최적화)**는 **Mode 1 (벤치마크 알고리즘)**과 동일한 거래를 시작 시점에 수행하다가, 충분한 샘플이 수집되면 모델을 학습하고 이후부터는 거래를 지연시켜 비용을 절감하려 시도합니다.

후보 알고리즘은 `DecisionTreeRegressor` 모델을 사용해 거래 비용을 예측합니다.

* 모델은 주문 비용(커미션 + 슬리피지)을 예측하도록 학습되며, 결과는 `_costs`에 저장됩니다.
* 예측된 거래 비용은 "투자금 1달러당 예상 비용"으로 변환됩니다.
* 모델은 이 예측 비용이 최근 10일간 체결된 평균 비용보다 낮을 경우에만 거래를 수행합니다.

**모든 주문은 `_order_fills` 데이터프레임에 다음과 같은 열로 저장됩니다**:

* `fill_price`
* `quantity`
* `cost`
* `tag` — 예: "Hit time limit"처럼 해당 주문에 대한 설명을 기록

알고리즘 종료 시점에 이 데이터는 CSV 파일로 저장됩니다.

```python
def on_end_of_algorithm(self):
    key = ("benchmark" if self._benchmark else "candidate") + "_order_fills"
    self._order_fills.to_csv(self.object_store.get_file_path(key))
```

QuantConnect 백테스트 결과는 "Costs" 및 "Costs per Dollar" 차트로 알고리즘의 효과를 시각적으로 보여줍니다.
Figure 6.39는 모델이 충분한 샘플을 수집한 이후 비용이 감소하기 시작하는 모습을 보여줍니다.

<img src="./images/fig_06_39.png" width=800>

### **Step 1: 주문 CSV 파일 로드**

```python
qb = QuantBook()
def get_order_fills(key):
    return pd.read_csv(
        qb.object_store.get_file_path(f"{key}_order_fills"),
        index_col=0, parse_dates=True
    )

benchmark_orders = get_order_fills("benchmark")
candidate_orders = get_order_fills("candidate")
```

### **Step 2: 요약 통계 계산**

```python
same_index = candidate_orders.index == benchmark_orders.index
matching_fill_times = candidate_orders[same_index].index

different_fill_times_for_candidate = candidate_orders[~same_index].index
different_fill_times_for_benchmark = benchmark_orders[~same_index].index

print(f"Number of trades: {len(benchmark_orders)}")
print("Number of trades with the same fill times:", len(matching_fill_times))
print("Number of trades with different fill times:", len(different_fill_times_for_candidate))

costs_saved = sum(benchmark_orders['cost'].values - candidate_orders['cost'].values)
print("Costs saved by delaying orders:", f"${round(costs_saved, 2)}")

candidate_quantities = candidate_orders['quantity'].values
benchmark_quantities = benchmark_orders['quantity'].values
if not all(candidate_quantities == benchmark_quantities):
    raise Exception('Error: The algorithms traded different quantities')

candidate_info = candidate_orders.loc[matching_fill_times].drop('tag', axis=1)
benchmark_info = benchmark_orders.loc[matching_fill_times].drop('tag', axis=1)
if not all(candidate_info == benchmark_info):
    raise Exception('Error: Fill prices and costs of identical orders do not match')
```

**출력 예시**:

* 거래 수: 1827
* 동일한 시간에 체결된 거래 수: 798
* 시간 차가 있는 거래 수: 1029
* 지연을 통해 절약된 비용: $36429.70


### **Step 3: 시간 제한 이전에 체결된 지연 주문 분석 (Figure 6.40)**

```python
from collections import Counter
import plotly.graph_objects as go

index_numbers_of_time_limited_orders = []
index_numbers_of_delayed_orders = []

for i in range(len(benchmark_orders.index)):
    if benchmark_orders.index[i] == candidate_orders.index[i]:
        continue
    if "Hit time limit" in candidate_orders.iloc[i]['tag']:
        index_numbers_of_time_limited_orders.append(i)
        continue
    index_numbers_of_delayed_orders.append(i)

def display_order_results(title, index_numbers):
    cost_deltas = (
        benchmark_orders.iloc[index_numbers]['cost'].values
        - candidate_orders.iloc[index_numbers]['cost'].values
    )
    candidate_dollar_volume = (
        abs(candidate_orders.iloc[index_numbers]['quantity'])
        * candidate_orders.iloc[index_numbers]['fill_price']
    ).values

    cost_deltas_per_dollar_volume = cost_deltas / candidate_dollar_volume

    fig = go.Figure(data=[
        go.Histogram(x=cost_deltas_per_dollar_volume, nbinsx=100)
    ])
    fig.update_layout(
        title=f"Distribution of Costs Saved<br><sup>Costs saved for Delayed Orders that {title}</sup>",
        xaxis_title="Costs Saved Per Dollar Volume (>0 = Candidate Saved Money)",
        yaxis_title="Count"
    )
    fig.show()

    print("Number of orders:", len(index_numbers))
    print("Number of orders that lowered costs:",
          len(cost_deltas[cost_deltas > 0]),
          f"({round(len(cost_deltas[cost_deltas > 0]) / len(candidate_orders) * 100, 2)}% of all orders)")
    print("Number of orders that didn't change costs:",
          len(cost_deltas[cost_deltas == 0]),
          f"({round(len(cost_deltas[cost_deltas == 0]) / len(candidate_orders) * 100, 2)}% of all orders)")
    print("Number of orders that raised costs:",
          len(cost_deltas[cost_deltas < 0]),
          f"({round(len(cost_deltas[cost_deltas < 0]) / len(candidate_orders) * 100, 2)}% of all orders)")
    print(f"Total costs saved: ${round(sum(cost_deltas), 2)}")

display_order_results("Filled Before the Time Limit", index_numbers_of_delayed_orders)
```

```python
minutes_delayed = (
    candidate_orders.iloc[index_numbers_of_delayed_orders].index -
    benchmark_orders.iloc[index_numbers_of_delayed_orders].index
)
num_orders_by_delay_duration = pd.Series(
    Counter([x.total_seconds() / 60 for x in minutes_delayed])
).sort_index()

fig = go.Figure(
    go.Scatter(
        x=num_orders_by_delay_duration.index,
        y=num_orders_by_delay_duration.values,
        mode='lines'
    )
)
fig.update_layout(
    title="Number of Orders Filled Per Delay Duration<br><sup>Most orders fill within 90 minutes</sup>",
    xaxis_title="Minutes Delayed",
    yaxis_title="Number of Orders Filled"
)
fig.show()
```

<img src="./images/fig_06_40.png" width=800>

**분석 결과**:

* 주문 수: 815
* 비용 절감된 주문 수: 774 (42.36%)
* 비용 변동 없음: 28 (1.53%)
* 비용 증가: 13 (0.71%)
* 총 절감 비용: $36260.20

<img src="./images/fig_06_41.png" width=800>


### **Step 4: 시간 제한에 도달하여 체결된 주문 분석 (Figure 6.42)**

```python
display_order_results("Hit the Time Limit", index_numbers_of_time_limited_orders)
```

<img src="./images/fig_06_42.png" width=800>


**분석 결과**:

* 주문 수: 214
* 비용 절감된 주문 수: 90 (4.93%)
* 비용 변동 없음: 43 (2.35%)
* 비용 증가: 81 (4.43%)
* 총 절감 비용: $169.50

---

## Example 13—PCA Statistical Arbitrage Mean Reversion

### Summary

이 알고리즘은 주성분 분석(PCA)과 선형 회귀를 활용한 통계적 차익거래(statistical arbitrage)를 설명합니다. 통계적 차익거래 전략은 평균회귀 모델을 사용하여 관련된 증권 간의 가격 차이를 활용합니다.

### Motivation

이 알고리즘은 주성분 분석(PCA)과 선형 회귀를 활용하여 효과적인 통계적 차익거래 전략을 구현합니다.
처음 세 개의 주성분은 증권 가격 변동의 근본적인 요인을 나타내며, 이는 정확한 모델링과 예측을 위해 매우 중요합니다. 선형 회귀 모델은 이 주성분들과 과거 가격 데이터를 이용해 각 증권의 가격을 예측합니다. 실제 가격과 예측 가격 간의 차이인 잔차(residual)를 비교하여 이상치를 식별합니다.

본 전략의 핵심 개념인 평균회귀는 시간이 지나면서 가격이 장기 평균으로 되돌아온다는 가정을 따릅니다. 따라서 잔차가 클 경우, 가격이 평균으로 복귀할 가능성이 높다고 판단합니다.

이 잔차를 기반으로 각 주식에 가중치를 부여하며, 가격이 평균으로 회귀할 때 수익을 기대합니다. 이 체계적인 방식은 일시적인 가격 왜곡을 이용하여 수익 가능성을 극대화하는 포트폴리오 조정 전략입니다.

### Model

* **입력 특징(Features)**: 표준화된 로그 수익률의 첫 세 개 PCA 주성분
* **예측 대상(Label)**: 표준화된 로그 수익률
* **모델**: 선형 회귀

선형 회귀는 단순하고 해석이 용이하며 계산 효율성이 높아 기본 모델로 선택되었습니다. 그러나 상황에 따라 Ridge 회귀, LASSO, 랜덤 포레스트 회귀, SVM 회귀, 신경망 등 대안 모델을 고려할 수 있습니다.

### Trading Universe

매월 초, 가격이 5달러 이상이고 유동성이 가장 높은 상위 100개 종목을 거래 유니버스로 선택합니다.

### Portfolio Construction

| 항목            | 내용                                                           |
| ------------- | ------------------------------------------------------------ |
| 모델 학습 시점      | 매월 첫 거래일, 시장 개장 1분 후                                         |
| 포트폴리오 리밸런싱 시점 | 모델 학습 직후                                                     |
| 포트폴리오 가중치     | 잔차의 Z-점수(z-score)가 -1.5 이하인 종목만 선택<br>각 종목은 Z-점수에 따라 가중치를 부여 |

잔차의 Z-점수가 -1.5보다 작은 종목, 즉 평균보다 훨씬 낮은 로그 수익률을 가진 종목을 선택합니다. 이는 평균회귀 관점에서 상승 가능성이 높다고 판단되는 종목들입니다.

### Trading Logic

매월 첫 거래일, 시장 개장 1분 후 과거 가격 데이터를 수집한 뒤 PCA를 수행하여 모든 종목의 로그 수익률을 첫 세 개 주성분으로 변환합니다.
그 후 각 종목에 대해 주성분을 입력으로 선형 회귀 모델을 학습하고, 잔차가 통계적으로 유의미한 종목을 선택하여 포트폴리오 가중치를 조정합니다.
자세한 내용은 [gnt.co/book-stat-arb-usequities](https://gnt.co/book-stat-arb-usequities)를 참고하세요.

### Tearsheet

결과 요약:

* Sharpe 비율은 `num_components=3`과 `lookback_days=126(6개월)`일 때 최대
* `lookback_days=63(3개월)` 또는 `84(4개월)`에서는 Sharpe 비율이 일반적으로 낮음
* 모든 파라미터 조합에서 수익 발생

**파라미터 설명**:

* **num\_components**

  * 최소값: 2 (다중 선형 회귀 가능 최소 주성분 수)
  * 최대값: 6 (첫 3개가 90% 이상의 분산을 설명하며, 이후는 효과 미미)
  * 증가 단위: 1

* **lookback\_days**

  * 최소값: 21 (1개월)
  * 최대값: 126 (6개월)
  * 증가 단위: 21

**백테스트 설정**:

* `num_components`: 3
* `lookback_days`: 63
* `z_score_threshold`: 1.5
* `universe_size`: 100

<img src="./images/fig_06_43.png" width=800>

Figure 6.43 Example 13의 주가 곡선, 성과 그래프, 사용자 정의 플롯

<img src="./images/fig_06_44.png" width=800>

Figure 6.44 Example 13의 월간 수익률, 위기 이벤트, 민감도 테스트
### Implementation Insights

트레이딩 알고리즘은 세 단계로 구성되어 있다:
Step 1: 과거 가격 데이터를 가져온다.

```python
tradeable_assets = [
    symbol
    for symbol in self._universe.selected
    if (self.securities[symbol].price and
        symbol in self.current_slice.quote_bars)
]
history = self.history(
    tradeable_assets, self._lookback, Resolution.DAILY,
    data_normalization_mode=DataNormalizationMode.SCALED_RAW
).close.unstack(level=0)
```

선택된 `data_normalization_mode`는 분할 및 배당 조정을 반영한 과거 가격 데이터를 요구한다.

Step 2: PCA를 수행하고, 각 종목에 대해 PCA 이후 선형 회귀의 잔차로부터 벗어난 정도에 따라 자산과 그 가중치를 선택한다.

```python
def _get_weights(self, history):
    # PCA용 샘플 데이터를 가져오고 np.log로 평활화
    sample = np.log(history.dropna(axis=1))
    sample -= sample.mean()  # 열 단위 중심화

    # 샘플 데이터로 PCA 모델 학습
    model = PCA().fit(sample)

    # 첫 n_components 요인을 가져옴
    factors = np.dot(sample, model.components_.T)[:, :self._num_components]

    # 절편을 위한 1 추가
    factors = sm.add_constant(factors)

    # 각 종목별로 OLS 선형 모델 학습
    model_by_ticker = {
        ticker: sm.OLS(sample[ticker], factors).fit()
        for ticker in sample.columns
    }

    # 각 종목에 대한 PCA 이후의 잔차 계산
    resids = pd.DataFrame(
        {ticker: model.resid for ticker, model in model_by_ticker.items()}
    )

    # 잔차를 표준화하여 Z-score 계산 (가장 최근 일자 기준)
    zscores = ((resids - resids.mean()) / resids.std()).iloc[-1]

    # 평균으로부터 멀리 떨어진 종목 선택 (평균 회귀 기대)
    selected = zscores[zscores < -self._z_score_threshold]

    # 선택된 종목의 가중치 계산 및 반환
    weights = selected * (1 / selected.abs().sum())
    return weights.sort_values()
```

### Step 3: Enter the position.

선택된 각 종목에 대해 표준화된 잔차가 0에서 벗어나 있으면, 평균 회귀를 기대하며 반대 방향으로 포지션을 진입한다.

```python
self.set_holdings(
    [
        PortfolioTarget(symbol, -weight)
        for symbol, weight in weights.items()
    ],
    True
)
```

---

## Example 14—Temporal CNN Prediction

| 예측 대상  | 가격 방향성                |
| :----- | :-------------------- |
| 기술     | 합성곱 신경망 (CNN)         |
| 자산 클래스 | 미국 주식                 |
| 난이도    | 어려움                   |
| 전략 유형  | 전체 전략                 |
| 소스 코드  | qnt.co/book-example14 |
### Summary

우리는 과거의 시계열 주가 정보(OHLCV: 시가, 고가, 저가, 종가, 거래량)를 기반으로 미래 주가 방향(상승, 하락, 정체)을 예측하기 위해 Temporal CNN을 적용한다.

### Motivation

주가가 오를지 내릴지를 예측할 수 있다면, 투자자는 증권을 사고팔거나 보유할지에 대한 결정을 더 잘 내릴 수 있다. 올바른 시점에 포지션을 진입 및 청산함으로써 수익을 극대화할 수 있다.

이 트레이딩 알고리즘은 과거 OHLCV 데이터를 기반으로 주가가 상승, 하락 또는 정체될지를 예측하기 위해 Temporal CNN을 사용한다.

Temporal CNN을 선택한 이유는 다음과 같다:

* **시간 패턴 처리**: 시계열 데이터를 다루도록 설계되어 다양한 시간 범위의 패턴을 인식할 수 있다.
* **특징 추출**: CNN은 원시 OHLCV 데이터로부터 유의미한 특징을 자동으로 추출할 수 있다.
* **비선형 관계 모델링**: 주가 움직임에서 나타나는 복잡한 비선형 관계를 모델링할 수 있다.
* **노이즈에 대한 강건성**: 의미 있는 패턴에만 집중함으로써 데이터의 노이즈를 제거할 수 있다.

CNN은 이 사례에서 다른 모델보다 우수한 성능을 보인다:

* 선형 회귀는 주가 데이터의 비선형 관계로 인해 적합하지 않다.
* 결정 트리나 랜덤 포레스트는 과적합되기 쉽고 시간 패턴을 학습하지 못한다.
* 서포트 벡터 머신은 비선형 분류는 잘하지만, 시퀀스 데이터를 자연스럽게 처리할 수 없다.

### Model

모델 특징
각 종목에 대해 다음 다섯 가지 요인을 사용한다:

* 시가 (Open)
* 고가 (High)
* 저가 (Low)
* 종가 (Close)
* 거래량 (Volume)

예측 레이블: 상승 / 하락 / 정체
모델: Temporal CNN

Temporal CNN은 개별 주식의 가격 움직임을 모델링하기 위해 사용된다:

* "Temporal"은 시간이 지남에 따라 변화하는 데이터를 처리한다는 의미이다.
* "CNN"은 합성곱 계층을 사용하여 특징을 추출한다는 의미이다.

각 종목에 대해 입력값으로 아래 다섯 가지 요인을 사용한다:

```python
factor_names = ['open', 'high', 'low', 'close', 'volume']
```

예측할 가격 방향은 다음과 같이 정수로 표현된다:

```python
class Direction:
    UP = 0
    DOWN = 1
    STATIONARY = 2
```

이 모델에는 Keras 딥러닝 라이브러리를 사용했다.

입력 레이어는 OHLCV 각 요인의 15일간 데이터를 받으며, 입력 형태는 (15, 5)이다:

```python
inputs = Input(shape=(15, 5))
```

입력 레이어는 특징 추출용 CNN 계층으로 전달되고, 시간 기준으로 long-term, mid-term, short-term 세 영역으로 나누어 처리한다:

```python
long_term = Lambda(lambda x: tf.split(x, num_or_size_splits=3, axis=1)[0])(feature_extraction)
mid_term = Lambda(lambda x: tf.split(x, num_or_size_splits=3, axis=1)[1])(feature_extraction)
short_term = Lambda(lambda x: tf.split(x, num_or_size_splits=3, axis=1)[2])(feature_extraction)

long_term_conv = Conv1D(1, 1, activation='relu')(long_term)
mid_term_conv = Conv1D(1, 1, activation='relu')(mid_term)
short_term_conv = Conv1D(1, 1, activation='relu')(short_term)
```

다음으로, 이 세 개의 계층을 결합하고 평탄화한다:

```python
combined = Concatenate(axis=1)([long_term_conv, mid_term_conv, short_term_conv])
flattened = Flatten()(combined)
```

출력 레이어는 3개의 클래스(상승, 정체, 하락)를 위한 소프트맥스 출력이다:

```python
outputs = Dense(3, activation='softmax')(flattened)
```

전체 모델은 다음과 같이 정의된다:

```python
def _create_model(self):
    """신경망 모델을 생성한다."""
    inputs = Input(shape=(self._n_tsteps, len(factor_names)))
    
    # CNN을 사용한 특징 추출
    feature_extraction = Conv1D(30, 4, activation='relu')(inputs)
    
    # 시간 기준 세 영역으로 분할
    long_term = Lambda(lambda x: tf.split(x, num_or_size_splits=3, axis=1)[0])(feature_extraction)
    mid_term = Lambda(lambda x: tf.split(x, num_or_size_splits=3, axis=1)[1])(feature_extraction)
    short_term = Lambda(lambda x: tf.split(x, num_or_size_splits=3, axis=1)[2])(feature_extraction)

    long_term_conv = Conv1D(1, 1, activation='relu')(long_term)
    mid_term_conv = Conv1D(1, 1, activation='relu')(mid_term)
    short_term_conv = Conv1D(1, 1, activation='relu')(short_term)

    # 세 계층 결합
    combined = Concatenate(axis=1)([long_term_conv, mid_term_conv, short_term_conv])
    
    # 평탄화
    flattened = Flatten()(combined)
    
    # 출력 레이어
    outputs = Dense(3, activation='softmax')(flattened)

    # 모델 정의 및 컴파일
    self._cnn = Model(inputs=inputs, outputs=outputs)
    self._cnn.compile(
        optimizer='adam',
        loss=CategoricalCrossentropy(from_logits=True)
    )
```

<img src="./images/fig_06_45.png" width=800>

그림 6.45: Temporal CNN 모델의 아키텍처
여기서 우리는 다중 클래스 분류 문제에 사용되는 `CategoricalCrossentropy` 함수를 사용하고 있으며, 타겟 레이블은 one-hot 인코딩된 형태로 `utils.to_categorical`을 통해 변환할 예정이다.
### Trading Universe

각 주가 시작될 때, QQQ ETF의 구성 종목 중 비중이 가장 높은 3개 종목을 선택한다.

### Portfolio Construction

**모델 학습**: 매주 첫 거래일 오전 9시에 수행됨.
**포트폴리오 리밸런싱 시점**: 매주 첫 거래일, 시장이 개장한 후 2분 뒤.
**포트폴리오 가중치**: 예측된 방향(상승 또는 하락)과 그에 대한 신뢰도를 기반으로 각 종목에 가중치를 할당하고, 전체 포트폴리오의 균형을 맞추기 위해 가중치의 절대값 합이 1 이하가 되도록 조정한다.

### Trading Logic

우리는 매주 첫 거래일, 시장 개장 2분 후에 거래를 수행한다.

```python
def initialize(self):
    #...
    self.schedule.on(
        date_rule, self.time_rules.after_market_open(etf, 2), self._trade
    )

def _trade(self):
    # 모든 종목에 대해 예측 수행
    weight_by_symbol = {}
    for symbol in self._universe.selected:
        security = self.securities[symbol]
        symbol_df = security.history.tail(15)
        prediction, confidence = security.cnn.predict(symbol_df)
        if (prediction != Direction.STATIONARY and
            not math.isnan(confidence) and
            confidence > 0.55):
            factor = (-1 if prediction == Direction.DOWN else 1)
            weight_by_symbol[security.symbol] = factor * confidence

    self.plot("Confidence", str(security.symbol.id), confidence)

    # 포트폴리오 가중치 계산 및 리밸런싱
    weight_sum = sum([abs(x) for x in weight_by_symbol.values()])
    weight_factor = 1 if weight_sum <= 1 else 1 / weight_sum
    portfolio_targets = [
        PortfolioTarget(symbol, weight * weight_factor)
        for symbol, weight in weight_by_symbol.items()
    ]
    self.set_holdings(portfolio_targets, True)
```

각 선택된 종목에 대해 다음 작업이 수행된다:

* `trade` 메서드는 최근 15일간의 OHLCV 데이터를 가져온다.
* 그 후 모델의 예측 메서드를 호출한다.
* 예측 방향이 `STATIONARY`가 아니고 신뢰도가 55%를 초과하면, 상승 예측인 경우는 신뢰도만큼의 양수 가중치를, 하락 예측인 경우는 -신뢰도만큼의 음수 가중치를 부여한다.
* 마지막으로 모든 종목의 타겟 가중치를 정규화하며, 가중치는 신뢰도에 비례하고 전체 절대값의 합이 1이 되도록 보장된다.

또한 이 트레이딩 전략은 주식 분할(splits)을 처리하는 예제도 제공한다:

```python
def on_splits(self, splits):
    for symbol, split in splits.items():
        if split.type == SplitType.SPLIT_OCCURRED:
            self._initialize_security(self.securities[symbol])
```

### Tearsheet

결과는 다음을 보여준다:

* 학습 샘플 수가 증가할수록 Sharpe 비율이 일반적으로 증가한다.
* 대부분의 파라미터 조합은 수익성이 없다.
* Figure 6.46 및 6.47 참조.

**파라미터 `training_samples`**:

* 최소값은 300으로, 이는 연간 거래일 수보다 약간 많기 때문이다.
* 최대값은 700으로, 너무 커져서 학습 속도가 느려지지 않게 하기 위한 적절한 크기다.
* 스텝 크기는 100으로, 적절한 라운드 넘버를 생성하기 위함이다.

**파라미터 `universe_size`**:

* 최소값은 2로, 알고리즘이 항상 투자 상태가 되도록 하기 위함이다. 하나의 종목만 있고 예측이 `stationary`이면, 리밸런싱 시 투자되지 않은 상태가 될 수 있다.
* 최대값은 10으로, 10개를 초과하면 백테스트에 1시간 이상이 걸리기 때문이다.
* 스텝 크기는 2로, 2에서 10 사이의 `universe_size`를 다섯 가지 값으로 나눌 수 있게 한다.

**백테스트 파라미터**:

* `training_samples`: 500
* `universe_size`: 3

<img src="./images/fig_06_46.png" width=800>

Figure 6.46: 예제 14의 주식 커브, 성과 차트, 커스텀 차트

<img src="./images/fig_06_47.png" width=800>

Figure 6.47: 예제 14의 월간 수익률, 금융 위기 이벤트, 민감도 분석

### Implementation Insights

모델 클래스의 생성자에서 `_create_model()` 메서드를 호출한다.

모델 학습을 위해 시계열 데이터를 모델이 수용 가능한 형식으로 인코딩해야 한다:

* 예측할 레이블을 생성한다.
* 5일 이동 평균 가격을 계산한다. 실제 가격보다 더 안정적이므로 5일 이동 평균을 선택한다.
* 이동 평균 종가의 수익률을 계산하고, 과거 데이터를 기반으로 미래의 상승/하락을 예측하는 모델을 만들기 위해 시계열을 뒤로 시프트한다.
* 수익률이 정체 임계값 이상/이하일 경우 각각 UP/DOWN으로 라벨링한다.
* 실제 시장 데이터를 2D 행렬 형태로 구성하고, `StandardScaler`를 이용해 정규화하여 학습 알고리즘의 수렴을 보장한다. 결과를 반환한다.

```python
def _prepare_data(self, data, rolling_avg_window_size=5, stationary_threshold=.0001):
    df = data[factor_names]
    shift = -(rolling_avg_window_size - 1)

    def label_data(row):
        if row['close_avg_change_pct'] > stationary_threshold:
            return Direction.UP
        elif row['close_avg_change_pct'] < -stationary_threshold:
            return Direction.DOWN
        else:
            return Direction.STATIONARY

    df['close_avg'] = df['close'].rolling(window=rolling_avg_window_size).mean().shift(shift)
    df['close_avg_change_pct'] = (df['close_avg'] - df['close']) / df['close']
    df['movement_labels'] = df.apply(label_data, axis=1)

    data = []
    labels = []
    for i in range(len(df) - self._n_tsteps + 1 + shift):
        label = df['movement_labels'].iloc[i + self._n_tsteps - 1]
        data.append(df[factor_names].iloc[i:i + self._n_tsteps].values)
        labels.append(label)

    data = np.array(data)
    dim1, dim2, dim3 = data.shape
    data = data.reshape(dim1 * dim2, dim3)
    data = self._scaler.fit_transform(data)
    data = data.reshape(dim1, dim2, dim3)
    return data, utils.to_categorical(labels, num_classes=3)
```

`sci-kit`의 `StandardScaler`는 2D 데이터에만 적용 가능하므로 데이터를 일시적으로 2D로 변환한 후 다시 복원해야 한다:

```python
dim1, dim2, dim3 = data.shape
data = data.reshape(dim1 * dim2, dim3)
data = self._scaler.fit_transform(data)
data = data.reshape(dim1, dim2, dim3)
```

학습 코드는 데이터를 준비하고, 생성자에서 만든 모델에 준비된 데이터를 학습시키는 것으로 구성된다:

```python
def train(self, data):
    data, labels = self._prepare_data(data)
    self._cnn.fit(data, labels, epochs=20)
```

데모 목적상 에포크 수는 20으로 설정했다.

마지막으로, 예측 코드는 매우 직관적이다:

* 시장 데이터를 정규화하기 전에 결측치를 앞의 값으로 채운다(`ffill`).
* 그런 다음, Keras의 예측 메서드를 사용해 레이블과 신뢰도를 반환한다.

```python
def predict(self, input_data):
    input_data = self._scaler.transform(
        input_data.fillna(method='ffill').values
    )
    prediction = self._cnn.predict(input_data[np.newaxis, :])[0]
    direction = np.argmax(prediction)
    confidence = prediction[direction]
    return direction, confidence
```

---

## Example 15—Gaussian Classifier for Direction Prediction

Direction Prediction

| 예측 대상  | 수익률 방향성                                 |
| :----- | :-------------------------------------- |
| 기술     | 가우시안 나이브 베이즈 분류기 (Gaussian Naive Bayes) |
| 자산 클래스 | 미국 주식                                   |
| 난이도    | 어려움                                     |
| 전략 유형  | 전체 전략                                   |
| 소스 코드  | qnt.co/book-example15                   |

### Summary

기술주 클래스의 일일 수익률을 예측하기 위해 Gaussian Naive Bayes (GNB) 분류기를 적용한다 (예측 결과: 양수, 음수, 정체).

### Motivation

이 예제는 이전 예제와 동일한 문제를 해결하지만, CNN 대신 GNB 분류기를 사용하여 기술주의 일일 수익률을 양/음/정체로 분류한다.

GNB는 단순함, 효율성, 고차원 데이터 처리 능력 덕분에 이 작업에 매우 적합하다.

CNN과 달리, GNB 모델은 단순하고 계산 비용이 적으며 해석이 용이하다. 이는 GNB가 특성(feature)이 정규 분포를 따르고 서로 독립이라는 가정을 하기 때문이다. 또한 GNB는 수작업 특징 추출과 전처리에 의존하며, CNN은 원시 데이터에서 자동으로 유의미한 특징을 추출한다.

GNB와 CNN 중 선택은 전략의 목적, 데이터 특성(GNB 가정에 적합한지 여부), 계산 자원, 해석 가능성과 예측력 간의 중요도 등에 따라 달라진다.

### Model

모델 입력 특징: 하루 수익률 4개 (시가 대비 종가 기준)
예측 레이블: 향후 22 거래일 내 수익률 방향을 나타내는 부호값 $(-1, 0, 1)$ (보유 기간은 30일 기준)
모델: GNB 분류기
이 모델은 다음 논문에서 영감을 받았다 (qnt.co/book-stock-market-index-direction 참조)

### Trading Universe

매주 시작 시 Morningstar 기준 기술 섹터 내에서 시가총액 상위 10개 종목을 선택한다.

### Portfolio Construction

| 모델 학습 시간      | 매주 첫 거래일 오전 9시                               |
| :------------ | :------------------------------------------- |
| 포트폴리오 리밸런싱 시간 | 매주 첫 거래일, 시장 개장 후 2분 뒤                       |
| 포트폴리오 가중치     | 모델이 양의 수익률(+1)을 예측한 모든 종목에 대해 동등 가중 포트폴리오 구성 |

### Tearsheet

결과는 다음과 같다:

* Sharpe 비율은 universe 크기가 작을 때(5) 가장 높았다. 이는 universe가 커질수록 모델의 독립 변수 수가 증가해 노이즈가 많아지고 학습이 어려워지기 때문으로 보인다.
* 모든 파라미터 조합이 수익성을 보였다.
* Figure 6.48과 6.49 참조

파라미터 `days_per_sample`:

* 최소값은 2로, 자산당 2개의 특징이 있기 때문
* 최대값은 8로, 이보다 크면 특징 수가 너무 많아져 모델에 노이즈가 많아질 수 있음
* 스텝 크기는 1로, 가장 작은 단위의 변화

파라미터 `universe_size`:

* 최소값은 5로, 그보다 작으면 일부 리밸런싱 시 현금 보유 상태가 될 수 있음
* 최대값은 25로, 너무 많은 자산을 포함하면 독립 변수 수가 지나치게 많아짐
* 스텝 크기는 5로, 5 단위로 변화시키기 적절함

백테스트 파라미터:

* days\_per\_sample: 4
* samples: 100
* universe\_size: 10

<img src="./images/fig_06_48.png" width=800>

Figure 6.48: 예제 15의 주식 커브, 성과 차트, 커스텀 차트

<img src="./images/fig_06_49.png" width=800>

Figure 6.49: 예제 15의 월간 수익률, 금융 위기 이벤트, 민감도 분석

### Implementation Insights

이 알고리즘은 백테스트 모드와 실거래(live) 모드 간의 차이를 보여준다. 실거래 모드일 경우 `live_mode` 필드는 `True`이며, 학습된 모델을 저장하기 위해 `pickle` 파일을 사용한다. 이 파일은 `initialize()`에서 불러오고, `_train`에서 업데이트하며, `on_end_of_algorithm`에서 저장한다.

```python
def initialize(self):
    # ...
    if self.live_mode:
        self._models_by_symbol = {}
        self._key = 'gnb_models.pkl'
        if self.object_store.contains_key(self._key):
            self._models_by_symbol = pickle.loads(
                self.object_store.read_bytes(self._key)
            )
```

```python
def _train(self):
    # ...
    for security in self._tradable_securities:
        symbol = security.symbol
        security.model = GaussianNB().fit(
            features.loc[idx],
            labels_by_symbol[symbol].loc[idx]
        )
        if self.live_mode:
            key = str(symbol.id)
            self._models_by_symbol[key] = pickle.dumps(security.model)
```

```python
def on_end_of_algorithm(self):
    if self.live_mode:
        self.object_store.save_bytes(
            self._key,
            pickle.dumps(self._models_by_symbol)
        )
```

이 예제에서는 커스텀 특징들을 사용한다. 예: 당일 시가 대비 종가 수익률, 전일 대비 시가 변화의 부호. 이들을 위해 intraday 가격 데이터를 직접 집계해야 한다.

트레이딩 유니버스에 새로운 종목이 추가되면, 해당 종목에 대해 일별 데이터 집계기(consolidator)를 설정하고 모델 학습용 특징과 라벨 데이터를 초기화한다.

```python
def on_securities_changed(self, changes):
    for security in changes.added_securities:
        security.model = None
        self._set_up_consolidator(security)
        self._warm_up(security)
    for security in changes.removed_securities:
        self.subscription_manager.remove_consolidator(
            security.symbol, security.consolidator
        )
```

각 시장 데이터가 수신될 때마다 집계기가 호출되어 해당 종목의 학습용 데이터 구조를 업데이트한다.

```python
def _set_up_consolidator(self, security):
    security.consolidator = self.consolidate(
        security.symbol, Resolution.DAILY, self._consolidation_handler
    )

def _consolidation_handler(self, bar):
    security = self.securities[bar.symbol]
    time = bar.end_time
    open_ = bar.open
    close = bar.close

    open_close_return = (close - open_) / open_
    if not self._update_features(security, time, open_close_return):
        return

    open_days = security.previous_opens[
        security.previous_opens.index <= time - timedelta(self._holding_period)
    ]
    if len(open_days) == 0:
        return
    open_day = open_days.index[-1]
    previous_open = security.previous_opens[open_day]

    open_open_return = (open_ - previous_open) / previous_open
    security.labels_by_day[open_day] = np.sign(open_open_return)
    security.labels_by_day = security.labels_by_day[-self._samples:]
    security.previous_opens.loc[time] = open_
    security.previous_opens = security.previous_opens[-self._holding_period:]
```

```python
def _update_features(self, security, day, open_close_return):
    security.roc_window = np.append(open_close_return, security.roc_window)[:self._days_per_sample]
    if len(security.roc_window) < self._days_per_sample:
        return False
    security.features_by_day.loc[day] = security.roc_window
    security.features_by_day = security.features_by_day[
        -(self._samples + self._holding_period + 2):
    ]
    return True
```

`_warm_up` 메서드는 모델 학습을 위한 핵심 데이터 구조 (`labels_by_day`, `features_by_day`) 와 보조 데이터 구조 (`roc_window`, `previous_opens`)를 초기화하고, 이를 과거 일별 데이터로 채운다.

```python
def _warm_up(self, security):
    security.roc_window = np.array([])
    security.previous_opens = pd.Series()
    security.labels_by_day = pd.Series()
    security.features_by_day = pd.DataFrame({
        f'{security.symbol.id}_(t-{i})': [] for i in range(1, self._days_per_sample + 1)
    })

    history = self.history(
        security.symbol, self._lookback, Resolution.DAILY,
        data_normalization_mode=DataNormalizationMode.SCALED_RAW
    )
    if history.empty or 'close' not in history:
        self.log(f"Not enough history for {security.symbol} yet")
        return

    history = history.loc[security.symbol]
    history['open_close_return'] = (history.close - history.open) / history.open
    start = history.shift(-1).open
    end = history.shift(-22).open
    history['future_return'] = (end - start) / start

    for day, row in history.iterrows():
        security.previous_opens[day] = row.open
        if not self._update_features(security, day, row.open_close_return):
            continue
        if not pd.isnull(row.future_return):
            security.labels_by_day[day] = np.sign(row.future_return)

    security.labels_by_day = security.labels_by_day[-self._samples:]
    security.previous_opens = security.previous_opens[-self._holding_period:]
```

주식 분할이 발생하면 관련 데이터 구조를 재설정한다.

```python
def on_splits(self, splits):
    for symbol, split in splits.items():
        if split.type != SplitType.SPLIT_OCCURRED:
            continue
        security = self.securities[symbol]
        self.subscription_manager.remove_consolidator(
            symbol, security.consolidator
        )
        self._set_up_consolidator(security)
        self._warm_up(security)
```

모델은 매 거래일 오전 9시에 학습되며, `_is_ready` 조건이 만족되었을 때만 실행된다.

```python
def initialize(self):
    schedule_symbol = Symbol.create("SPY", SecurityType.EQUITY, Market.USA)
    date_rule = self.date_rules.week_start(schedule_symbol)
    self.train(date_rule, self.time_rules.at(9, 0), self._train)

def _is_ready(self, security):
    return security.features_by_day.shape[0] == self._samples + self._holding_period + 2
```

```python
def _train(self):
    """
    GNB 모델을 학습한다.
    """
    features = pd.DataFrame()
    labels_by_symbol = {}
    self._tradable_securities = []

    for symbol in self._universe.selected:
        security = self.securities[symbol]
        if self._is_ready(security):
            self._tradable_securities.append(security)
            features = pd.concat([features, security.features_by_day], axis=1)
            labels_by_symbol[symbol] = security.labels_by_day

    features.dropna(inplace=True)

    idx = set(features.index)
    for symbol, labels in labels_by_symbol.items():
        idx &= set(labels.index)
    idx = sorted(list(idx))

    for security in self._tradable_securities:
        symbol = security.symbol
        security.model = GaussianNB().fit(
            features.loc[idx],
            labels_by_symbol[symbol].loc[idx]
        )
        if self.live_mode:
            key = str(symbol.id)
            self._models_by_symbol[key] = pickle.dumps(security.model)
```
### Trading

우리는 매 거래일, 시장 개장 2분 후에 거래를 수행한다.

```python
def initialize(self):
    # ...
    schedule_symbol = Symbol.create("SPY", SecurityType.EQUITY, Market.USA)
    date_rule = self.date_rules.week_start(schedule_symbol)
    self.schedule.on(
        date_rule,
        self.time_rules.after_market_open(schedule_symbol, 2),
        self._trade
    )

def _trade(self):
    # 특징(feature) 추출
    features = [[]]
    for security in self._tradable_securities:
        features[0].extend(security.features_by_day.iloc[-1].values)

    # 모델이 양의 수익률을 예측한 자산만 선택
    long_symbols = []
    for security in self._tradable_securities:
        # 실시간 거래에서 _train이 호출되지 않았다면, 저장소에서 모델을 불러온다.
        key = str(security.symbol.id)
        if (self.live_mode and
            not hasattr(security, 'model') and
            key in self._models_by_symbol):
            security.model = pickle.loads(self._models_by_symbol[key])

        # 모델 예측이 1이면 해당 종목을 long 대상으로 저장
        if security.model.predict(features) == 1:
            long_symbols.append(security.symbol)

    if len(long_symbols) == 0:
        return

    # 포트폴리오 리밸런싱
    weight = 1 / len(long_symbols)
    self.set_holdings(
        [PortfolioTarget(symbol, weight) for symbol in long_symbols],
        True
    )
```

---

## Example 16—LLM Summarization of Tiingo News Articles

Tiingo 뉴스 기사

| 예측 대상  | 뉴스 감정 (Sentiment)     |
| :----- | :-------------------- |
| 기술     | 대형 언어 모델 (LLM)        |
| 자산 클래스 | 미국 주식                 |
| 난이도    | 중상                    |
| 전략 유형  | 리서치 노트북               |
| 소스 코드  | gnt.co/book-example16 |

### Summary

우리는 미국 기술 기업과 관련된 대체 뉴스 데이터를 기반으로 OpenAI GPT-4 모델을 활용한 롤링 감정 분석 모델을 구축하고, 이 감정 추세에 따라 거래를 수행한다.

### Motivation

트레이더는 전통적인 금융 지표를 넘어서 통찰을 확보하기 위해 비전통적인 데이터 소스를 활용해 경쟁 우위를 얻고자 한다.

뉴스 데이터에 감성 분석을 적용하면, 시장이 이벤트와 발표에 대해 감정적으로 어떻게 반응하는지를 포착할 수 있다. 이를 통해 다가오는 시장 트렌드, 특히 하락 가능성에 대한 조기 탐지와 함께, 뉴스가 투자자 심리에 미치는 영향을 정량화할 수 있다.

### Model

| 모델 특징  | 뉴스 기사            |
| :----- | :--------------- |
| 예측 레이블 | -10 \~ 10 범위의 점수 |

* -10: 매우 부정적 감정
* 0: 중립
* 10: 매우 긍정적 감정 |
  \| 모델 | GPT-4 |

### Trading Universe

우리는 TSLA 주식만 거래 대상에 포함하며, 롱 또는 숏 포지션을 취한다.

### Portfolio Construction

| 모델 학습         | 리서치 노트북 내에서 필요 시 즉시 재계산 |
| :------------ | :---------------------- |
| 포트폴리오 리밸런싱 주기 | 매시간                     |

**포트폴리오 가중치**

* 감정 점수가 증가 중이고 현재 롱 TSLA 포지션이 아니라면, 포트폴리오의 100%를 롱 TSLA에 할당
* 감정이 부정적이고 점수가 감소 중이며 현재 숏 포지션이 아니라면, 포트폴리오의 100%를 TSLA 숏 포지션에 할당
### Trading Logic

리서치 노트북은 각 날짜에 대해 다음과 같은 세 개의 열을 가진 CSV 파일을 생성한다:

* `hour` (인덱스): 각 행은 특정 시각(시간 단위)에 해당한다.
* `sentiment`: 해당 시간 동안 분석된 모든 뉴스 기사들의 집계된 감정 점수. 감정 점수는 -10(매우 부정적)부터 +10(매우 긍정적)까지이며, 0은 중립적인 감정을 의미한다.
* `volume`: 해당 시간 동안 분석된 뉴스 기사 수.

트레이딩 알고리즘은 오브젝트 스토어의 해당 CSV 파일을 모니터링하고, 시장이 개장한 이후 업데이트가 발생할 때 다음 조건을 검사한다:

* 감정 점수가 **정체 또는 증가** 중이고, 아직 **롱 포지션이 아닌 경우**, 롱 포지션을 진입한다.
  (감정 점수가 음수여도 트렌드가 상승 중이면 가격도 반등할 것으로 기대하므로, 감정 값이 음수인지는 확인하지 않는다.)
* 감정 점수가 **음수이며**, **감정 추세가 하락 중이고**, 아직 **숏 포지션이 아닌 경우**, 숏 포지션을 진입한다.

```python
if self._roc.current.value >= 0 and not self._tsla.holdings.is_long:
    self.set_holdings(self._tsla.symbol, 1)
elif (sentiment < 0 and
      self._roc.current.value < 0 and
      not self._tsla.holdings.is_short):
    self.set_holdings(self._tsla.symbol, -1)
```

### Tearsheet

이 전략은 TSLA 벤치마크 전략(TSLA 단순 매수 및 보유)에 비해 **위험 대비 수익률** 측면에서 우수한 성과를 보였다:

* 이 알고리즘의 샤프 비율은 **1.695**
* 반면, 벤치마크(TSLA 매수 후 보유 전략)의 샤프 비율은 **-0.06**
* Figure 6.51 참조

<img src="./images/fig_06_50.png" width=800>

Figure 6.50: 예제 16의 주식 커브, 성과 차트, 커스텀 차트

<img src="./images/fig_06_51.png" width=800>

Figure 6.51: 예제 16의 월간 수익률

### Implementation Insights

우리는 리서치 환경(`research.ipynb`)에서 필요한 데이터를 수집하고, 모델을 학습시킨 뒤 결과를 QuantConnect의 오브젝트 스토어에 저장하는 것으로 시작한다.

### Step 1: 분석 대상 자산에 대한 뉴스 기사 구독**

우리는 TSLA에 대한 TiingoNews 데이터를 구독한다. 즉, TSLA가 언급된 뉴스 기사를 2023년 11월 1일부터 2024년 3월 1일까지의 백테스트 구간 동안 수집한다.

```python
qb = QuantBook()
symbol = qb.add_equity("TSLA").symbol
dataset_symbols = qb.add_data(TiingoNews, symbol).symbol
news_articles = qb.history[TiingoNews](
    dataset_symbols,
    datetime(2023, 11, 1),
    datetime(2024, 3, 1),
    Resolution.DAILY
)
```

### Step 2: 날짜별로 뉴스 기사 그룹화 및 중복 제거

시간 단위의 감정 점수를 생성하기 위해, 우선 날짜별로 뉴스 기사를 그룹화하고 중복된 기사들을 제거한다. 그 결과는 `deduplicated_articles_by_date`라는 딕셔너리에 저장한다.
### Step 3: Get hourly sentiment values from OpenAI.

기사를 시간 단위로 그룹화하는 이유는 다음과 같다:

1. 시간별 감정 점수가 더 안정적이다.
2. OpenAI API 호출 횟수를 줄일 수 있다.

우리는 각 시간 단위 내의 모든 기사를 ChatGPT에 전달하고 다음과 같은 프롬프트로 시간별 감정 점수를 요청한다:

```
Article <i> title: <제목>  
Article <i> description: <설명>
```

프롬프트 마지막에는 다음과 같이 지시한다:

```
위 기사 제목과 설명을 검토하고 TSLA에 대한 감정의 긍정 정도를 나타내는 집계 감정 점수를 생성하세요.  
-10은 극도로 부정적, +10은 극도로 긍정적, 0은 중립을 의미합니다.  
**숫자 값만 JSON 형식으로 응답하세요.**  
예: `{ "sentiment-score": 0 }`
```

이후 점수는 QuantConnect의 Object Store에 CSV로 저장된다:

```python
from openai import OpenAI
client = OpenAI(api_key="<your_api_key>")

for date, articles in deduplicated_articles_by_date.items():
    print(date)
    articles_by_hour = {}
    for article in articles:
        hour = article.end_time.hour
        if hour not in articles_by_hour:
            articles_by_hour[hour] = []
        articles_by_hour[hour].append(article)

    sentiment_by_hour = pd.DataFrame(dtype=float)

    for hour, articles in articles_by_hour.items():
        prompt = ""
        for i, article in enumerate(articles):
            prompt += (
                f"Article {i+1} title: {article.title}\n"
                + f"Article {i+1} description: {article.description}\n\n"
            )
        prompt += (
            "Review the news titles and descriptions above and then create an "
            + "aggregated sentiment score which represents the emotional "
            + "positivity towards TSLA after seeing all of the news articles. "
            + "-10 represents extreme negative sentiment, +10 represents "
            + "extreme positive sentiment, and 0 represents neutral sentiment. "
            + 'Reply ONLY with the numerical value in JSON format. For example, `{ "sentiment-score": 0 }`'
        )

        chat_completion = client.chat.completions.create(
            messages=[{"role": "user", "content": prompt}],
            model="gpt-4"
        )
        sentiment = json.loads(
            chat_completion.choices[0].message.content
        )['sentiment-score']

        sentiment_by_hour.loc[hour, 'sentiment'] = sentiment
        sentiment_by_hour.loc[hour, 'volume'] = len(articles)

    file_path = qb.object_store.get_file_path(
        f"tiingo-{date.strftime('%Y-%m-%d')}.csv"
    )
    sentiment_by_hour.to_csv(file_path)
```

이제 QuantConnect 백테스트 환경에서 이 CSV 파일들을 사용할 준비가 되었다.

### Step 4: Apply the trained results in a backtesting algorithm.

이제 우리는 백테스트 환경으로 전환하여 ChatGPT로부터 생성된 감정 데이터를 사용하는 알고리즘을 구성한다. `main.py` 파일에서 Object Store의 CSV 파일을 읽고 알고리즘에 주입하기 위한 커스텀 데이터 클래스를 정의한다:

```python
class TiingoNewsSentiment(PythonData):
    def get_source(self, config, date, is_live):
        return SubscriptionDataSource(
            f"tiingo-{date.strftime('%Y-%m-%d')}.csv",
            SubscriptionTransportMedium.OBJECT_STORE,
            FileFormat.CSV
        )

    def reader(self, config, line, date, is_live):
        if line[0] == ",":
            return None
        data = line.split(',')
        t = TiingoNewsSentiment()
        t.symbol = config.symbol
        t.time = date.replace(hour=int(data[0]), minute=0, second=0)
        t.end_time = t.time + timedelta(hours=1)
        t.value = float(data[1])
        t["sentiment"] = t.value
        t["volume"] = float(data[2])
        return t
```

`RateOfChange` 지표를 적용하여 감정 변화 방향을 관찰한다:

```python
def initialize(self):
    self.set_start_date(2023, 11, 1)
    self.set_end_date(2024, 3, 1)
    self.set_cash(100_000)

    self._tsla = self.add_equity("TSLA")
    self._dataset_symbol = self.add_data(
        TiingoNewsSentiment, "TiingoNewsSentiment", Resolution.HOUR
    ).symbol
    self._roc = self.roc(self._dataset_symbol, 2)
    self.set_benchmark(self._tsla.symbol)
```

트레이딩 로직은 다음과 같다:

* 감정이 **정체 또는 상승** 중이고 현재 롱 포지션이 아니면 **롱 진입**
* 감정이 **음수이고 감소 중**이며 현재 숏 포지션이 아니면 **숏 진입**

```python
def on_data(self, data):
    if self._dataset_symbol not in data:
        return

    sentiment = data[self._dataset_symbol].value

    if not self.is_market_open(self._tsla.symbol):
        return

    if self._roc.current.value >= 0 and not self._tsla.holdings.is_long:
        self.set_holdings(self._tsla.symbol, 1)
    elif (sentiment < 0 and
          self._roc.current.value < 0 and
          not self._tsla.holdings.is_short):
        self.set_holdings(self._tsla.symbol, -1)
```

---

## Example 17—Head Shoulders Pattern Matching with CNN

| Predicting  | Pattern matching      |
| :---------- | :-------------------- |
| Technology  | CNN                   |
| Asset Class | Forex                 |
| Difficulty  | Hard                  |
| Type        | Research notebook     |
| Source Code | qnt.co/book-example17 |

### Summary

이 예제는 일차원 CNN을 사용하여 기술적 헤드 앤 숄더(H\&S) 트레이딩 패턴을 감지하는 방법을 소개한다. 우리는 합성 학습 데이터를 구성한 후, 외환 시장에서 유사한 H\&S 패턴을 감지하도록 CNN을 학습시킨다.

### Motivation

기술적 분석은 과거의 가격 및 거래량 데이터를 이용해 미래의 시장 움직임을 예측한다. 이는 시장 참여자들의 심리와 집단 행동을 반영하며, 매매 결정을 위한 규칙 기반 기준을 제공한다.

헤드 앤 숄더 패턴은 상승세에서 하락세로의 전환을 예측하는 데 사용되는 기술적 분석 차트 패턴이다. 이 패턴은 다음과 같은 독특한 형태로 나타난다:

* 왼쪽 어깨: 가격이 정점까지 상승한 뒤 하락
* 머리: 왼쪽 어깨 이후 더 높은 정점에 도달한 뒤 다시 하락
* 오른쪽 어깨: 머리보다 낮은 수준으로 상승 후 다시 하락
* 넥라인: 왼쪽 어깨와 오른쪽 어깨의 저점을 연결한 지지선 역할

이 패턴을 활용한 일반적인 거래 방법은 오른쪽 어깨 이후 가격이 넥라인 아래로 종가 기준으로 떨어질 때 자산을 공매도하는 것이다. 아래 그림 6.52를 참고하라.

<img src="./images/fig_06_52.png" width=800>

Figure 6.52 헤드 앤 숄더 패턴 및 일반적인 진입 시점
이러한 트레이딩 패턴을 감지하고 활용하는 것은 트레이딩 성과와 수익성을 크게 향상시킬 수 있다. 본 알고리즘은 일차원 CNN을 사용하여 H\&S 패턴을 감지한다. CNN은 데이터 내 복잡한 패턴을 식별하고, 원시 가격 데이터에서 관련 특징을 자동으로 추출하며, 노이즈를 걸러내는 데 뛰어나기 때문에 선택되었다.

특히 글로벌 이벤트에 민감하게 반응하는 외환 시장은 가격 변동성이 높아 이 패턴이 자주 나타나며 신뢰도 또한 높다. 이를 활용해 수익 기회를 극대화하는 것이 본 알고리즘의 목적이다.

### Model

모델은 종가 데이터 25개 포인트를 특징으로 사용한다.
예측 레이블은 다음 두 가지 클래스의 확률이다:

* 클래스 $0$ = 패턴 없음
* 클래스 $1$ = 패턴 존재

모델
일차원 CNN

이 순차적 CNN(계층을 순서대로 추가) 아키텍처는 다음과 같다:

* 입력 레이어: 25개의 데이터 포인트(특징)를 입력받으며, 각 특징은 고유한 채널로 처리된다.
* 일차원 합성곱 레이어: 커널 크기 5의 필터 32개가 데이터에 슬라이딩되며, relu 활성화 함수는 비선형적이고 복잡한 패턴 학습을 돕는다.
* 맥스 풀링 레이어: 출력 차원을 2로 축소하고 각 풀 영역에서 최대값을 취해 스케일 불변성과 계산 효율성을 확보한다.
* 플래튼 레이어를 통해 출력을 평탄화하여 출력 레이어에 연결
* 출력 레이어는 이진 분류 문제에 표준으로 사용되는 sigmoid 활성화 함수를 사용

Keras 구현:

```python
Model = Sequential()
model.add(Input(shape=(X_train.shape[1], 1)))
model.add(Conv1D(filters=32, kernel_size=5, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))
```
### Trading Universe

우리는 USD/CAD 통화쌍에만 관심을 둔다.

### Portfolio Construction

| 모델            | 우리는 모델을 연구 노트북에서 합성 데이터로 한 번만 학습시킨다.                                                                                                           |
| :------------ | :--------------------------------------------------------------------------------------------------------------------------------------------- |
| 학습 시간         | 연구 노트북에서 한 번만 학습됨                                                                                                                              |
| 포트폴리오 리밸런싱 시점 | 각 모니터링된 종가가 도착할 때마다 수행됨                                                                                                                        |
| 포트폴리오 비중      | 과거 25\~100일(10일 단위) 가격 경로를 수집하고, 각 경로를 25개의 데이터 포인트로 다운샘플한 뒤 CNN에 입력함. 모델이 해당 샘플이 헤드 앤 숄더 패턴을 포함할 확률이 50% 이상이라고 판단하면, 해당 외환쌍을 10,000 USD 공매도함. |

### Trading Logic

백테스트를 위해 2019년 1월 1일부터 2024년 1월 1일까지의 USD/CAD 종가 데이터를 수집한다.
거래 알고리즘은 매일 새로운 USD/CAD 종가에 반응한다:

* 각 시점에서 다양한 lookback 윈도우 크기로 히스토리를 슬라이스함. 최소 윈도우 크기는 CNN 입력에 맞춘 25개의 종가이며, 최대는 100개의 종가까지이며, 10씩 증가함.

학습된 CNN 모델은 25개의 데이터 포인트를 입력으로 받지만, H\&S 패턴은 더 긴 기간에 걸쳐 발생할 수 있기 때문에, 입력 데이터를 25개의 포인트로 다운샘플하면서 가격 경로의 전체 형태는 유지해야 한다.
이 다운샘플링은 $1920 \times 1080$ 이미지를 $1280 \times 720$으로 줄이는 것과 비슷하며, 시각적으로 거의 같지만 데이터 양은 줄어든다.

이러한 다운샘플링은 다양한 lookback 윈도우 크기에 대해 패턴 존재 여부를 탐색하기 위해 사용된다.
각 윈도우의 데이터를 25개로 줄인 뒤 정규화하여 CNN에 입력하여 예측을 얻는다.

* 모델 출력은 각 클래스(클래스 $0$ = 패턴 없음, 클래스 $1$ = 패턴 존재)의 확률임.
* 각 lookback 윈도우에서 클래스 1의 확률이 50% 이상이면, 10,000 USD 공매도를 실행하고 10일 동안 포지션을 유지함.

```python
def initialize(self):
    self.set_start_date(2019, 1, 1)
    self.set_end_date(2024, 4, 1)
    self.set_cash(100_000)
    self._security = self.add_forex("USDCAD", Resolution.DAILY)
    self._symbol = self._security.symbol
    self._max_size = self.get_parameter('max_size', 100)
    self._step_size = self.get_parameter('step_size', 10)
    self._confidence_threshold = self.get_parameter('confidence_threshold', 0.5)  # 0.5 => 50%
    self._holding_period = timedelta(self.get_parameter('holding_period', 10))
    self._model = load_model(self.object_store.get_file_path("head-and-shoulders-model.keras"))
    self._trailing_prices = pd.Series()
    self._liquidation_quantities = []
```

```python
def on_data(self, data):
    t = self.time
    price = data[self._symbol].close
    # 과거 가격 업데이트
    self._trailing_prices.loc[t] = price
    self._trailing_prices = self._trailing_prices.iloc[-self._max_size:]
    
    quantity = 0
    
    for size in range(self._min_size, self._max_size + 1, self._step_size):
        if len(self._trailing_prices) < size:
            continue
        window_trailing_prices = self._trailing_prices.iloc[-size:]
        
        low_res_window = downsample(window_trailing_prices.values)
        
        factors = np.array(
            ((low_res_window - low_res_window.mean()) / low_res_window.std()).reshape(1, self._min_size, 1)
        )
        
        prediction = self._model.predict(factors, verbose=0)[0][0]
        
        if prediction > self._confidence_threshold:
            self.log(
                f"{t}: Pattern detected between {window_trailing_prices.index[0]} and "
                f"{window_trailing_prices.index[-1]} with {round(prediction * 100, 1)}% confidence."
            )
            quantity -= 10_000

    if quantity:
        self._cad_before_sell = self.portfolio.cash_book['CAD'].amount
        self.market_order(self._symbol, quantity)
        
        t_exit = t + self._holding_period
        self.schedule.on(
            self.date_rules.on(t_exit.year, t_exit.month, t_exit.day),
            self.time_rules.at(t_exit.hour, t_exit.minute),
            self._liquidate_position
        )
```

```python
def _liquidate_position(self):
    quantity = round(self._liquidation_quantities.pop(0) / self._security.ask_price)
    if quantity:
        self.market_order(self._symbol, quantity)

def on_order_event(self, order_event):
    if (order_event.status == OrderStatus.FILLED and order_event.direction == OrderDirection.SELL):
        self._liquidation_quantities.append(
            self.portfolio.cash_book['CAD'].amount - self._cad_before_sell
        )
```
### Tearsheet

결과는 다음과 같다:

* 샤프 비율은 보유 기간이 길수록 증가한다.
* 샤프 비율은 신뢰도 임계값이 높아질수록 감소한다(아마도 거래 수가 줄기 때문).
* 모든 파라미터 조합이 수익을 냈다.
* 그림 6.53과 6.54 참고.

**신뢰도 임계값(confidence\_threshold) 파라미터:**

* 최소값은 0.3으로 설정. 이보다 낮으면 $<30 %$ 확률로 패턴이 존재한다고 판단하는 것이므로 신뢰도가 낮음.
* 최대값은 0.9. 이미 거래 수가 적기 때문.
* 스텝 크기는 0.1로, 깔끔한 값 구간을 만들기 위해 설정.

**보유 기간(holding\_period) 파라미터:**

* 최소값은 2. 일별 데이터 기준 가장 짧은 보유 기간.
* 최대값은 10. 경험 기반으로 설정.
* 스텝 크기는 1. 가장 작은 스텝 크기.

**백테스트 파라미터:**

* `max_size`: 100
* `step_size`: 10
* `confidence_threshold`: 0.8
* `holding_period`: 10

<img src="./images/fig_06_53.png" width=800>

**그림 6.53** 예제 17의 누적 수익곡선, 성과 지표, 사용자 정의 시각화

<img src="./images/fig_06_54.png" width=800>

**그림 6.54** 예제 17의 월별 수익률, 위기 이벤트, 민감도 분석

### Implementation Insights

신경망은 연구 노트북에서 학습하였다.

### Step 1: 합성 입력 데이터셋 생성**

우리는 헤드 앤 숄더 패턴 샘플 100,000개를 생성하는 것으로 시작한다.

<img src="./images/fig_06_55.png" width=800>
**Figure 6.55** 합성된 H\&S 패턴의 선 그래프

각 샘플은 25개의 데이터 포인트로 구성되며, 학습 데이터셋에서 고유하게 만들기 위해 랜덤 요소가 포함된다. 학습 전 모든 샘플을 정규화한다.

```python
from scipy.stats import norm, uniform
np.random.seed(1)
ref_count = 100_000
v1 = np.array([0] * ref_count) + 0.02 * norm.rvs(size=(ref_count,))
p1 = np.array([1] * ref_count) + 0.2 * norm.rvs(size=(ref_count,))
v2 = v1 + 0.2 * norm.rvs(size=(ref_count,))
v3 = v1 + 0.2 * norm.rvs(size=(ref_count,))
p3 = p1 + 0.02 * norm.rvs(size=(ref_count,))
p2 = 1.5 * np.maximum(p1, p3) + abs(uniform.rvs(size=(ref_count,)))
v4 = v1 + 0.02 * norm.rvs(size=(ref_count,))
```

```python
ref = pd.DataFrame([
    v1,
    (v1*0.75 + p1*0.25) + 0.2 * norm.rvs(size=(ref_count,)),
    (v1 + p1)/2 + 0.2 * norm.rvs(size=(ref_count,)),
    (v1*0.25 + p1*0.75) + 0.2 * norm.rvs(size=(ref_count,)),
    p1,
    (v2*0.25 + p1*0.75) + 0.2 * norm.rvs(size=(ref_count,)),
    (v2 + p1)/2 + 0.2 * norm.rvs(size=(ref_count,)),
    (v2*0.75 + p1*0.25) + 0.2 * norm.rvs(size=(ref_count,)),
    v2,
    (v2*0.75 + p2*0.25) + 0.2 * norm.rvs(size=(ref_count,)),
    (v2 + p2)/2 + 0.2 * norm.rvs(size=(ref_count,)),
    (v2*0.25 + p2*0.75) + 0.2 * norm.rvs(size=(ref_count,)),
    p2,
    (v3*0.25 + p2*0.75) + 0.2 * norm.rvs(size=(ref_count,)),
    (v3 + p2)/2 + 0.2 * norm.rvs(size=(ref_count,)),
    (v3*0.75 + p2*0.25) + 0.2 * norm.rvs(size=(ref_count,)),
    v3,
    (v3*0.75 + p3*0.25) + 0.2 * norm.rvs(size=(ref_count,)),
    (v3 + p3)/2 + 0.2 * norm.rvs(size=(ref_count,)),
    (v3*0.25 + p3*0.75) + 0.2 * norm.rvs(size=(ref_count,)),
    p3,
    (v4*0.25 + p3*0.75) + 0.2 * norm.rvs(size=(ref_count,)),
    (v4 + p3)/2 + 0.2 * norm.rvs(size=(ref_count,)),
    (v4*0.75 + p3*0.25) + 0.2 * norm.rvs(size=(ref_count,)),
    v4
])
ref = ((ref - ref.mean()) / ref.std()).T
positive_samples = []
for _, row in ref.iterrows():
    positive_samples.append(row.values)
```

우리는 v1, p1, v2, p2, v3, p3, v4 등의 점들의 y좌표를 생성하고, 각 연속된 점 사이에 선형 보간된 점 3개를 추가해 총 25개의 포인트로 구성한다.

예를 들어 v1과 p1 사이에는 다음과 같은 3개의 보간 점이 포함된다:

```python
(v1*0.75 + p1*0.25) + 0.2 * norm.rvs(size=(ref_count,)),
(v1 + p1)/2 + 0.2 * norm.rvs(size=(ref_count,)),
(v1*0.25 + p1*0.75) + 0.2 * norm.rvs(size=(ref_count,))
```

정규분포 기반의 랜덤 노이즈로 이런 패턴을 10만 개 생성한다.

그 다음, 25개 포인트를 가진 랜덤워크로 구성된 음성 샘플 10만 개도 생성한다.

```python
equity_curves = (np.random.randn(ref.shape[0], ref.shape[1]) / 1000 + 1).cumprod(axis=1)
negative_samples = (equity_curves - np.mean(equity_curves, axis=1, keepdims=True)) / np.std(equity_curves, axis=1, keepdims=True)
```

데이터셋은 양성 → 음성 → 양성 → 음성 순으로 번갈아가며 구성한다.

```python
X = np.array(
    [value for pair in zip(positive_samples, negative_samples) for value in pair]
)
y = np.array([1 if i % 2 == 0 else 0 for i in range(len(X))])
```

### Step 2: 입력 데이터셋을 학습/테스트 세트로 분할**

전체 20만 개 샘플을 80:20 비율로 학습 및 테스트용으로 나눈다.

```python
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Input
from keras.utils import set_random_seed

set_random_seed(0)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
```
### Step 3: Create the neural network model.

순차 신경망 모델을 구현한다.

```python
model = Sequential()
model.add(Input(shape=(X_train.shape[1], 1)))
model.add(Conv1D(filters=32, kernel_size=5, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))
```

### Step 4: Compile, train, and evaluate the model.

모델을 컴파일하고, 학습시키며, 테스트 세트에서 평가한다. 테스트 세트에서 정확도는 $99.9%$에 도달한다.

```python
# 모델 컴파일
model.compile(
    optimizer='adam', loss='binary_crossentropy', metrics=['accuracy']
)

# 모델 학습
model.fit(
    X_train, y_train, epochs=10, batch_size=16,
    validation_data=(X_test, y_test)
)

# 모델 평가
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f'Test Accuracy: {test_acc}')
```

**Step 5:** 모델을 객체 저장소에 저장한다.

```python
model.save(qb.object_store.get_file_path("head-and-shoulders-model.keras"))
```

모델을 저장하면 백테스트에서 사용할 수 있다.


---

## Example 18—Amazon Chronos Model

| 예측 대상 | 향후 가격 경로              |
| :---- | :-------------------- |
| 기술    | Amazon Chronos        |
| 자산군   | 미국 주식                 |
| 난이도   | 어려움                   |
| 유형    | 전체 전략                 |
| 소스코드  | qnt.co/book-example18 |

### Summary

이 예제는 HuggingFace의 사전학습된 모델 "amazon/chronos-t5-tiny"를 사용하여 시장에서 가장 유동성이 높은 5개 자산의 향후 성과를 예측하는 방법을 보여준다. 예측 결과를 기반으로 알고리즘은 SciPy 패키지를 활용해 샤프 비율을 최대화하도록 포트폴리오 비중을 최적화한다. 포트폴리오는 3개월마다 최신 시장 예측과 조건에 맞춰 리밸런싱된다.

### Motivation

이 예제는 두 가지 강력한 요소를 결합한다: Amazon Chronos 모델의 고급 예측 기능과 샤프 비율을 활용한 강력한 포트폴리오 최적화.

HuggingFace의 사전학습 모델인 "amazon/chronos-t5-tiny"는 시계열 예측에서 뛰어난 성능을 보여주며, 학습에 드는 시간과 자원을 줄이고 빠른 배포 및 즉시 적용이 가능하다. 또한 GPU 등 고성능 하드웨어에서 대용량 데이터 처리도 가능하다. 전통적인 ARIMA 모델이나 LSTM 같은 다른 AI 기반 솔루션과 비교했을 때, Amazon Chronos는 금융 시장 예측에 특화되어 있다.

이 알고리즘은 샤프 비율을 사용해 수익 대비 위험이 최적화된 포트폴리오를 구성하며, 이는 현대 포트폴리오 이론의 모범 사례에 부합한다. 단, 샤프 비율은 수익률이 정규분포를 따른다고 가정하기 때문에, 현실의 극단적 사건이나 꼬리 위험을 반영하지 못할 수 있다. 이와 달리 Sortino 비율(하방 위험 중심), VaR(최대 손실 예측), 최대 낙폭(MDD) 등은 서로 다른 관점을 제공하지만, 샤프 비율만큼 수익과 위험을 종합적으로 통합하진 못한다.

### Model

* **모델 입력 특징:** 일일 종가
* **예측 레이블:** 향후 3개월 간 (3 × 21일) 일일 종가
* **모델:** Amazon Chronos

Amazon Chronos는 금융 시장 예측을 위해 특화된 최신 기계 학습 모델로, 높은 정확도, 확장성, 사용 편의성을 제공한다.

이 예제에서는 베이스 모델과 파인튜닝된 Amazon Chronos 모델을 사용한 두 가지 거래 전략을 구현한다.

대안이 되는 주요 예측 모델들은 다음과 같다:

* **ARIMA (자기회귀 누적 이동 평균):** 선형 패턴이 있는 정상 시계열 데이터에 효과적임.
* **Prophet (Meta/Facebook 제공):** 계절성과 추세를 잘 처리함.
* **LSTM (Long Short-Term Memory):** 순차 데이터의 장기 의존성을 잘 포착하는 순환 신경망(RNN) 계열 모델.
* **SVM (Support Vector Machines):** 고차원 데이터를 다루며 커널 함수로 비선형 관계도 잘 포착하는 기계 학습 모델.
* **XGBoost:** 테이블 형태 데이터에 강하며, 시계열 예측에 맞게 커스터마이징 가능.

어떤 예측 모델이 적합한지는 사용 사례에 따라 다르며, 다양한 대안 모델을 실험해보는 것이 권장된다.
### Trading Universe

매월 초, 알고리즘은 S\&P 500 지수를 추종하는 ETF인 SPY 내에서 유동성이 가장 높은 상위 5개 자산을 유니버스로 선택한다.

### Portfolio Construction

| 모델 학습 시간      | 첫 번째 트레이딩 전략은 학습되지 않은 기본 Amazon Chronos 모델을 사용한다.     |
| :------------ | :---------------------------------------------------- |
|               | 두 번째 트레이딩 전략은 매월 첫 거래일 자정에 Amazon Chronos 모델을 파인튜닝한다. |
| 포트폴리오 리밸런싱 시간 | 매 분기마다 첫 거래일 자정에 리밸런싱한다.                              |
| 포트폴리오 비중      | 향후 샤프 비율을 최대화하는 방식으로 비중을 설정한다.                        |

### Trading Logic

첫 번째 트레이딩 전략은 유니버스 내 자산들의 지난 1년간 종가 이력을 다운로드하고, 향후 3개월 동안의 각 날짜에 대해 예측된 가격의 중앙값을 계산한다. 이후 포트폴리오 최적화 결과로부터 샤프 비율을 최대화하는 방식으로 포트폴리오 비중을 설정한다.

두 번째 트레이딩 전략은 첫 번째 전략을 확장한 것으로, 향후 3개월의 종가를 예측하기 전에 Amazon Chronos 모델을 재학습한다.

우리가 수행하는 포트폴리오 최적화는 주어진 리스크 수준에서 수익을 최대화하는 자산 조합을 찾는 것이다.

형식적으로는 다음과 같은 목적함수를 최대화한다:

$$
\max \left(\frac{R_{p}-R_{f}}{\sigma_{p}}\right)
$$

다음 제약 조건 하에서:

* 포트폴리오 비중의 합은 1: $\sum w\_{i}=1$
* 공매도 금지: $0 \leq w\_{i} \leq 1$

여기서:

* $R\_{p}$: 포트폴리오 수익률
* $R\_{f}$: 무위험 이자율 (FOMC 기준금리 사용)
* $\sigma\_{p}$: 포트폴리오 수익률의 표준편차
* $w\_{i}$: 포트폴리오 내 각 자산의 비중

Python에서는 SciPy의 `minimize` 함수를 사용해 문제를 해결한다.
최대화 문제를 목적함수에 -1을 곱해 최소화 문제로 전환한 후, Sequential Least Squares Programming 알고리즘을 사용하여 제약 조건을 만족하면서 최적의 비중을 구한다.

### Tearsheet

사전 학습된 HuggingFace 모델은 2019년 1월 1일 이후의 데이터로 학습되었을 경우, 미래 정보 반영으로 인한 룩어헤드 바이어스가 있을 수 있다.

**트레이딩 전략 1 (기본 모델):**

* Figure 6.56 및 6.57 참고

<img src="./images/fig_06_56.png" width=800>

Figure 6.56 예제 18.1의 누적 수익곡선 및 성과 지표

<img src="./images/fig_06_57.png" width=800>

Figure 6.57 예제 18.1의 월별 수익률 및 위기 이벤트

**트레이딩 전략 2 (파인튜닝 모델):**

* `context_length` 인자는 문맥 윈도우 길이를 정의하며, 과거 토큰 중 몇 개를 기반으로 다음 토큰을 예측할지 결정한다. 기본값은 512지만, 각 자산에 대해 252개(1년치)의 데이터만 제공되므로 126(6개월)로 줄였다.
* `max_steps` 인자는 최대 학습 반복 수를 정의한다. 기본값은 200,000이지만 알고리즘이 10분 내 학습을 완료하도록 3으로 설정했다.
* 그림 6.58 및 6.59 참고

<img src="./images/fig_06_58.png" width=800>]

Figure 6.58 예제 18.2의 누적 수익곡선 및 성과 지표

<img src="./images/fig_06_59.png" width=800>

Figure 6.59 예제 18.2의 월별 수익률 및 위기 이벤트

### Implementation Insights

Amazon Chronos 모델은 각 미래 종가에 대해 여러 예측값을 출력하며, 각 값에는 해당 확률이 부여된다. 간단히 하기 위해, 우리는 중앙값 예측값을 사용한다.

```python
# 과거 주가 곡선 가져오기
history = self.history(symbols, self._lookback_period)['close'].unstack(0)

# 미래 주가 곡선 예측
all_forecasts = self._pipeline.predict(
    {
        torch.tensor(history[symbol].dropna())
        for symbol in symbols
    },
    self._prediction_length
)
```

```python
# 각 자산에 대해 중앙값 예측값 추출
forecasts_df = pd.DataFrame(
    {
        symbol: np.quantile(all_forecasts[i].numpy(), 0.5, axis=0)  # 0.5 = median
        for i, symbol in enumerate(symbols)
    }
)
```

여기서 모델은 사전학습된 기본 모델일 수 있다:

```python
# 사전학습 모델 불러오기
self._pipeline = ChronosPipeline.from_pretrained(
    "amazon/chronos-t5-tiny",
    device_map="cuda" if torch.cuda.is_available() else "cpu",
    torch_dtype=torch.bfloat16,
)
```

또는 이 예제에서 실제 데이터를 사용하여 학습한 파인튜닝 모델일 수 있다:

```python
# 학습 데이터 수집
training_data_by_symbol = {}
for symbol in symbols:
    df = history[[symbol]].dropna()
    if df.shape[0] < 10:  # 데이터가 너무 적으면 제외
        continue
    adjusted_df = df.reset_index()[['time', symbol]]
    adjusted_df = adjusted_df.rename(columns={str(symbol.id): 'target'})
    adjusted_df['time'] = pd.to_datetime(adjusted_df['time'])
    adjusted_df.set_index('time', inplace=True)
    adjusted_df = adjusted_df.resample('D').asfreq()
    training_data_by_symbol[symbol] = adjusted_df

tradable_symbols = list(training_data_by_symbol.keys())
```

```python
# 모델 파인튜닝
output_dir_path = self._train_chronos(
    list(training_data_by_symbol.values()),
    context_length=int(252/2),  # 6개월
    prediction_length=self._prediction_length,
    optim=self._optimizer,
    model_id=self._model_name,
    output_dir=self._model_path,
    learning_rate=1e-5,
    tf32=False,  # Ampere GPU 필요 (예: A100)
    max_steps=3
)

# 파인튜닝된 모델 불러오기
pipeline = ChronosPipeline.from_pretrained(
    output_dir_path,
    device_map=self._device_map,
    torch_dtype=torch.bfloat16,
)
```

`_train_chronos` 메서드는 실제 학습을 수행하며,
포트폴리오 최적화는 예측된 샤프 비율을 최대화하도록 구성된다:

```python
optimal_weights = self._optimize_portfolio(forecasts_df)
```

`_optimize_portfolio`는 SciPy의 `minimize` 함수를 호출한다:

```python
def _sharpe_ratio(self, weights, returns, risk_free_rate, trading_days_per_year=252):
    # 샤프 비율 계산 방식 정의
    mean_returns = returns.mean() * trading_days_per_year
    cov_matrix = returns.cov() * trading_days_per_year
    portfolio_return = np.sum(mean_returns * weights)
    portfolio_std = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))
    sharpe_ratio = (portfolio_return - risk_free_rate) / portfolio_std
    # 최소화 함수에 사용하기 위해 음수로 반환
    return -sharpe_ratio
```

```python
def _optimize_portfolio(self, equity_curves):
    returns = equity_curves.pct_change().dropna()
    num_assets = returns.shape[1]
    initial_guess = num_assets * [1. / num_assets,]
    
    result = minimize(
        self._sharpe_ratio,
        initial_guess,
        args=(
            returns,
            self.risk_free_interest_rate_model.get_interest_rate(self.time)
        ),
        method='SLSQP',
        bounds=tuple((0, 1) for _ in range(num_assets)),
        constraints=(
            {'type': 'eq', 'fun': lambda weights: np.sum(weights) - 1}
        )
    )
    return result.x
```

---

## Example 19—FinBERT Model

| Predicting  | 뉴스 감성 분석              |
| :---------- | :-------------------- |
| Technology  | LLM                   |
| Asset Class | 미국 주식                 |
| Difficulty  | 어려움                   |
| Type        | 전체 전략                 |
| Source Code | qnt.co/book-example19 |

### Summary

이 알고리즘은 사전 학습된 "ProsusAI/finbert" HuggingFace 모델을 활용하여 자산과 관련된 최신 뉴스 발표의 감성을 평가하는 방법을 보여줍니다. 매월 초, 알고리즘은 상위 10개의 유동성이 높은 자산 중 가장 변동성이 큰 자산을 선택하고, 지난 10일간의 뉴스 발표를 기반으로 집계된 감성 점수를 계산합니다. 긍정적인 감성이 부정적인 감성보다 강하면 롱 포지션을 취하고, 그렇지 않으면 숏 포지션에 진입합니다. Example 18과 유사하게, 기본 모델을 시연한 후, 모델을 파인튜닝합니다.

### Motivation

고급 자연어 처리(NLP) 모델을 활용하여 시장 감성을 분석하는 것은 향후 트렌드를 포착하는 데 중요한 우위를 제공할 수 있습니다. 일반적으로 긍정적인 감성은 가격 상승을, 부정적인 감성은 하락 가능성을 의미합니다.

이 알고리즘은 HuggingFace의 사전 학습된 "ProsusAI/finbert" 모델을 사용하여 S\&P 500 지수 내 상위 10개 유동성 자산 중 가장 변동성이 큰 자산에 대한 최신 뉴스 발표의 감성을 평가합니다. 변동성은 잠재적으로 큰 가격 변동의 지표로 활용되며, 알고리즘은 가장 높은 수익 가능성이 있는 자산을 타깃으로 하여 이를 실현하고자 합니다. 또한 유동성이 높은 자산을 선택함으로써 포지션 진입 및 청산이 용이해져 슬리피지 및 거래 비용을 최소화할 수 있습니다.

### Model

| Model     | 뉴스 기사                                                           |
| :-------- | :-------------------------------------------------------------- |
| Features  |                                                                 |
| Predicted | 각 감성 범주(긍정, 중립, 부정)에 대한 확률                                      |
| Label     | 긍정 감성은 자산에 대한 긍정적인 전망을, 부정 감성은 부정적인 전망을, 중립 감성은 중립적인 전망을 나타냅니다. |
|           |                                                                 |

Example 16에서는 OpenAI의 GPT-4 모델을 사용한 감성 분석을 다루었습니다. FinBERT나 BERT와 같은 모델과 GPT-4를 비교하면 다음과 같은 성능 및 비용 차이가 있습니다:

* 성능 — FinBERT는 금융 감성 분석에 특화되어 파인튜닝된 BERT의 변형으로, 금융 뉴스의 맥락과 뉘앙스를 잘 이해하는 데 탁월합니다. FinBERT 및 BERT 모델은 GPT-4와 같은 대형 모델보다 작기 때문에 일반적으로 추론 속도가 빠릅니다.

반면 GPT-4는 범용 모델로서 더 깊은 문맥 이해가 가능하고, 뉴스 기사에 대한 보다 정교한 해석을 생성할 수 있지만, 이 과정에서 더 많은 계산 자원과 시간이 소요됩니다.

* 비용 — FinBERT 및 BERT 모델은 훈련 및 추론 시 필요한 계산량이 적기 때문에 비용 효율적이며, 구현도 상대적으로 간단해 개발 시간 및 비용이 줄어듭니다.

반대로 GPT-4는 모델 크기와 복잡성이 높아 계산 자원이 많이 필요하고, 운영 비용도 더 높습니다. 또한 금융 감성 분석에 GPT-4를 활용하기 위해서는 추가적인 파인튜닝이 필요할 수 있으며, 이로 인해 개발 비용이 더욱 증가할 수 있습니다.

우리는 이 트레이딩 전략을 사전 학습된 FinBERT 기본 모델과 실제 뉴스 기사 및 해당 이후 주가 수익률을 바탕으로 감성을 재학습시킨 파인튜닝 모델 양쪽에 구현합니다.

### Trading Universe

매월 첫 번째 거래일에, 우리는 S\&P 지수 구성 종목 중 상위 10개 유동성 자산에서 가장 변동성이 큰 주식을 선택합니다.
### Portfolio Construction

| 모델 학습 시점              | 포트폴리오 리밸런싱 시점  | 포트폴리오 비중                                                                          |
| --------------------- | -------------- | --------------------------------------------------------------------------------- |
| 기본 모델은 추가 학습을 수행하지 않음 | 매월 첫 번째 거래일 자정 | 집계된 감성이 긍정적일 가능성이 높으면 포트폴리오의 100%를 해당 자산에 투자, 그렇지 않으면 포트폴리오의 1/4을 해당 자산에 숏 포지션 취함 |

### Trading Logic

기본 FinBERT 모델에서는 TiingoNews에서 유니버스 내 자산과 관련된 최근 10일간의 뉴스 기사를 가져와 토크나이징한 후 사전 학습된 모델에 적용합니다.

```python
# 대상 종목 가져오기
security = self.securities[list(self._universe.selected)[0]]

# 최신 뉴스 기사 가져오기
articles = self.history[TiingoNews](security.dataset_symbol, 10, Resolution.DAILY)
article_text = [article.description for article in articles]
if not article_text:
    return

# 입력 문장 준비
inputs = self._tokenizer(article_text, padding=True, truncation=True, return_tensors='tf')

# 모델 출력 가져오기
outputs = self._model(**inputs)

# 소프트맥스를 적용하여 확률 얻기
scores = tf.nn.softmax(outputs.logits, axis=-1).numpy()
self.log(f"{str(scores)}")

# 감성 점수 집계
scores = self._aggregate_sentiment_scores(scores)

# 포트폴리오 리밸런싱
weight = 1 if scores[2] > scores[0] else -0.25
self.set_holdings(security.symbol, weight, True)
self._last_rebalance_time = self.time
```

`_aggregate_sentiment_scores`는 최신 뉴스일수록 더 높은 가중치를 부여하여 지수 가중 평균 점수를 계산하고, 이를 단일 점수로 집계합니다.

파인튜닝된 FinBERT 모델에서는 최근 30일간의 종가와 뉴스 기사를 활용합니다.

학습 데이터셋을 구성하는 과정에서 각 날짜별 감성 레이블은 다음 논리를 통해 생성됩니다:

1. 종가의 일일 수익률을 계산합니다.
2. 양의 수익률을 내림차순으로 정렬한 후 상위 75%에 긍정 레이블을 부여합니다.
3. 음의 수익률을 오름차순으로 정렬한 후 상위 75%에 부정 레이블을 부여합니다.
4. 나머지 값에는 중립 레이블을 부여합니다.

이 학습 데이터셋을 사용하여 모델을 재학습한 후, 기본 모델과 동일한 방식으로 진행합니다.

### Tearsheet

사전 학습된 HuggingFace 모델은 2019년 1월 1일 이후의 데이터를 기반으로 학습되었을 경우 선견 편향이 포함되어 있을 수 있습니다.

**트레이딩 전략 1 (기본 모델)**

* Figure 6.60 및 6.61 참조

<img src="./images/fig_06_60.png" width=800>

Figure 6.60: Example 19.1의 자산 곡선, 성과 차트, 커스텀 차트

<img src="./images/fig_06_61.png" width=800>

Figure 6.61: Example 19.1의 월별 수익률 및 위기 이벤트

**트레이딩 전략 2 (파인튜닝 모델)**

* Figure 6.62 및 6.63 참조

<img src="./images/fig_06_62.png" width=800>

Figure 6.62: Example 19.2의 자산 곡선, 성과 차트, 커스텀 차트

<img src="./images/fig_06_63.png" width=800>

Figure 6.63: Example 19.2의 월별 수익률 및 위기 이벤트

### Implementation Insights

FinBERT는 데이터 분할, 가중치 초기화, 학습을 위해 난수 생성기를 사용합니다. 재현 가능성을 위해 초기 랜덤 시드를 설정하는 것이 좋습니다. 예를 들어 다음과 같이 설정할 수 있습니다:
`set_seed(1, True)`
감성 점수는 지수 가중치를 사용하여 집계되며, 최신 뉴스 감성이 가장 큰 영향을 미치도록 설계되어 있습니다.

```python
def _aggregate_sentiment_scores(self, sentiment_scores):
    n = sentiment_scores.shape[0]
    # 지수적으로 증가하는 가중치 생성
    weights = np.exp(np.linspace(0, 1, n))

    # 가중치를 정규화하여 합이 1이 되도록 함
    weights /= weights.sum()

    # 감성 점수에 가중치를 적용
    weighted_scores = sentiment_scores * weights[:, np.newaxis]

    # 가중 점수를 합산하여 집계 점수 생성
    aggregated_scores = weighted_scores.sum(axis=0)
    return aggregated_scores
```

\*\*트레이딩 전략 2 (파인튜닝 모델)\*\*에서는 학습 데이터를 생성하고 모델을 학습한 후, 트레이딩 전략 1과 동일한 방식으로 진행합니다.

```python
security = self.securities[list(self._universe.selected)[0]]

# 모델 파인튜닝용 샘플 생성
samples = pd.DataFrame(columns=['text', 'label'])
news_history = self.history(security.dataset_symbol, 30, Resolution.DAILY)
if news_history.empty:
    return

news_history = news_history.loc[security.dataset_symbol]['description']
asset_history = self.history(
    security.symbol, timedelta(30), Resolution.SECOND
).loc[security.symbol]['close']

for i in range(len(news_history.index) - 1):
    # 뉴스 기사 텍스트 추출
    factor = news_history.iloc[i]
    if not factor:
        continue

    # 뉴스 발표 시간 변환
    release_time = self._convert_to_eastern(news_history.index[i])
    next_release_time = self._convert_to_eastern(news_history.index[i + 1])

    # 뉴스 발표 후의 시장 반응 구간 설정
    reaction_period = asset_history[
        (asset_history.index > release_time) &
        (asset_history.index < next_release_time + timedelta(seconds=1))
    ]

    if reaction_period.empty:
        continue

    # 수익률 계산
    label = (
        (reaction_period.iloc[-1] - reaction_period.iloc[0]) /
        reaction_period.iloc[0]
    )

    # 학습 샘플 저장
    samples.loc[len(samples), :] = [factor, label]
```

```python
samples = samples.iloc[-100:]
if samples.shape[0] < 10:
    self.liquidate()
    return

# 시장 반응을 긍정/부정/중립 클래스로 분류
# 가장 부정적인 75% → class 0 (부정)
# 가장 긍정적인 75% → class 2 (긍정)
# 나머지 → class 1 (중립)

sorted_samples = samples.sort_values(by='label', ascending=False).reset_index(drop=True)
percent_signed = 0.75

positive_cutoff = int(percent_signed * len(sorted_samples[sorted_samples.label > 0]))
negative_cutoff = len(sorted_samples) - int(percent_signed * len(sorted_samples[sorted_samples.label < 0]))

sorted_samples.loc[list(range(negative_cutoff, len(sorted_samples))), 'label'] = 0
sorted_samples.loc[list(range(positive_cutoff, negative_cutoff)), 'label'] = 1
sorted_samples.loc[list(range(0, positive_cutoff)), 'label'] = 2
```

### Load the pre-trained model.

```python
model = TFBertForSequenceClassification.from_pretrained(
    self._model_name, num_labels=3, from_pt=True
)

# 모델 컴파일
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=3e-5),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
)

# 학습 데이터셋 생성
dataset = Dataset.from_pandas(sorted_samples)
dataset = dataset.map(
    lambda sample: self._tokenizer(
        sample['text'], padding='max_length', truncation=True
    )
)

dataset = model.prepare_tf_dataset(
    dataset, shuffle=True, tokenizer=self._tokenizer
)

# 모델 학습
model.fit(dataset, epochs=2)
```
